# Rogue Trader Space Combat

**Notes:**  
The purpose of the file is to create export table of the characteristics of the game space combat objects

In [1986]:
import pandas as pd
import numpy as np
import json
import os
import re
from bs4 import BeautifulSoup
import jsonpath_ng as jp
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 300)
import warnings
import openpyxl
warnings.filterwarnings('ignore')
import flatten_json as fj

### Functions

In [1987]:
# Fuction to show missed data in a table
def missing_values_tab(df):
    #Count missed values
    mis_val = df.isnull().sum()
    #Count share of missed values of all data 
    mis_val_percent = round(100 * df.isnull().sum() / len(df),2)
    
    mv_table = pd.concat([mis_val, mis_val_percent], axis=1)
    
    mv_table = mv_table.rename(columns = {0 : 'missed_values', 1 : '%_of_all'})
    
    mv_table['data_type'] = df.dtypes
    mv_table = mv_table[mv_table.iloc[:,1]!=0].sort_values(by='missed_values',ascending=False)
    print ("Dataframe contains " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " strings.\n")
    print("It has  " + str(mv_table.shape[0]) +" columns with missed values")    
    return display(mv_table) 

In [1988]:
def column_name_formatter(col):
    """used for delete prefixes 'Data_m_' in columns' names
    """ 
    if 'Data' in col:
        parts = col.split('_')
        new_name = parts[-1]
        return new_name
    else:
        return col

In [1989]:
def show_uniques(df):
    for col in df.columns[1:-1]:
        try:
            print(col, ':    ', df[col].unique(),'\n')
        except:
            continue

In [1990]:
# Функция для очистки строк от префикса '!bp_'
def clear_rows(row):
        
    if re.match('^\!bp_\w+', str(row)):
        tmp = row.split('_')
        new_value = tmp[1]        
        return new_value
    else:
        return row

In [1991]:
#Function to extract json into dataframes
def create_df(filelist):
    recipes_df = pd.DataFrame()
    dfs_list = []
    for file in filelist:
        with open(file) as user_file:
            file_contents = user_file.read()
            data = json.loads(file_contents)
            dic = (fj.flatten(data))
            df = pd.DataFrame.from_dict(dic, orient='index')
            dfs_list.append(df)
    for df in dfs_list:
        recipes_df = recipes_df.join(df, how = 'outer',lsuffix = 'x_',rsuffix = 'y_')
    #assign new column names
    column_numbers = list(range(len(recipes_df.columns)))
    recipes_df.columns = column_numbers
    #fill Nans
    #recipes_df = recipes_df.fillna(0)
    #remove prefix '!bp_'
    for col in recipes_df.columns:
        recipes_df[col] = recipes_df[col].apply(clear_rows)  
    #transpose dataframe
    recipes_df = recipes_df.T
    #-------------------------------------
    cols_to_drop = []
    for col in recipes_df.columns:
        if 'Visual' in col or 'FXSettings' in col: #drop columns with visual parameters
            cols_to_drop.append(col)
        elif 'Data_Comment' in col or 'PrototypeLink' in col:  #drop columns with dev comments
            cols_to_drop.append(col)
        elif 'Meta_ShadowDeleted' in col: #drop trash column
            cols_to_drop.append(col)
        elif 'JsonPath' in col or 'PropertyPath' in col or 'OwnerString' in col or 'assetguid' in col: #drop columns with fileways
            cols_to_drop.append(col)
        elif 'UI' in col or 'Sound' in col or 'Icon' in col: #drop columns with sound, ui properties
            cols_to_drop.append(col)
    recipes_df = recipes_df.drop(cols_to_drop, axis = 1)
    #add column Filepath
    recipes_df['Filepath'] = filelist
    return recipes_df

In [1992]:
# Function to get rid of characters between curly brackets
""" Sometimes not working correctly
def format_description(row):
    if '{' in row:
        row = re.sub(r'\s*{.*}\s*','',row)
    return row
"""
def format_description(row):      
    openining_braces = '\{'
    closing_braces = '\}'
    text_between = '.*?'
    x = re.sub(f'[{openining_braces}]{text_between}[{closing_braces}]','',row)
    #get rid of html tags
    x = re.sub('<[^>]*>',"",x)
    return x

In [1993]:
#Function to find columns with string keys
def match_pattern(row):
        if re.match('^\w{8}-\w{4}-\w{4}-\w{4}-\w{12}$', str(row)):
            return True   
        else:
            return False
   
        
match_pattern('ca24785c24384d142a6c22d7585bb3a6, BlueprintWeaponEnchantment')
"""
#finding columns with string keys
stringkey_cols = []
for col in weapon_enchants_df.columns:    
        a = weapon_enchants_df[col].apply(match_pattern).any()
        if a:            
            stringkey_cols.append(col)
print(stringkey_cols)   
    """

'\n#finding columns with string keys\nstringkey_cols = []\nfor col in weapon_enchants_df.columns:    \n        a = weapon_enchants_df[col].apply(match_pattern).any()\n        if a:            \n            stringkey_cols.append(col)\nprint(stringkey_cols)   \n    '

In [1994]:
#Function to find columns with asset ids
def find_assetIds_cols(row):
        if re.match('^[a-zA-Z0-9]{32}$', str(row)):
            return True   
        else:
            return False      

"""
#finding columns with asset ids
assetIds_cols = []
for col in weapon_enchants_df.columns:    
        a = weapon_enchants_df[col].apply(find_assetIds_cols).any()
        if a:            
            assetIds_cols.append(col)
print(assetIds_cols)   
    """

'\n#finding columns with asset ids\nassetIds_cols = []\nfor col in weapon_enchants_df.columns:    \n        a = weapon_enchants_df[col].apply(find_assetIds_cols).any()\n        if a:            \n            assetIds_cols.append(col)\nprint(assetIds_cols)   \n    '

In [1995]:
#Function to find columns with context actions, containing $ signs
def find_context_columns(row):
        if re.match('^\$[A-Za-z0-9]+\$\w{8}-\w{4}-\w{4}-\w{4}-\w{12}$', str(row)):
            return True   
        else:
            return False
"""
#find columns with rows containing $$
context_cols = []
for col in weapon_enchants_df.columns:    
        a = weapon_enchants_df[col].apply(find_context_columns).any()
        if a:            
            context_cols.append(col)
print(context_cols)   
    """

'\n#find columns with rows containing $$\ncontext_cols = []\nfor col in weapon_enchants_df.columns:    \n        a = weapon_enchants_df[col].apply(find_context_columns).any()\n        if a:            \n            context_cols.append(col)\nprint(context_cols)   \n    '

In [1996]:
def find_item_type(row):
    """Function to return item type from the column "Data_$type"
    """
    tmp = str(row).split(',')
    data_type = tmp[1]    
    parts = re.split('(?<=[a-z])(?=[A-Z])', data_type)
    item_type = parts[-1]   
    return item_type

In [1997]:
def create_text_columns(df, index_df, columns_in, column_out, format_text=False):
    """ Function to create text columns with names and descriptions got from 
        stringkeys in index file in the form of dict
        """
    # list of columns with string keys of items' names
    cols_names = columns_in
    #created column
    column_out = str(column_out )
    index_df = index_df # index_dict object got from parsing json file with text information
    index_dict = dict(zip(index_df['index'],index_df['text'])) #dict for mapping text information between two tables    
    
    #_____________________________Get items names and descriptions__________________________ 
    
    for col in cols_names:
        list_of_ids = df[col].unique() # list of stringkeys
        #filter out empty elements
        list_of_ids = list(filter(None,list_of_ids))
        for id in list_of_ids:
            try: #if stringkey is in index file return text
                df.loc[df[col]==id, column_out] = df[col].map(index_dict)
            except: #if not return empty row
                df.loc[df[col]==id, column_out] = ''          
    
         # fill missed values with empty row       
    df[column_out] = df[column_out].fillna('')
    # format description to get rid of {}
    if format_text == True:
        df[column_out] = df[column_out].apply(format_description)

   
    return df

In [1998]:
def find_property_cols(row):
    #if  re.match('^Data_Components_[0-9]_\$type$',row) or re.match('^Data_Components_[0-9]_m_(?:Fact|Facts)(?:$|_[0-9]$)',row):   
    if re.match('^Data_Components_[0-9]_(?:\$type$|m_(?:Fact|Facts)(?:$|_[0-9]$))',str(row)):
        return True
    else:
        return False

"""
#find columns with rows containing property values
property_cols = []
for col in equip_items_df.columns:    
        a = equip_items_df[col].apply(find_property_cols).any()
        if a:            
            property_cols.append(col)
print(property_cols)   
    """    

'\n#find columns with rows containing property values\nproperty_cols = []\nfor col in equip_items_df.columns:    \n        a = equip_items_df[col].apply(find_property_cols).any()\n        if a:            \n            property_cols.append(col)\nprint(property_cols)   \n    '

In [1999]:
def find_properties(row):
    #if  re.match('^Data_Components_[0-9]_\$type$',row) or re.match('^Data_Components_[0-9]_m_(?:Fact|Facts)(?:$|_[0-9]$)',row):   
    if re.match('^Data_Components_[0-9]_(?:\$type$|m_[A-Za-z]+(?:$|_[0-9]$))',str(row)):
        return True
    else:
        return False
def select_columns(df, keywords):
    """function to select columns to keep from a dataframe and return 4 lists of
        stringkey columns, assetids columns, misc columns and combined list of columns
    Arguments: df - dataframe to choose columns from, 
                keywords - list of keywords or names of columns to select for misc list    
    """
    selected_cols = [] # list of needed columns
    assetIds_cols = [] #list of columns with asset ids
    stringkey_cols = [] # list of columns with string keyes
    misc_cols = [] # list of misc columns
    property_cols = [] # list of property columns
    intro_cols = []   # first columns: AssetId and Data_$type in any table

    def find_ids(col):
        if re.match('^[a-z0-9]{32}(?:$|[, A-Za-z])', str(col)):
            return True
        else:
            return False      

    #find columns with asset ids
    for col in df.columns:
        if col !='AssetId':
            a = df[col].apply(find_assetIds_cols).any()
            if a:            
                assetIds_cols.append(col)
        else:
            intro_cols.append(col)

    #find columns with string keys
    for col in df.columns:    
        a = df[col].apply(match_pattern).any()
        if a:            
            stringkey_cols.append(col)
    
    #find columns with rows containing property values

    for col in df.columns: 
        # if name of the column is according to the pattern   
        if  re.match('^Data_Components_[0-9]_(?:\$type$|m_[A-Za-z]+(?:$|_[0-9]$))',str(col)):
            a = df[col].apply(find_ids).any()
            if a:                        
        #and its values are assetIds keys  add to property list                 
                property_cols.append(col) 
      
    # find miscellanious columns
    if len(keywords)>0:
        keywords = keywords
        for keyword in keywords:
            for col in df.columns:
                if keyword in col:
                    misc_cols.append(col)
    #else:
        #misc_cols = [d for d in df.columns if d not in assetIds_cols if d not in stringkey_cols if d not in property_cols]
    #update assetIds_cols
    assetIds_cols = [d for d in assetIds_cols if d not in property_cols]
    #Must- have columns, 'AssetId' and 'Data_$type'
    intro_cols.append(df.columns[1]) #add Data_$type
    #create combined list            
    selected_cols = intro_cols + assetIds_cols+ property_cols + stringkey_cols + misc_cols + ['Filepath']
      
    return selected_cols, assetIds_cols, stringkey_cols, property_cols, misc_cols

In [2000]:
def find_property_cols(row):
    #if  re.match('^Data_Components_[0-9]_\$type$',row) or re.match('^Data_Components_[0-9]_m_(?:Fact|Facts)(?:$|_[0-9]$)',row):   
    if re.match('^Data_Components_[0-9]_(?:\$type$|m_(?:Fact|Facts)(?:$|_[0-9]$))',str(row)):
        return True
    else:
        return False

"""
#find columns with rows containing property values
property_cols = []
for col in equip_items_df.columns:    
        a = equip_items_df[col].apply(find_property_cols).any()
        if a:            
            property_cols.append(col)
print(property_cols)   
    """    
       

'\n#find columns with rows containing property values\nproperty_cols = []\nfor col in equip_items_df.columns:    \n        a = equip_items_df[col].apply(find_property_cols).any()\n        if a:            \n            property_cols.append(col)\nprint(property_cols)   \n    '

In [2001]:
def find_item_columns(df):
    #Function to find columns with names like Data_Items_0_m_Item
    item_cols = []
    for col in df.columns:
        try:
            if re.match('^Data_Items_[0-9]+_m_Item$',str(col)):
                item_cols.append(col)
        except Exception as err:
                    print(f'Unexpected {err=}, {type(err)=}',col)
            
    return item_cols

In [2002]:
def drop_trash_columns (df):
    df = df.loc[:,~df.where(df.astype(bool)).isna().all(axis = 0)]
    return df

In [2003]:
def get_text_info(df, map_df, index_df, columns_in, keylist = None, return_temp = False, suffix = None, format_text=False, add_aggr_col = False):
    """ Function to create text columns with names and descriptions. Works on columns with asset ids
        and get filenames of those asset ids in asset_df and then look into files to extract strinkeys,
        max length of keylist is 4.
        Arguments:
        df - target df apply changes to;
        map_df - assets_df to get assets ids of files;
        index_df - dataframe of index file to get text info from;
        columns_in - list of columns apply changes over;
        keylist - keys of the received file get information from, if None search string keys to get ONLY names (not description keys);
        return_temp - if True return list of first columns with data, skipping further search and creation of columns;
                     if False, intermediate columns will be dropped.
        suffix - assign siffix to names of intermediate columns, if you return them, its better dont use 'text' for it,
                 because it is used inside function.
        format_text - if True apply format_description() function to the columns;
        add_aggr_col - create column with sum of values (or concatenation of strings) for provided columns;       
        """    
    # list of columns with string keys of items' names
    cols_names = columns_in
    #created column        
    # map_df - a dataframe for mapping    
    assets_dict = dict(zip(map_df.iloc[:,0],map_df.iloc[:,1])) #dict for mapping text information between two tables
    #reference to dataframe go from localization file    
    
    def find_parent(D):
        """find first key path to value containing stringkey"""    
        for k, v in D.items():        
            if isinstance(v,dict):
                parent = find_parent(v)
                if parent:
                    return [k] + parent
            elif match_pattern(v):                
                return [k]
    def find_stringkeys(row, keylist = None):    
        try:
            row = row.strip()
            with open(row) as asset_file:           
                file_contents = asset_file.read()
                data = json.loads(file_contents)
                try: 
                    if keylist is None:                   
                        keylist = find_parent(data)
                        if len(keylist)==3:
                            key = data[keylist[0]][keylist[1]][keylist[2]]
                            return key
                        else:
                            key = data[keylist[0]][keylist[1]][keylist[2]][keylist[3]]
                            return key
                    else:
                        if len(keylist)==1:
                            key = data[keylist[0]]
                            return key
                        if len(keylist)==2:
                            key = data[keylist[0]][keylist[1]]
                            return key
                        if len(keylist)==3:
                            key = data[keylist[0]][keylist[1]][keylist[2]]
                            return key
                        else:
                            key = data[keylist[0]][keylist[1]][keylist[2]][keylist[3]]
                            return key                  
                except:                         
                        return ''
                   
                                   
                
        except:
            return ''      
    
    #_____________________________Get items names and descriptions__________________________ 
    temp_list = []
    columns_out = []
    if suffix == None:
        suffix=='stringkey'
    for col in cols_names:                
                df[col] = df[col].str.strip()         
                filenames_series = df[col].map(assets_dict)               
                     
                df['{}_{}'.format(col,suffix)]  = filenames_series.apply(find_stringkeys, keylist = keylist)
                

                temp_list.append('{}_{}'.format(col,suffix))
                text_column_name = '{}_text'.format(col)
                columns_out.append(text_column_name)
    if return_temp == True:
          if format_text == True:            
            df[temp_list] = df[temp_list].apply(format_description,format_text = format_text)
            return df
          else:
              return df
    else:
         
        for i in range(len(columns_out)):
            col_in = [temp_list[i]]
            df = create_text_columns(df, index_df, col_in, columns_out[i], format_text = format_text)
        #drop created stringkey cols
        df = df.drop(columns = temp_list, axis = 1)
        if add_aggr_col == True:
            df['All_Items'] =df[columns_out].agg(sum)  
        return df, columns_out

### Folders

In [2004]:
#folder with the gameobjects' parameters
work_folder = r'e:\Steam\RogueTrader\WorkFolder\Blueprints'
# file with the english localization
index_file = r'e:\Steam\RogueTrader\Localization\enGB.json'
#file with the russian localization
index_file_ru = r'e:\Steam\RogueTrader\Localization\ruRU.json'
save_folder = r'e:\Steam\RogueTrader\SaveFolder'
load_folder = r'e:\Steam\RogueTrader\LoadFolder'

## Work Part

### Load support tables

In [2005]:
#loading support dataframes
assets_df = pd.read_csv(load_folder + r'\assets_df.csv')
index_df = pd.read_csv(load_folder + r'\index_df.csv')
index_df_ru = pd.read_csv(load_folder + r'\index_df_ru.csv')

### Folders

In [2006]:
folder_star_systems = r'e:\Steam\RogueTrader\WorkFolder\Blueprints\GlobalMaps\StarSystems'
folder_space_combat = r'e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat'

### Table of star systems

In [2007]:
list_of_files = []
for root,dirs,files in os.walk(folder_star_systems):
        for file in files:
            list_of_files.append(os.path.join(root, file))
    

print(len(list_of_files))
star_systems_data = create_df(list_of_files)
star_systems_data = star_systems_data.replace('[]', np.nan)
star_systems_data = drop_trash_columns(star_systems_data)

61


In [2008]:
display(star_systems_data.head(2))

,AssetId,Data_$type,Data_BookEventConditions_Conditions_0_$type,Data_BookEventConditions_Conditions_0_State,Data_BookEventConditions_Conditions_0_m_QuestObjective,Data_BookEventConditions_Conditions_0_name,Data_BookEventConditions_Operation,Data_ConditionsToVisitAutomatically_Conditions_0_$type,Data_ConditionsToVisitAutomatically_Conditions_0_Playing,Data_ConditionsToVisitAutomatically_Conditions_0_m_Etude,Data_ConditionsToVisitAutomatically_Conditions_0_name,Data_ConditionsToVisitAutomatically_Operation,Data_IsFakeSystem,Data_OverrideBookEvent,Data_StarSystemAreaPoint,Data_StarSystemToTransit,Data_m_DisplayName_Shared_stringkey,Filepath
0,61147fb71a8c41e590f3959457d75f60,"26ce44c3d7634d54afef5387f8b9b300, BlueprintSectorMapPointStarSystem",NaN,NaN,NaN,NaN,And,"ea981728db8a5f84888ecba390671a05, EtudeStatus",True,42f3de41578641e1b6853ab353d7d030,$EtudeStatus$2ebd4b61-01fe-4faa-bd7a-a856811fc3de,And,False,None,5ec9d0c12391417d8268008534e4abff,18713c8095954567a20a5b406146ceab,360e78bb-8767-4adf-adaf-3422616edf5e,e:\Steam\RogueTrader\WorkFolder\Blueprints\GlobalMaps\StarSystems\AdermattIISystem.jbp
1,48d2872318ea4474a7717eb9d2f1726d,"26ce44c3d7634d54afef5387f8b9b300, BlueprintSectorMapPointStarSystem",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0298b0820b9448ccafccd7655d488a4d,49054ec8edfa49bb9e93871b8f84ce44,b648d69e-043b-421c-ae62-156a3a9d571f,e:\Steam\RogueTrader\WorkFolder\Blueprints\GlobalMaps\StarSystems\AtlassianReachSystem.jbp


In [2009]:
#Check for missing values
missing_values_tab(star_systems_data)

Dataframe contains 18 columns and 61 strings.

It has  14 columns with missed values


,missed_values,%_of_all,data_type
Data_BookEventConditions_Conditions_0_$type,60,98.36,object
Data_BookEventConditions_Conditions_0_State,60,98.36,object
Data_BookEventConditions_Conditions_0_m_QuestObjective,60,98.36,object
Data_BookEventConditions_Conditions_0_name,60,98.36,object
Data_ConditionsToVisitAutomatically_Conditions_0_$type,59,96.72,object
Data_ConditionsToVisitAutomatically_Conditions_0_Playing,59,96.72,object
Data_ConditionsToVisitAutomatically_Conditions_0_m_Etude,59,96.72,object
Data_ConditionsToVisitAutomatically_Conditions_0_name,59,96.72,object
Data_ConditionsToVisitAutomatically_Operation,59,96.72,object
Data_OverrideBookEvent,59,96.72,object


Calculate column groups:

In [2010]:
keywords = []
selected_cols, assetIds_cols, stringkey_cols, property_cols, misc_cols = select_columns(star_systems_data, keywords)

Create columns with text information

In [2011]:
print(stringkey_cols)

['Data_m_DisplayName_Shared_stringkey']


In [2012]:
#english version
star_systems_data = create_text_columns(df = star_systems_data,
                                        index_df = index_df,
                                        columns_in=stringkey_cols,
                                        column_out='System_Name', 
                                        format_text=True)
#russian version
star_systems_data = create_text_columns(df = star_systems_data,
                                        index_df = index_df_ru,
                                        columns_in=stringkey_cols,
                                        column_out='System_Name_Ru', 
                                        format_text=True)



In [2013]:
display(star_systems_data.head())

,AssetId,Data_$type,Data_BookEventConditions_Conditions_0_$type,Data_BookEventConditions_Conditions_0_State,Data_BookEventConditions_Conditions_0_m_QuestObjective,Data_BookEventConditions_Conditions_0_name,Data_BookEventConditions_Operation,Data_ConditionsToVisitAutomatically_Conditions_0_$type,Data_ConditionsToVisitAutomatically_Conditions_0_Playing,Data_ConditionsToVisitAutomatically_Conditions_0_m_Etude,Data_ConditionsToVisitAutomatically_Conditions_0_name,Data_ConditionsToVisitAutomatically_Operation,Data_IsFakeSystem,Data_OverrideBookEvent,Data_StarSystemAreaPoint,Data_StarSystemToTransit,Data_m_DisplayName_Shared_stringkey,Filepath,System_Name,System_Name_Ru
0,61147fb71a8c41e590f3959457d75f60,"26ce44c3d7634d54afef5387f8b9b300, BlueprintSectorMapPointStarSystem",NaN,NaN,NaN,NaN,And,"ea981728db8a5f84888ecba390671a05, EtudeStatus",True,42f3de41578641e1b6853ab353d7d030,$EtudeStatus$2ebd4b61-01fe-4faa-bd7a-a856811fc3de,And,False,None,5ec9d0c12391417d8268008534e4abff,18713c8095954567a20a5b406146ceab,360e78bb-8767-4adf-adaf-3422616edf5e,e:\Steam\RogueTrader\WorkFolder\Blueprints\GlobalMaps\StarSystems\AdermattIISystem.jbp,Adermatt II,Ардематт II
1,48d2872318ea4474a7717eb9d2f1726d,"26ce44c3d7634d54afef5387f8b9b300, BlueprintSectorMapPointStarSystem",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0298b0820b9448ccafccd7655d488a4d,49054ec8edfa49bb9e93871b8f84ce44,b648d69e-043b-421c-ae62-156a3a9d571f,e:\Steam\RogueTrader\WorkFolder\Blueprints\GlobalMaps\StarSystems\AtlassianReachSystem.jbp,Atlassian Reach,Атласов Предел
2,399a57be63e5427e8c738acfdf1bc0e0,"26ce44c3d7634d54afef5387f8b9b300, BlueprintSectorMapPointStarSystem",NaN,NaN,NaN,NaN,And,"ea981728db8a5f84888ecba390671a05, EtudeStatus",True,42f3de41578641e1b6853ab353d7d030,$EtudeStatus$9c2046e0d677416396e9cdc0cee2ef4d,And,False,None,c63204009b1943328c3fe0850a65db8d,ca7a7cc559ba4825912b48845a8c1230,cb1cdb2c-48eb-481a-8f20-8f5169414ebc,e:\Steam\RogueTrader\WorkFolder\Blueprints\GlobalMaps\StarSystems\AviorusSystem.jbp,Aviorus,Авиорус
3,2b67dafba2de415ba0c60440805aa908,"26ce44c3d7634d54afef5387f8b9b300, BlueprintSectorMapPointStarSystem",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,4a3efc3bd16d41c9b28cb1577be0bea8,f0cd97de043a471f9df04cac15a1c58e,10b654c2-cf16-4e8d-a761-4cacd8e4d73b,e:\Steam\RogueTrader\WorkFolder\Blueprints\GlobalMaps\StarSystems\CradleofKhepriSystem.jbp,Cradle of Khepri,Колыбель Хепри
4,daff71f3024b4e9fbf30646e94120834,"26ce44c3d7634d54afef5387f8b9b300, BlueprintSectorMapPointStarSystem",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,cd1ca313b31a424db08173c8b2acc9a3,3869a056cf4244a1bedbcd116e797508,95a8b903-c568-47a3-90fe-a153e9d2a3ad,e:\Steam\RogueTrader\WorkFolder\Blueprints\GlobalMaps\StarSystems\CranachSystem.jbp,Cranach,Кранах


### Space combat table

list_of_files = []
skip_folders = ['Player', 'Progression'] # skip player ship data and progression systems
for root,dirs,files in os.walk(folder_space_combat):
        dirs[:] = [d for d in dirs if d not in skip_folders]
        for file in files:
            if 'VisualSettings' not in file: # skip files with graphic data
                  list_of_files.append(os.path.join(root, file))   
print(len(list_of_files))
space_combat_data = create_df(list_of_files)
space_combat_data = space_combat_data.replace('[]', np.nan)
space_combat_data = drop_trash_columns(space_combat_data)
print(space_combat_data.shape)
display(space_combat_data.head(2))


<span style = "color:yellow"> Saving space combat data:</span>

In [2014]:
#space_combat_data.to_csv(save_folder + r'\space_combat_data.csv', index = False)

<span style = "color:yellow"> Loading savecopy of space combat data:</span>

In [2015]:
space_combat_data = pd.read_csv(load_folder + r'\space_combat_data.csv')
display(space_combat_data.head())

,AssetId,Data_$type,Data_AbilityParamsSource,Data_AbilitySettings_0_AbilityCastSpot,Data_AbilitySettings_0_AbilitySource_Type,Data_AbilitySettings_0_AbilitySource_m_Ability,Data_AbilitySettings_0_AbilitySource_m_Equipment,Data_AbilitySettings_0_AbilityValue_Getters_0_$type,Data_AbilitySettings_0_AbilityValue_Getters_0_Settings_Limit,Data_AbilitySettings_0_AbilityValue_Getters_0_Settings_Progression,Data_AbilitySettings_0_AbilityValue_Getters_0_Value_Property,Data_AbilitySettings_0_AbilityValue_Getters_0_Value_PropertyName,Data_AbilitySettings_0_AbilityValue_Getters_0_Value_Value,Data_AbilitySettings_0_AbilityValue_Getters_0_Value_ValueRank,Data_AbilitySettings_0_AbilityValue_Getters_0_Value_ValueShared,Data_AbilitySettings_0_AbilityValue_Getters_0_Value_ValueType,Data_AbilitySettings_0_AbilityValue_Getters_0_m_Property,Data_AbilitySettings_0_AbilityValue_Getters_0_name,Data_AbilitySettings_0_AbilityValue_Operation,Data_AbilitySettings_0_AbilityValue_TargetType,Data_AbilitySettings_0_OptimumDistance,Data_AbilitySettings_1_AbilityCastSpot,Data_AbilitySettings_1_AbilitySource_Type,Data_AbilitySettings_1_AbilitySource_m_Ability,Data_AbilitySettings_1_AbilitySource_m_Equipment,Data_AbilitySettings_1_AbilityValue_Getters_0_$type,Data_AbilitySettings_1_AbilityValue_Getters_0_Settings_Limit,Data_AbilitySettings_1_AbilityValue_Getters_0_Settings_Progression,Data_AbilitySettings_1_AbilityValue_Getters_0_Value_Property,Data_AbilitySettings_1_AbilityValue_Getters_0_Value_PropertyName,Data_AbilitySettings_1_AbilityValue_Getters_0_Value_Value,Data_AbilitySettings_1_AbilityValue_Getters_0_Value_ValueRank,Data_AbilitySettings_1_AbilityValue_Getters_0_Value_ValueShared,Data_AbilitySettings_1_AbilityValue_Getters_0_Value_ValueType,Data_AbilitySettings_1_AbilityValue_Getters_0_m_Property,Data_AbilitySettings_1_AbilityValue_Getters_0_name,Data_AbilitySettings_1_AbilityValue_Operation,Data_AbilitySettings_1_AbilityValue_TargetType,Data_AbilitySettings_1_OptimumDistance,Data_AbilitySettings_2_AbilityCastSpot,Data_AbilitySettings_2_AbilitySource_Type,Data_AbilitySettings_2_AbilitySource_m_Ability,Data_AbilitySettings_2_AbilitySource_m_Equipment,Data_AbilitySettings_2_AbilityValue_Getters_0_$type,Data_AbilitySettings_2_AbilityValue_Getters_0_Settings_Limit,Data_AbilitySettings_2_AbilityValue_Getters_0_Settings_Progression,Data_AbilitySettings_2_AbilityValue_Getters_0_Value_Property,Data_AbilitySettings_2_AbilityValue_Getters_0_Value_PropertyName,Data_AbilitySettings_2_AbilityValue_Getters_0_Value_Value,Data_AbilitySettings_2_AbilityValue_Getters_0_Value_ValueRank,Data_AbilitySettings_2_AbilityValue_Getters_0_Value_ValueShared,Data_AbilitySettings_2_AbilityValue_Getters_0_Value_ValueType,Data_AbilitySettings_2_AbilityValue_Getters_0_m_Property,Data_AbilitySettings_2_AbilityValue_Getters_0_name,Data_AbilitySettings_2_AbilityValue_Operation,Data_AbilitySettings_2_AbilityValue_TargetType,Data_AbilitySettings_2_OptimumDistance,Data_AbilitySettings_3_AbilityCastSpot,Data_AbilitySettings_3_AbilitySource_Type,Data_AbilitySettings_3_AbilitySource_m_Ability,Data_AbilitySettings_3_AbilitySource_m_Equipment,Data_AbilitySettings_3_AbilityValue_Getters_0_$type,Data_AbilitySettings_3_AbilityValue_Getters_0_Settings_Limit,Data_AbilitySettings_3_AbilityValue_Getters_0_Settings_Progression,Data_AbilitySettings_3_AbilityValue_Getters_0_Value_Property,Data_AbilitySettings_3_AbilityValue_Getters_0_Value_PropertyName,Data_AbilitySettings_3_AbilityValue_Getters_0_Value_Value,Data_AbilitySettings_3_AbilityValue_Getters_0_Value_ValueRank,Data_AbilitySettings_3_AbilityValue_Getters_0_Value_ValueShared,Data_AbilitySettings_3_AbilityValue_Getters_0_Value_ValueType,Data_AbilitySettings_3_AbilityValue_Getters_0_name,Data_AbilitySettings_3_AbilityValue_Operation,Data_AbilitySettings_3_AbilityValue_TargetType,Data_AbilitySettings_3_OptimumDistance,Data_AbilitySettings_4_AbilityCastSpot,Data_AbilitySettings_4_AbilitySource_Type,Data_AbilitySettings_4_AbilitySource_m_Ability,Data_Abi

What data types are in the table:

In [2016]:
display(space_combat_data['Data_$type'].unique())

array(['da11db195c86e0d4dae17a2c03a4ba9a, BlueprintAbility',
       'cb208b98ceacca84baee15dba53b1979, BlueprintFeature',
       '618a7e0d54149064ab3ffa5d9057362c, BlueprintBuff',
       '63bb90208198450095c55997ebc6ae0a, ActionsHolder',
       '932f7051c5b3e14458ad57af12803b85, BlueprintStarshipBrain',
       '84a976c8e48e6274e8367073fad4a237, BlueprintAbilityGroup',
       '1fddf23bccefc9b49b5450c0fe823062, BlueprintStarshipAmmo',
       '973c613b8443cf14495c283e293d35f9, BlueprintAbilityResource',
       'd2513f4af02e4554bb8f9f79a5c35a46, BlueprintStarshipWeapon',
       '19be5dcf539a404487f98692dc73d872, PropertyCalculatorBlueprint',
       '19be5dcf539a404487f98692dc73d872, BlueprintEntityProperty',
       'f6ca6d5da1433d44a93b03a4c9bded9e, BlueprintItemArmorPlating',
       '10e0c4e16b29cf24ba962c87a49146d8, BlueprintItemAugerArray',
       'ca5f3cfd04654b9489e1b267fc30a044, BlueprintItemPlasmaDrives',
       '427d75a81e7bb9a40ac9b7eed773b946, BlueprintItemVoidShieldGenerator',
 

### Space enemies

In [2017]:
folder_enemies = r'e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Units\Enemies'
list_of_files = []
for root,dirs,files in os.walk(folder_enemies):
        for file in files:
            if 'VisualSettings' not in file:
                  list_of_files.append(os.path.join(root, file))    

print(len(list_of_files))
space_enemies_data = create_df(list_of_files)
space_enemies_data = space_enemies_data.replace('[]', np.nan)
space_enemies_data = drop_trash_columns(space_enemies_data)
print(space_enemies_data.shape)
display(space_enemies_data.head(2))

122


(122, 792)


,AssetId,Data_$type,Data_AbilityParamsSource,Data_AbilitySettings_0_AbilityCastSpot,Data_AbilitySettings_0_AbilitySource_Type,Data_AbilitySettings_0_AbilitySource_m_Ability,Data_AbilitySettings_0_AbilitySource_m_Equipment,Data_AbilitySettings_0_AbilityValue_Getters_0_$type,Data_AbilitySettings_0_AbilityValue_Getters_0_Settings_Limit,Data_AbilitySettings_0_AbilityValue_Getters_0_Settings_Progression,Data_AbilitySettings_0_AbilityValue_Getters_0_Value_Property,Data_AbilitySettings_0_AbilityValue_Getters_0_Value_PropertyName,Data_AbilitySettings_0_AbilityValue_Getters_0_Value_Value,Data_AbilitySettings_0_AbilityValue_Getters_0_Value_ValueRank,Data_AbilitySettings_0_AbilityValue_Getters_0_Value_ValueShared,Data_AbilitySettings_0_AbilityValue_Getters_0_Value_ValueType,Data_AbilitySettings_0_AbilityValue_Getters_0_m_Property,Data_AbilitySettings_0_AbilityValue_Getters_0_name,Data_AbilitySettings_0_AbilityValue_Operation,Data_AbilitySettings_0_AbilityValue_TargetType,Data_AbilitySettings_0_OptimumDistance,Data_AbilitySettings_1_AbilityCastSpot,Data_AbilitySettings_1_AbilitySource_Type,Data_AbilitySettings_1_AbilitySource_m_Ability,Data_AbilitySettings_1_AbilitySource_m_Equipment,Data_AbilitySettings_1_AbilityValue_Getters_0_$type,Data_AbilitySettings_1_AbilityValue_Getters_0_Settings_Limit,Data_AbilitySettings_1_AbilityValue_Getters_0_Settings_Progression,Data_AbilitySettings_1_AbilityValue_Getters_0_Value_Property,Data_AbilitySettings_1_AbilityValue_Getters_0_Value_PropertyName,Data_AbilitySettings_1_AbilityValue_Getters_0_Value_Value,Data_AbilitySettings_1_AbilityValue_Getters_0_Value_ValueRank,Data_AbilitySettings_1_AbilityValue_Getters_0_Value_ValueShared,Data_AbilitySettings_1_AbilityValue_Getters_0_Value_ValueType,Data_AbilitySettings_1_AbilityValue_Getters_0_m_Property,Data_AbilitySettings_1_AbilityValue_Getters_0_name,Data_AbilitySettings_1_AbilityValue_Operation,Data_AbilitySettings_1_AbilityValue_TargetType,Data_AbilitySettings_1_OptimumDistance,Data_AbilitySettings_2_AbilityCastSpot,Data_AbilitySettings_2_AbilitySource_Type,Data_AbilitySettings_2_AbilitySource_m_Ability,Data_AbilitySettings_2_AbilitySource_m_Equipment,Data_AbilitySettings_2_AbilityValue_Getters_0_$type,Data_AbilitySettings_2_AbilityValue_Getters_0_Settings_Limit,Data_AbilitySettings_2_AbilityValue_Getters_0_Settings_Progression,Data_AbilitySettings_2_AbilityValue_Getters_0_Value_Property,Data_AbilitySettings_2_AbilityValue_Getters_0_Value_PropertyName,Data_AbilitySettings_2_AbilityValue_Getters_0_Value_Value,Data_AbilitySettings_2_AbilityValue_Getters_0_Value_ValueRank,Data_AbilitySettings_2_AbilityValue_Getters_0_Value_ValueShared,Data_AbilitySettings_2_AbilityValue_Getters_0_Value_ValueType,Data_AbilitySettings_2_AbilityValue_Getters_0_m_Property,Data_AbilitySettings_2_AbilityValue_Getters_0_name,Data_AbilitySettings_2_AbilityValue_Operation,Data_AbilitySettings_2_AbilityValue_TargetType,Data_AbilitySettings_2_OptimumDistance,Data_AbilitySettings_3_AbilityCastSpot,Data_AbilitySettings_3_AbilitySource_Type,Data_AbilitySettings_3_AbilitySource_m_Ability,Data_AbilitySettings_3_AbilitySource_m_Equipment,Data_AbilitySettings_3_AbilityValue_Getters_0_$type,Data_AbilitySettings_3_AbilityValue_Getters_0_Settings_Limit,Data_AbilitySettings_3_AbilityValue_Getters_0_Settings_Progression,Data_AbilitySettings_3_AbilityValue_Getters_0_Value_Property,Data_AbilitySettings_3_AbilityValue_Getters_0_Value_PropertyName,Data_AbilitySettings_3_AbilityValue_Getters_0_Value_Value,Data_AbilitySettings_3_AbilityValue_Getters_0_Value_ValueRank,Data_AbilitySettings_3_AbilityValue_Getters_0_Value_ValueShared,Data_AbilitySettings_3_AbilityValue_Getters_0_Value_ValueType,Data_AbilitySettings_3_AbilityValue_Getters_0_name,Data_AbilitySettings_3_AbilityValue_Operation,Data_AbilitySettings_3_AbilityValue_TargetType,Data_AbilitySettings_3_OptimumDistance,Data_AbilitySettings_4_AbilityCastSpot,Data_AbilitySettings_4_AbilitySource_Type,Data_AbilitySettings_4_AbilitySource_m_Ability,Data_Abi

In [2018]:
#data types in the table
display(space_enemies_data['Data_$type'].unique())

array(['b1dda40dbcb7b8647af156cf539d3ed8, BlueprintStarship',
       '932f7051c5b3e14458ad57af12803b85, BlueprintStarshipBrain',
       '10e0c4e16b29cf24ba962c87a49146d8, BlueprintItemAugerArray',
       'cb208b98ceacca84baee15dba53b1979, BlueprintFeature',
       '427d75a81e7bb9a40ac9b7eed773b946, BlueprintItemVoidShieldGenerator',
       'da11db195c86e0d4dae17a2c03a4ba9a, BlueprintAbility',
       '618a7e0d54149064ab3ffa5d9057362c, BlueprintBuff',
       'd2513f4af02e4554bb8f9f79a5c35a46, BlueprintStarshipWeapon',
       '4e19ee98b71c98b40ba235cfa715b760, BlueprintAbilityAreaEffect',
       'ca5f3cfd04654b9489e1b267fc30a044, BlueprintItemPlasmaDrives'],
      dtype=object)

#### Ship data table

**Notes:**  
Data type==```b1dda40dbcb7b8647af156cf539d3ed8, BlueprintStarship``` contains ship characteristics.  

Create table with enemies data:

In [2019]:
enemies_df = space_enemies_data[space_enemies_data['Data_$type']=='b1dda40dbcb7b8647af156cf539d3ed8, BlueprintStarship']
enemies_df = drop_trash_columns(enemies_df)
print(enemies_df.shape)
display(enemies_df.head())

(33, 134)


,AssetId,Data_$type,Data_AiDesiredDistanceToPlayer,Data_Alignment,Data_AlternativeBrains_0,Data_ApproachStarSystemObjectRadius,Data_ArmourAft,Data_ArmourFore,Data_ArmourPort,Data_ArmourStarboard,Data_Body_DisableHands,Data_Color_a,Data_Color_b,Data_Color_g,Data_Color_r,Data_Components_0_$type,Data_Components_0_CR,Data_Components_0_Encounter,Data_Components_0_HpPerUnit,Data_Components_0_LandingActions_Actions_0_$type,Data_Components_0_LandingActions_Actions_0_DC_Property,Data_Components_0_LandingActions_Actions_0_DC_PropertyName,Data_Components_0_LandingActions_Actions_0_DC_ValueRank,Data_Components_0_LandingActions_Actions_0_DC_ValueShared,Data_Components_0_LandingActions_Actions_0_DC_ValueType,Data_Components_0_LandingActions_Actions_0_SpellLevel_Property,Data_Components_0_LandingActions_Actions_0_SpellLevel_PropertyName,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueRank,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueShared,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueType,Data_Components_0_LandingActions_Actions_0_m_Spell,Data_Components_0_LandingActions_Actions_0_name,Data_Components_0_LandingActions_Actions_1_$type,Data_Components_0_LandingActions_Actions_1_NoLog,Data_Components_0_LandingActions_Actions_1_name,Data_Components_0_LandingDistance,Data_Components_0_Modifier,Data_Components_0_name,Data_Components_1_$type,Data_Components_1_ExpirationActions_Actions_0_$type,Data_Components_1_ExpirationActions_Actions_0_PrefabLink_AssetId,Data_Components_1_ExpirationActions_Actions_0_name,Data_Components_1_ExpirationActions_Actions_1_$type,Data_Components_1_ExpirationActions_Actions_1_NoExp,Data_Components_1_ExpirationActions_Actions_1_name,Data_Components_1_Rewards_0_Condition_0_HasBuff,Data_Components_1_Rewards_0_Condition_0_Not,Data_Components_1_Rewards_0_Condition_0_m_Buff,Data_Components_1_Rewards_0_ItemCounts_0,Data_Components_1_Rewards_0_ItemCounts_1,Data_Components_1_Rewards_0_Items_0,Data_Components_1_Rewards_0_Items_1,Data_Components_1_Rewards_0_Scrap,Data_Components_1_m_ExpendAllFuelOnAttack,Data_Components_1_m_FuelBuff,Data_Components_1_m_LandingBuff,Data_Components_1_m_Overrides_0,Data_Components_1_name,Data_CrewQuartersThroughput,Data_DifficultyType,Data_Evasion,Data_Gender,Data_HullIntegrity,Data_HullSlots_Weapons_0_OffsetFromProw,Data_HullSlots_Weapons_0_Type,Data_HullSlots_Weapons_0_Weapon,Data_HullSlots_Weapons_1_OffsetFromProw,Data_HullSlots_Weapons_1_Type,Data_HullSlots_Weapons_1_Weapon,Data_HullSlots_Weapons_2_OffsetFromProw,Data_HullSlots_Weapons_2_Type,Data_HullSlots_Weapons_2_Weapon,Data_HullSlots_Weapons_3_OffsetFromProw,Data_HullSlots_Weapons_3_Type,Data_HullSlots_Weapons_3_Weapon,Data_HullSlots_Weapons_4_Type,Data_HullSlots_Weapons_4_Weapon,Data_HullSlots_Weapons_5_OffsetFromProw,Data_HullSlots_Weapons_5_Type,Data_HullSlots_Weapons_5_Weapon,Data_HullSlots_m_AugerArray,Data_HullSlots_m_PlasmaDrives,Data_HullSlots_m_VoidShieldGenerator,Data_Inertia,Data_Initiative,Data_InspirationAmount,Data_IsSoftUnit,Data_LocalizedName_stringkey,Data_MaxHP,Data_MilitaryRating,Data_Morale,Data_OldWarhammerAgility,Data_OldWarhammerBallisticSkill,Data_OldWarhammerFellowship,Data_OldWarhammerIntelligence,Data_OldWarhammerPerception,Data_OldWarhammerStrength,Data_OldWarhammerToughness,Data_OldWarhammerWeaponSkill,Data_OldWarhammerWillpower,Data_Prefab_AssetId,Data_Size,Data_SpeedOnStarSystemMap,Data_Speed_m_Value,Data_StarshipSpeed,Data_Subtype,Data_WarhammerAgilitySetting_Modifier,Data_WarhammerAgilitySetting_NotModified,Data_WarhammerBallisticSkillSetting_NotModified,Data_WarhammerFellowshipSetting_NotModified,Data_WarhammerInitialAPBlue,Data_WarhammerInitialAPYellow,Data_WarhammerIntelligenceSetting_NotModified,Data_WarhammerMovementApPerCell,Data_WarhammerMovementApPerCellThreateningArea,Data_WarhammerPerceptionSetting_Modifier,Data_WarhammerPerceptionSetting_NotModified,Data_WarhammerStrengthSetting_NotModified,Data_WarhammerToughnessSetting_NotModified,Data_WarhammerWeaponSkillSett

Calculate column groups:

In [2020]:
keywords = []
selected_cols, assetIds_cols, stringkey_cols, property_cols, misc_cols = select_columns(enemies_df, keywords)

Create text columns with ship names:

In [2021]:
# Create separate dataframe for russian version
enemies_df_ru = enemies_df.copy(deep = True)

In [2022]:
#english version
enemies_df = create_text_columns(df = enemies_df,
                                        index_df = index_df,
                                        columns_in=stringkey_cols,
                                        column_out='Ship_Name', 
                                        format_text=True)
#russian version
enemies_df_ru = create_text_columns(df = enemies_df_ru,
                                        index_df = index_df_ru,
                                        columns_in=stringkey_cols,
                                        column_out='Ship_Name', 
                                        format_text=True)

In [2023]:
display(enemies_df.head(2), enemies_df_ru.head(2))

,AssetId,Data_$type,Data_AiDesiredDistanceToPlayer,Data_Alignment,Data_AlternativeBrains_0,Data_ApproachStarSystemObjectRadius,Data_ArmourAft,Data_ArmourFore,Data_ArmourPort,Data_ArmourStarboard,Data_Body_DisableHands,Data_Color_a,Data_Color_b,Data_Color_g,Data_Color_r,Data_Components_0_$type,Data_Components_0_CR,Data_Components_0_Encounter,Data_Components_0_HpPerUnit,Data_Components_0_LandingActions_Actions_0_$type,Data_Components_0_LandingActions_Actions_0_DC_Property,Data_Components_0_LandingActions_Actions_0_DC_PropertyName,Data_Components_0_LandingActions_Actions_0_DC_ValueRank,Data_Components_0_LandingActions_Actions_0_DC_ValueShared,Data_Components_0_LandingActions_Actions_0_DC_ValueType,Data_Components_0_LandingActions_Actions_0_SpellLevel_Property,Data_Components_0_LandingActions_Actions_0_SpellLevel_PropertyName,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueRank,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueShared,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueType,Data_Components_0_LandingActions_Actions_0_m_Spell,Data_Components_0_LandingActions_Actions_0_name,Data_Components_0_LandingActions_Actions_1_$type,Data_Components_0_LandingActions_Actions_1_NoLog,Data_Components_0_LandingActions_Actions_1_name,Data_Components_0_LandingDistance,Data_Components_0_Modifier,Data_Components_0_name,Data_Components_1_$type,Data_Components_1_ExpirationActions_Actions_0_$type,Data_Components_1_ExpirationActions_Actions_0_PrefabLink_AssetId,Data_Components_1_ExpirationActions_Actions_0_name,Data_Components_1_ExpirationActions_Actions_1_$type,Data_Components_1_ExpirationActions_Actions_1_NoExp,Data_Components_1_ExpirationActions_Actions_1_name,Data_Components_1_Rewards_0_Condition_0_HasBuff,Data_Components_1_Rewards_0_Condition_0_Not,Data_Components_1_Rewards_0_Condition_0_m_Buff,Data_Components_1_Rewards_0_ItemCounts_0,Data_Components_1_Rewards_0_ItemCounts_1,Data_Components_1_Rewards_0_Items_0,Data_Components_1_Rewards_0_Items_1,Data_Components_1_Rewards_0_Scrap,Data_Components_1_m_ExpendAllFuelOnAttack,Data_Components_1_m_FuelBuff,Data_Components_1_m_LandingBuff,Data_Components_1_m_Overrides_0,Data_Components_1_name,Data_CrewQuartersThroughput,Data_DifficultyType,Data_Evasion,Data_Gender,Data_HullIntegrity,Data_HullSlots_Weapons_0_OffsetFromProw,Data_HullSlots_Weapons_0_Type,Data_HullSlots_Weapons_0_Weapon,Data_HullSlots_Weapons_1_OffsetFromProw,Data_HullSlots_Weapons_1_Type,Data_HullSlots_Weapons_1_Weapon,Data_HullSlots_Weapons_2_OffsetFromProw,Data_HullSlots_Weapons_2_Type,Data_HullSlots_Weapons_2_Weapon,Data_HullSlots_Weapons_3_OffsetFromProw,Data_HullSlots_Weapons_3_Type,Data_HullSlots_Weapons_3_Weapon,Data_HullSlots_Weapons_4_Type,Data_HullSlots_Weapons_4_Weapon,Data_HullSlots_Weapons_5_OffsetFromProw,Data_HullSlots_Weapons_5_Type,Data_HullSlots_Weapons_5_Weapon,Data_HullSlots_m_AugerArray,Data_HullSlots_m_PlasmaDrives,Data_HullSlots_m_VoidShieldGenerator,Data_Inertia,Data_Initiative,Data_InspirationAmount,Data_IsSoftUnit,Data_LocalizedName_stringkey,Data_MaxHP,Data_MilitaryRating,Data_Morale,Data_OldWarhammerAgility,Data_OldWarhammerBallisticSkill,Data_OldWarhammerFellowship,Data_OldWarhammerIntelligence,Data_OldWarhammerPerception,Data_OldWarhammerStrength,Data_OldWarhammerToughness,Data_OldWarhammerWeaponSkill,Data_OldWarhammerWillpower,Data_Prefab_AssetId,Data_Size,Data_SpeedOnStarSystemMap,Data_Speed_m_Value,Data_StarshipSpeed,Data_Subtype,Data_WarhammerAgilitySetting_Modifier,Data_WarhammerAgilitySetting_NotModified,Data_WarhammerBallisticSkillSetting_NotModified,Data_WarhammerFellowshipSetting_NotModified,Data_WarhammerInitialAPBlue,Data_WarhammerInitialAPYellow,Data_WarhammerIntelligenceSetting_NotModified,Data_WarhammerMovementApPerCell,Data_WarhammerMovementApPerCellThreateningArea,Data_WarhammerPerceptionSetting_Modifier,Data_WarhammerPerceptionSetting_NotModified,Data_WarhammerStrengthSetting_NotModified,Data_WarhammerToughnessSetting_NotModified,Data_WarhammerWeaponSkillSett

,AssetId,Data_$type,Data_AiDesiredDistanceToPlayer,Data_Alignment,Data_AlternativeBrains_0,Data_ApproachStarSystemObjectRadius,Data_ArmourAft,Data_ArmourFore,Data_ArmourPort,Data_ArmourStarboard,Data_Body_DisableHands,Data_Color_a,Data_Color_b,Data_Color_g,Data_Color_r,Data_Components_0_$type,Data_Components_0_CR,Data_Components_0_Encounter,Data_Components_0_HpPerUnit,Data_Components_0_LandingActions_Actions_0_$type,Data_Components_0_LandingActions_Actions_0_DC_Property,Data_Components_0_LandingActions_Actions_0_DC_PropertyName,Data_Components_0_LandingActions_Actions_0_DC_ValueRank,Data_Components_0_LandingActions_Actions_0_DC_ValueShared,Data_Components_0_LandingActions_Actions_0_DC_ValueType,Data_Components_0_LandingActions_Actions_0_SpellLevel_Property,Data_Components_0_LandingActions_Actions_0_SpellLevel_PropertyName,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueRank,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueShared,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueType,Data_Components_0_LandingActions_Actions_0_m_Spell,Data_Components_0_LandingActions_Actions_0_name,Data_Components_0_LandingActions_Actions_1_$type,Data_Components_0_LandingActions_Actions_1_NoLog,Data_Components_0_LandingActions_Actions_1_name,Data_Components_0_LandingDistance,Data_Components_0_Modifier,Data_Components_0_name,Data_Components_1_$type,Data_Components_1_ExpirationActions_Actions_0_$type,Data_Components_1_ExpirationActions_Actions_0_PrefabLink_AssetId,Data_Components_1_ExpirationActions_Actions_0_name,Data_Components_1_ExpirationActions_Actions_1_$type,Data_Components_1_ExpirationActions_Actions_1_NoExp,Data_Components_1_ExpirationActions_Actions_1_name,Data_Components_1_Rewards_0_Condition_0_HasBuff,Data_Components_1_Rewards_0_Condition_0_Not,Data_Components_1_Rewards_0_Condition_0_m_Buff,Data_Components_1_Rewards_0_ItemCounts_0,Data_Components_1_Rewards_0_ItemCounts_1,Data_Components_1_Rewards_0_Items_0,Data_Components_1_Rewards_0_Items_1,Data_Components_1_Rewards_0_Scrap,Data_Components_1_m_ExpendAllFuelOnAttack,Data_Components_1_m_FuelBuff,Data_Components_1_m_LandingBuff,Data_Components_1_m_Overrides_0,Data_Components_1_name,Data_CrewQuartersThroughput,Data_DifficultyType,Data_Evasion,Data_Gender,Data_HullIntegrity,Data_HullSlots_Weapons_0_OffsetFromProw,Data_HullSlots_Weapons_0_Type,Data_HullSlots_Weapons_0_Weapon,Data_HullSlots_Weapons_1_OffsetFromProw,Data_HullSlots_Weapons_1_Type,Data_HullSlots_Weapons_1_Weapon,Data_HullSlots_Weapons_2_OffsetFromProw,Data_HullSlots_Weapons_2_Type,Data_HullSlots_Weapons_2_Weapon,Data_HullSlots_Weapons_3_OffsetFromProw,Data_HullSlots_Weapons_3_Type,Data_HullSlots_Weapons_3_Weapon,Data_HullSlots_Weapons_4_Type,Data_HullSlots_Weapons_4_Weapon,Data_HullSlots_Weapons_5_OffsetFromProw,Data_HullSlots_Weapons_5_Type,Data_HullSlots_Weapons_5_Weapon,Data_HullSlots_m_AugerArray,Data_HullSlots_m_PlasmaDrives,Data_HullSlots_m_VoidShieldGenerator,Data_Inertia,Data_Initiative,Data_InspirationAmount,Data_IsSoftUnit,Data_LocalizedName_stringkey,Data_MaxHP,Data_MilitaryRating,Data_Morale,Data_OldWarhammerAgility,Data_OldWarhammerBallisticSkill,Data_OldWarhammerFellowship,Data_OldWarhammerIntelligence,Data_OldWarhammerPerception,Data_OldWarhammerStrength,Data_OldWarhammerToughness,Data_OldWarhammerWeaponSkill,Data_OldWarhammerWillpower,Data_Prefab_AssetId,Data_Size,Data_SpeedOnStarSystemMap,Data_Speed_m_Value,Data_StarshipSpeed,Data_Subtype,Data_WarhammerAgilitySetting_Modifier,Data_WarhammerAgilitySetting_NotModified,Data_WarhammerBallisticSkillSetting_NotModified,Data_WarhammerFellowshipSetting_NotModified,Data_WarhammerInitialAPBlue,Data_WarhammerInitialAPYellow,Data_WarhammerIntelligenceSetting_NotModified,Data_WarhammerMovementApPerCell,Data_WarhammerMovementApPerCellThreateningArea,Data_WarhammerPerceptionSetting_Modifier,Data_WarhammerPerceptionSetting_NotModified,Data_WarhammerStrengthSetting_NotModified,Data_WarhammerToughnessSetting_NotModified,Data_WarhammerWeaponSkillSett

Create Faction text column:

Create text columns for fact columns:

In [2024]:
weapon_columns = [x for x in assetIds_cols if 'Weapon' in x]
weapon_columns

['Data_HullSlots_Weapons_0_Weapon',
 'Data_HullSlots_Weapons_1_Weapon',
 'Data_HullSlots_Weapons_2_Weapon',
 'Data_HullSlots_Weapons_3_Weapon',
 'Data_HullSlots_Weapons_4_Weapon',
 'Data_HullSlots_Weapons_5_Weapon']

In [2025]:
# columns with ship weapons
weapon_columns = [x for x in assetIds_cols if 'Weapon' in x]
#get_text_info(df, map_df, index_df, columns_in, keylist = None, return_temp = False, suffix = '', format_text=False, add_aggr_col = False):
enemies_df, created_columns = get_text_info(df = enemies_df, 
                                  map_df = assets_df,
                                  index_df=index_df,
                                  columns_in= weapon_columns,
                                  keylist= None, format_text=True)
enemies_df_ru, created_columns = get_text_info(df = enemies_df_ru, 
                                  map_df = assets_df,
                                  index_df=index_df_ru,
                                  columns_in= weapon_columns,
                                  keylist= None, format_text=True)

In [2026]:
enemies_df_ru.head(2)

,AssetId,Data_$type,Data_AiDesiredDistanceToPlayer,Data_Alignment,Data_AlternativeBrains_0,Data_ApproachStarSystemObjectRadius,Data_ArmourAft,Data_ArmourFore,Data_ArmourPort,Data_ArmourStarboard,Data_Body_DisableHands,Data_Color_a,Data_Color_b,Data_Color_g,Data_Color_r,Data_Components_0_$type,Data_Components_0_CR,Data_Components_0_Encounter,Data_Components_0_HpPerUnit,Data_Components_0_LandingActions_Actions_0_$type,Data_Components_0_LandingActions_Actions_0_DC_Property,Data_Components_0_LandingActions_Actions_0_DC_PropertyName,Data_Components_0_LandingActions_Actions_0_DC_ValueRank,Data_Components_0_LandingActions_Actions_0_DC_ValueShared,Data_Components_0_LandingActions_Actions_0_DC_ValueType,Data_Components_0_LandingActions_Actions_0_SpellLevel_Property,Data_Components_0_LandingActions_Actions_0_SpellLevel_PropertyName,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueRank,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueShared,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueType,Data_Components_0_LandingActions_Actions_0_m_Spell,Data_Components_0_LandingActions_Actions_0_name,Data_Components_0_LandingActions_Actions_1_$type,Data_Components_0_LandingActions_Actions_1_NoLog,Data_Components_0_LandingActions_Actions_1_name,Data_Components_0_LandingDistance,Data_Components_0_Modifier,Data_Components_0_name,Data_Components_1_$type,Data_Components_1_ExpirationActions_Actions_0_$type,Data_Components_1_ExpirationActions_Actions_0_PrefabLink_AssetId,Data_Components_1_ExpirationActions_Actions_0_name,Data_Components_1_ExpirationActions_Actions_1_$type,Data_Components_1_ExpirationActions_Actions_1_NoExp,Data_Components_1_ExpirationActions_Actions_1_name,Data_Components_1_Rewards_0_Condition_0_HasBuff,Data_Components_1_Rewards_0_Condition_0_Not,Data_Components_1_Rewards_0_Condition_0_m_Buff,Data_Components_1_Rewards_0_ItemCounts_0,Data_Components_1_Rewards_0_ItemCounts_1,Data_Components_1_Rewards_0_Items_0,Data_Components_1_Rewards_0_Items_1,Data_Components_1_Rewards_0_Scrap,Data_Components_1_m_ExpendAllFuelOnAttack,Data_Components_1_m_FuelBuff,Data_Components_1_m_LandingBuff,Data_Components_1_m_Overrides_0,Data_Components_1_name,Data_CrewQuartersThroughput,Data_DifficultyType,Data_Evasion,Data_Gender,Data_HullIntegrity,Data_HullSlots_Weapons_0_OffsetFromProw,Data_HullSlots_Weapons_0_Type,Data_HullSlots_Weapons_0_Weapon,Data_HullSlots_Weapons_1_OffsetFromProw,Data_HullSlots_Weapons_1_Type,Data_HullSlots_Weapons_1_Weapon,Data_HullSlots_Weapons_2_OffsetFromProw,Data_HullSlots_Weapons_2_Type,Data_HullSlots_Weapons_2_Weapon,Data_HullSlots_Weapons_3_OffsetFromProw,Data_HullSlots_Weapons_3_Type,Data_HullSlots_Weapons_3_Weapon,Data_HullSlots_Weapons_4_Type,Data_HullSlots_Weapons_4_Weapon,Data_HullSlots_Weapons_5_OffsetFromProw,Data_HullSlots_Weapons_5_Type,Data_HullSlots_Weapons_5_Weapon,Data_HullSlots_m_AugerArray,Data_HullSlots_m_PlasmaDrives,Data_HullSlots_m_VoidShieldGenerator,Data_Inertia,Data_Initiative,Data_InspirationAmount,Data_IsSoftUnit,Data_LocalizedName_stringkey,Data_MaxHP,Data_MilitaryRating,Data_Morale,Data_OldWarhammerAgility,Data_OldWarhammerBallisticSkill,Data_OldWarhammerFellowship,Data_OldWarhammerIntelligence,Data_OldWarhammerPerception,Data_OldWarhammerStrength,Data_OldWarhammerToughness,Data_OldWarhammerWeaponSkill,Data_OldWarhammerWillpower,Data_Prefab_AssetId,Data_Size,Data_SpeedOnStarSystemMap,Data_Speed_m_Value,Data_StarshipSpeed,Data_Subtype,Data_WarhammerAgilitySetting_Modifier,Data_WarhammerAgilitySetting_NotModified,Data_WarhammerBallisticSkillSetting_NotModified,Data_WarhammerFellowshipSetting_NotModified,Data_WarhammerInitialAPBlue,Data_WarhammerInitialAPYellow,Data_WarhammerIntelligenceSetting_NotModified,Data_WarhammerMovementApPerCell,Data_WarhammerMovementApPerCellThreateningArea,Data_WarhammerPerceptionSetting_Modifier,Data_WarhammerPerceptionSetting_NotModified,Data_WarhammerStrengthSetting_NotModified,Data_WarhammerToughnessSetting_NotModified,Data_WarhammerWeaponSkillSett

Create columns for ```Data_HullSlots_m_AugerArray``` with hit chance and crit chance values:

"hitChances": 95,
    "critChances": 5

In [2027]:
#english version
columns = ['Data_HullSlots_m_AugerArray']
enemies_df = get_text_info(df = enemies_df, 
                                  map_df = assets_df,
                                  index_df=index_df,
                                  columns_in=columns, suffix='hitChance',
                                  keylist=['Data','hitChances'], return_temp=True )
enemies_df = get_text_info(df = enemies_df, 
                                  map_df = assets_df,
                                  index_df=index_df,
                                  columns_in=columns, suffix='critChance',
                                  keylist=['Data','critChances'], return_temp=True )
#russian version
enemies_df_ru = get_text_info(df = enemies_df_ru, 
                                  map_df = assets_df,
                                  index_df=index_df_ru,
                                  columns_in=columns, suffix='hitChance',
                                  keylist=['Data','hitChances'], return_temp=True )
enemies_df_ru = get_text_info(df = enemies_df_ru, 
                                  map_df = assets_df,
                                  index_df=index_df_ru,
                                  columns_in=columns, suffix='critChance',
                                  keylist=['Data','critChances'], return_temp=True )

In [2028]:
display(enemies_df.head(2), enemies_df_ru.head(2))

,AssetId,Data_$type,Data_AiDesiredDistanceToPlayer,Data_Alignment,Data_AlternativeBrains_0,Data_ApproachStarSystemObjectRadius,Data_ArmourAft,Data_ArmourFore,Data_ArmourPort,Data_ArmourStarboard,Data_Body_DisableHands,Data_Color_a,Data_Color_b,Data_Color_g,Data_Color_r,Data_Components_0_$type,Data_Components_0_CR,Data_Components_0_Encounter,Data_Components_0_HpPerUnit,Data_Components_0_LandingActions_Actions_0_$type,Data_Components_0_LandingActions_Actions_0_DC_Property,Data_Components_0_LandingActions_Actions_0_DC_PropertyName,Data_Components_0_LandingActions_Actions_0_DC_ValueRank,Data_Components_0_LandingActions_Actions_0_DC_ValueShared,Data_Components_0_LandingActions_Actions_0_DC_ValueType,Data_Components_0_LandingActions_Actions_0_SpellLevel_Property,Data_Components_0_LandingActions_Actions_0_SpellLevel_PropertyName,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueRank,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueShared,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueType,Data_Components_0_LandingActions_Actions_0_m_Spell,Data_Components_0_LandingActions_Actions_0_name,Data_Components_0_LandingActions_Actions_1_$type,Data_Components_0_LandingActions_Actions_1_NoLog,Data_Components_0_LandingActions_Actions_1_name,Data_Components_0_LandingDistance,Data_Components_0_Modifier,Data_Components_0_name,Data_Components_1_$type,Data_Components_1_ExpirationActions_Actions_0_$type,Data_Components_1_ExpirationActions_Actions_0_PrefabLink_AssetId,Data_Components_1_ExpirationActions_Actions_0_name,Data_Components_1_ExpirationActions_Actions_1_$type,Data_Components_1_ExpirationActions_Actions_1_NoExp,Data_Components_1_ExpirationActions_Actions_1_name,Data_Components_1_Rewards_0_Condition_0_HasBuff,Data_Components_1_Rewards_0_Condition_0_Not,Data_Components_1_Rewards_0_Condition_0_m_Buff,Data_Components_1_Rewards_0_ItemCounts_0,Data_Components_1_Rewards_0_ItemCounts_1,Data_Components_1_Rewards_0_Items_0,Data_Components_1_Rewards_0_Items_1,Data_Components_1_Rewards_0_Scrap,Data_Components_1_m_ExpendAllFuelOnAttack,Data_Components_1_m_FuelBuff,Data_Components_1_m_LandingBuff,Data_Components_1_m_Overrides_0,Data_Components_1_name,Data_CrewQuartersThroughput,Data_DifficultyType,Data_Evasion,Data_Gender,Data_HullIntegrity,Data_HullSlots_Weapons_0_OffsetFromProw,Data_HullSlots_Weapons_0_Type,Data_HullSlots_Weapons_0_Weapon,Data_HullSlots_Weapons_1_OffsetFromProw,Data_HullSlots_Weapons_1_Type,Data_HullSlots_Weapons_1_Weapon,Data_HullSlots_Weapons_2_OffsetFromProw,Data_HullSlots_Weapons_2_Type,Data_HullSlots_Weapons_2_Weapon,Data_HullSlots_Weapons_3_OffsetFromProw,Data_HullSlots_Weapons_3_Type,Data_HullSlots_Weapons_3_Weapon,Data_HullSlots_Weapons_4_Type,Data_HullSlots_Weapons_4_Weapon,Data_HullSlots_Weapons_5_OffsetFromProw,Data_HullSlots_Weapons_5_Type,Data_HullSlots_Weapons_5_Weapon,Data_HullSlots_m_AugerArray,Data_HullSlots_m_PlasmaDrives,Data_HullSlots_m_VoidShieldGenerator,Data_Inertia,Data_Initiative,Data_InspirationAmount,Data_IsSoftUnit,Data_LocalizedName_stringkey,Data_MaxHP,Data_MilitaryRating,Data_Morale,Data_OldWarhammerAgility,Data_OldWarhammerBallisticSkill,Data_OldWarhammerFellowship,Data_OldWarhammerIntelligence,Data_OldWarhammerPerception,Data_OldWarhammerStrength,Data_OldWarhammerToughness,Data_OldWarhammerWeaponSkill,Data_OldWarhammerWillpower,Data_Prefab_AssetId,Data_Size,Data_SpeedOnStarSystemMap,Data_Speed_m_Value,Data_StarshipSpeed,Data_Subtype,Data_WarhammerAgilitySetting_Modifier,Data_WarhammerAgilitySetting_NotModified,Data_WarhammerBallisticSkillSetting_NotModified,Data_WarhammerFellowshipSetting_NotModified,Data_WarhammerInitialAPBlue,Data_WarhammerInitialAPYellow,Data_WarhammerIntelligenceSetting_NotModified,Data_WarhammerMovementApPerCell,Data_WarhammerMovementApPerCellThreateningArea,Data_WarhammerPerceptionSetting_Modifier,Data_WarhammerPerceptionSetting_NotModified,Data_WarhammerStrengthSetting_NotModified,Data_WarhammerToughnessSetting_NotModified,Data_WarhammerWeaponSkillSett

,AssetId,Data_$type,Data_AiDesiredDistanceToPlayer,Data_Alignment,Data_AlternativeBrains_0,Data_ApproachStarSystemObjectRadius,Data_ArmourAft,Data_ArmourFore,Data_ArmourPort,Data_ArmourStarboard,Data_Body_DisableHands,Data_Color_a,Data_Color_b,Data_Color_g,Data_Color_r,Data_Components_0_$type,Data_Components_0_CR,Data_Components_0_Encounter,Data_Components_0_HpPerUnit,Data_Components_0_LandingActions_Actions_0_$type,Data_Components_0_LandingActions_Actions_0_DC_Property,Data_Components_0_LandingActions_Actions_0_DC_PropertyName,Data_Components_0_LandingActions_Actions_0_DC_ValueRank,Data_Components_0_LandingActions_Actions_0_DC_ValueShared,Data_Components_0_LandingActions_Actions_0_DC_ValueType,Data_Components_0_LandingActions_Actions_0_SpellLevel_Property,Data_Components_0_LandingActions_Actions_0_SpellLevel_PropertyName,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueRank,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueShared,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueType,Data_Components_0_LandingActions_Actions_0_m_Spell,Data_Components_0_LandingActions_Actions_0_name,Data_Components_0_LandingActions_Actions_1_$type,Data_Components_0_LandingActions_Actions_1_NoLog,Data_Components_0_LandingActions_Actions_1_name,Data_Components_0_LandingDistance,Data_Components_0_Modifier,Data_Components_0_name,Data_Components_1_$type,Data_Components_1_ExpirationActions_Actions_0_$type,Data_Components_1_ExpirationActions_Actions_0_PrefabLink_AssetId,Data_Components_1_ExpirationActions_Actions_0_name,Data_Components_1_ExpirationActions_Actions_1_$type,Data_Components_1_ExpirationActions_Actions_1_NoExp,Data_Components_1_ExpirationActions_Actions_1_name,Data_Components_1_Rewards_0_Condition_0_HasBuff,Data_Components_1_Rewards_0_Condition_0_Not,Data_Components_1_Rewards_0_Condition_0_m_Buff,Data_Components_1_Rewards_0_ItemCounts_0,Data_Components_1_Rewards_0_ItemCounts_1,Data_Components_1_Rewards_0_Items_0,Data_Components_1_Rewards_0_Items_1,Data_Components_1_Rewards_0_Scrap,Data_Components_1_m_ExpendAllFuelOnAttack,Data_Components_1_m_FuelBuff,Data_Components_1_m_LandingBuff,Data_Components_1_m_Overrides_0,Data_Components_1_name,Data_CrewQuartersThroughput,Data_DifficultyType,Data_Evasion,Data_Gender,Data_HullIntegrity,Data_HullSlots_Weapons_0_OffsetFromProw,Data_HullSlots_Weapons_0_Type,Data_HullSlots_Weapons_0_Weapon,Data_HullSlots_Weapons_1_OffsetFromProw,Data_HullSlots_Weapons_1_Type,Data_HullSlots_Weapons_1_Weapon,Data_HullSlots_Weapons_2_OffsetFromProw,Data_HullSlots_Weapons_2_Type,Data_HullSlots_Weapons_2_Weapon,Data_HullSlots_Weapons_3_OffsetFromProw,Data_HullSlots_Weapons_3_Type,Data_HullSlots_Weapons_3_Weapon,Data_HullSlots_Weapons_4_Type,Data_HullSlots_Weapons_4_Weapon,Data_HullSlots_Weapons_5_OffsetFromProw,Data_HullSlots_Weapons_5_Type,Data_HullSlots_Weapons_5_Weapon,Data_HullSlots_m_AugerArray,Data_HullSlots_m_PlasmaDrives,Data_HullSlots_m_VoidShieldGenerator,Data_Inertia,Data_Initiative,Data_InspirationAmount,Data_IsSoftUnit,Data_LocalizedName_stringkey,Data_MaxHP,Data_MilitaryRating,Data_Morale,Data_OldWarhammerAgility,Data_OldWarhammerBallisticSkill,Data_OldWarhammerFellowship,Data_OldWarhammerIntelligence,Data_OldWarhammerPerception,Data_OldWarhammerStrength,Data_OldWarhammerToughness,Data_OldWarhammerWeaponSkill,Data_OldWarhammerWillpower,Data_Prefab_AssetId,Data_Size,Data_SpeedOnStarSystemMap,Data_Speed_m_Value,Data_StarshipSpeed,Data_Subtype,Data_WarhammerAgilitySetting_Modifier,Data_WarhammerAgilitySetting_NotModified,Data_WarhammerBallisticSkillSetting_NotModified,Data_WarhammerFellowshipSetting_NotModified,Data_WarhammerInitialAPBlue,Data_WarhammerInitialAPYellow,Data_WarhammerIntelligenceSetting_NotModified,Data_WarhammerMovementApPerCell,Data_WarhammerMovementApPerCellThreateningArea,Data_WarhammerPerceptionSetting_Modifier,Data_WarhammerPerceptionSetting_NotModified,Data_WarhammerStrengthSetting_NotModified,Data_WarhammerToughnessSetting_NotModified,Data_WarhammerWeaponSkillSett

Create columns for shield generator column ```Data_HullSlots_m_VoidShieldGenerator```

"Fore": 75,
    "Port": 75,
    "Starboard": 75,
    "Aft": 75,
    "damageExtraResistance": "Direct",
    "extraResistanceDamageReductionPercent": 0

Fore shield:

In [2029]:
columns = ['Data_HullSlots_m_VoidShieldGenerator']
#english version
enemies_df = get_text_info(df = enemies_df, 
                                  map_df = assets_df,
                                  index_df=index_df,
                                  columns_in=columns, suffix='shield_fore',
                                  keylist=['Data','Fore'], return_temp=True )
# russian version
enemies_df_ru = get_text_info(df = enemies_df_ru, 
                                  map_df = assets_df,
                                  index_df=index_df_ru,
                                  columns_in=columns, suffix='shield_fore',
                                  keylist=['Data','Fore'], return_temp=True )

Port shield:

In [2030]:
columns = ['Data_HullSlots_m_VoidShieldGenerator']
#english version
enemies_df = get_text_info(df = enemies_df, 
                                  map_df = assets_df,
                                  index_df=index_df,
                                  columns_in=columns, suffix='shield_port',
                                  keylist=['Data','Port'], return_temp=True )
# russian version
enemies_df_ru = get_text_info(df = enemies_df_ru, 
                                  map_df = assets_df,
                                  index_df=index_df_ru,
                                  columns_in=columns, suffix='shield_port',
                                  keylist=['Data','Port'], return_temp=True )

Starboard shield:

In [2031]:
columns = ['Data_HullSlots_m_VoidShieldGenerator']
#english version
enemies_df = get_text_info(df = enemies_df, 
                                  map_df = assets_df,
                                  index_df=index_df,
                                  columns_in=columns, suffix='shield_starboard',
                                  keylist=['Data','Starboard'], return_temp=True )
# russian version
enemies_df_ru = get_text_info(df = enemies_df_ru, 
                                  map_df = assets_df,
                                  index_df=index_df_ru,
                                  columns_in=columns, suffix='shield_starboard',
                                  keylist=['Data','Starboard'], return_temp=True )

Aft shield:

In [2032]:
columns = ['Data_HullSlots_m_VoidShieldGenerator']
#english version
enemies_df = get_text_info(df = enemies_df, 
                                  map_df = assets_df,
                                  index_df=index_df,
                                  columns_in=columns, suffix='shield_aft',
                                  keylist=['Data','Aft'], return_temp=True )
# russian version
enemies_df_ru = get_text_info(df = enemies_df_ru, 
                                  map_df = assets_df,
                                  index_df=index_df_ru,
                                  columns_in=columns, suffix='shield_aft',
                                  keylist=['Data','Aft'], return_temp=True )

Fill shield columns with zeroes if its empty, so it means that it has no shields:

In [2033]:
shield_columns = ['Data_HullSlots_m_VoidShieldGenerator_shield_fore',	'Data_HullSlots_m_VoidShieldGenerator_shield_port',
                  	'Data_HullSlots_m_VoidShieldGenerator_shield_starboard',	'Data_HullSlots_m_VoidShieldGenerator_shield_aft']
#english version
for col in enemies_df[shield_columns].columns:
    enemies_df.loc[enemies_df[col]=="", col] = 0
# russian version
for col in enemies_df_ru[shield_columns].columns:
    enemies_df_ru.loc[enemies_df_ru[col]=="", col] = 0

Create an aggregate column for shield columns:

In [2034]:
#english version
enemies_df['Shields_Stats'] = ('Fore: ' + enemies_df['Data_HullSlots_m_VoidShieldGenerator_shield_fore'].astype('str') + ', '
                               + 'Port: ' + enemies_df['Data_HullSlots_m_VoidShieldGenerator_shield_port'].astype('str') + ', '
                               + 'Starboard: ' + enemies_df['Data_HullSlots_m_VoidShieldGenerator_shield_starboard'].astype('str') + ', '
                               + 'Aft: ' + enemies_df['Data_HullSlots_m_VoidShieldGenerator_shield_aft'].astype('str') + ', '
                                )

#russian version
enemies_df_ru['Shields_Stats'] = ('Нос: ' + enemies_df_ru['Data_HullSlots_m_VoidShieldGenerator_shield_fore'].astype('str') + ', '
                               + 'Левый борт: ' + enemies_df_ru['Data_HullSlots_m_VoidShieldGenerator_shield_port'].astype('str') + ', '
                               + 'Правый борт: ' + enemies_df_ru['Data_HullSlots_m_VoidShieldGenerator_shield_starboard'].astype('str') + ', '
                               + 'Корма: ' + enemies_df_ru['Data_HullSlots_m_VoidShieldGenerator_shield_aft'].astype('str') + ', '
                                )
display(enemies_df.head(2), enemies_df_ru.head(2))

,AssetId,Data_$type,Data_AiDesiredDistanceToPlayer,Data_Alignment,Data_AlternativeBrains_0,Data_ApproachStarSystemObjectRadius,Data_ArmourAft,Data_ArmourFore,Data_ArmourPort,Data_ArmourStarboard,Data_Body_DisableHands,Data_Color_a,Data_Color_b,Data_Color_g,Data_Color_r,Data_Components_0_$type,Data_Components_0_CR,Data_Components_0_Encounter,Data_Components_0_HpPerUnit,Data_Components_0_LandingActions_Actions_0_$type,Data_Components_0_LandingActions_Actions_0_DC_Property,Data_Components_0_LandingActions_Actions_0_DC_PropertyName,Data_Components_0_LandingActions_Actions_0_DC_ValueRank,Data_Components_0_LandingActions_Actions_0_DC_ValueShared,Data_Components_0_LandingActions_Actions_0_DC_ValueType,Data_Components_0_LandingActions_Actions_0_SpellLevel_Property,Data_Components_0_LandingActions_Actions_0_SpellLevel_PropertyName,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueRank,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueShared,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueType,Data_Components_0_LandingActions_Actions_0_m_Spell,Data_Components_0_LandingActions_Actions_0_name,Data_Components_0_LandingActions_Actions_1_$type,Data_Components_0_LandingActions_Actions_1_NoLog,Data_Components_0_LandingActions_Actions_1_name,Data_Components_0_LandingDistance,Data_Components_0_Modifier,Data_Components_0_name,Data_Components_1_$type,Data_Components_1_ExpirationActions_Actions_0_$type,Data_Components_1_ExpirationActions_Actions_0_PrefabLink_AssetId,Data_Components_1_ExpirationActions_Actions_0_name,Data_Components_1_ExpirationActions_Actions_1_$type,Data_Components_1_ExpirationActions_Actions_1_NoExp,Data_Components_1_ExpirationActions_Actions_1_name,Data_Components_1_Rewards_0_Condition_0_HasBuff,Data_Components_1_Rewards_0_Condition_0_Not,Data_Components_1_Rewards_0_Condition_0_m_Buff,Data_Components_1_Rewards_0_ItemCounts_0,Data_Components_1_Rewards_0_ItemCounts_1,Data_Components_1_Rewards_0_Items_0,Data_Components_1_Rewards_0_Items_1,Data_Components_1_Rewards_0_Scrap,Data_Components_1_m_ExpendAllFuelOnAttack,Data_Components_1_m_FuelBuff,Data_Components_1_m_LandingBuff,Data_Components_1_m_Overrides_0,Data_Components_1_name,Data_CrewQuartersThroughput,Data_DifficultyType,Data_Evasion,Data_Gender,Data_HullIntegrity,Data_HullSlots_Weapons_0_OffsetFromProw,Data_HullSlots_Weapons_0_Type,Data_HullSlots_Weapons_0_Weapon,Data_HullSlots_Weapons_1_OffsetFromProw,Data_HullSlots_Weapons_1_Type,Data_HullSlots_Weapons_1_Weapon,Data_HullSlots_Weapons_2_OffsetFromProw,Data_HullSlots_Weapons_2_Type,Data_HullSlots_Weapons_2_Weapon,Data_HullSlots_Weapons_3_OffsetFromProw,Data_HullSlots_Weapons_3_Type,Data_HullSlots_Weapons_3_Weapon,Data_HullSlots_Weapons_4_Type,Data_HullSlots_Weapons_4_Weapon,Data_HullSlots_Weapons_5_OffsetFromProw,Data_HullSlots_Weapons_5_Type,Data_HullSlots_Weapons_5_Weapon,Data_HullSlots_m_AugerArray,Data_HullSlots_m_PlasmaDrives,Data_HullSlots_m_VoidShieldGenerator,Data_Inertia,Data_Initiative,Data_InspirationAmount,Data_IsSoftUnit,Data_LocalizedName_stringkey,Data_MaxHP,Data_MilitaryRating,Data_Morale,Data_OldWarhammerAgility,Data_OldWarhammerBallisticSkill,Data_OldWarhammerFellowship,Data_OldWarhammerIntelligence,Data_OldWarhammerPerception,Data_OldWarhammerStrength,Data_OldWarhammerToughness,Data_OldWarhammerWeaponSkill,Data_OldWarhammerWillpower,Data_Prefab_AssetId,Data_Size,Data_SpeedOnStarSystemMap,Data_Speed_m_Value,Data_StarshipSpeed,Data_Subtype,Data_WarhammerAgilitySetting_Modifier,Data_WarhammerAgilitySetting_NotModified,Data_WarhammerBallisticSkillSetting_NotModified,Data_WarhammerFellowshipSetting_NotModified,Data_WarhammerInitialAPBlue,Data_WarhammerInitialAPYellow,Data_WarhammerIntelligenceSetting_NotModified,Data_WarhammerMovementApPerCell,Data_WarhammerMovementApPerCellThreateningArea,Data_WarhammerPerceptionSetting_Modifier,Data_WarhammerPerceptionSetting_NotModified,Data_WarhammerStrengthSetting_NotModified,Data_WarhammerToughnessSetting_NotModified,Data_WarhammerWeaponSkillSett

,AssetId,Data_$type,Data_AiDesiredDistanceToPlayer,Data_Alignment,Data_AlternativeBrains_0,Data_ApproachStarSystemObjectRadius,Data_ArmourAft,Data_ArmourFore,Data_ArmourPort,Data_ArmourStarboard,Data_Body_DisableHands,Data_Color_a,Data_Color_b,Data_Color_g,Data_Color_r,Data_Components_0_$type,Data_Components_0_CR,Data_Components_0_Encounter,Data_Components_0_HpPerUnit,Data_Components_0_LandingActions_Actions_0_$type,Data_Components_0_LandingActions_Actions_0_DC_Property,Data_Components_0_LandingActions_Actions_0_DC_PropertyName,Data_Components_0_LandingActions_Actions_0_DC_ValueRank,Data_Components_0_LandingActions_Actions_0_DC_ValueShared,Data_Components_0_LandingActions_Actions_0_DC_ValueType,Data_Components_0_LandingActions_Actions_0_SpellLevel_Property,Data_Components_0_LandingActions_Actions_0_SpellLevel_PropertyName,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueRank,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueShared,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueType,Data_Components_0_LandingActions_Actions_0_m_Spell,Data_Components_0_LandingActions_Actions_0_name,Data_Components_0_LandingActions_Actions_1_$type,Data_Components_0_LandingActions_Actions_1_NoLog,Data_Components_0_LandingActions_Actions_1_name,Data_Components_0_LandingDistance,Data_Components_0_Modifier,Data_Components_0_name,Data_Components_1_$type,Data_Components_1_ExpirationActions_Actions_0_$type,Data_Components_1_ExpirationActions_Actions_0_PrefabLink_AssetId,Data_Components_1_ExpirationActions_Actions_0_name,Data_Components_1_ExpirationActions_Actions_1_$type,Data_Components_1_ExpirationActions_Actions_1_NoExp,Data_Components_1_ExpirationActions_Actions_1_name,Data_Components_1_Rewards_0_Condition_0_HasBuff,Data_Components_1_Rewards_0_Condition_0_Not,Data_Components_1_Rewards_0_Condition_0_m_Buff,Data_Components_1_Rewards_0_ItemCounts_0,Data_Components_1_Rewards_0_ItemCounts_1,Data_Components_1_Rewards_0_Items_0,Data_Components_1_Rewards_0_Items_1,Data_Components_1_Rewards_0_Scrap,Data_Components_1_m_ExpendAllFuelOnAttack,Data_Components_1_m_FuelBuff,Data_Components_1_m_LandingBuff,Data_Components_1_m_Overrides_0,Data_Components_1_name,Data_CrewQuartersThroughput,Data_DifficultyType,Data_Evasion,Data_Gender,Data_HullIntegrity,Data_HullSlots_Weapons_0_OffsetFromProw,Data_HullSlots_Weapons_0_Type,Data_HullSlots_Weapons_0_Weapon,Data_HullSlots_Weapons_1_OffsetFromProw,Data_HullSlots_Weapons_1_Type,Data_HullSlots_Weapons_1_Weapon,Data_HullSlots_Weapons_2_OffsetFromProw,Data_HullSlots_Weapons_2_Type,Data_HullSlots_Weapons_2_Weapon,Data_HullSlots_Weapons_3_OffsetFromProw,Data_HullSlots_Weapons_3_Type,Data_HullSlots_Weapons_3_Weapon,Data_HullSlots_Weapons_4_Type,Data_HullSlots_Weapons_4_Weapon,Data_HullSlots_Weapons_5_OffsetFromProw,Data_HullSlots_Weapons_5_Type,Data_HullSlots_Weapons_5_Weapon,Data_HullSlots_m_AugerArray,Data_HullSlots_m_PlasmaDrives,Data_HullSlots_m_VoidShieldGenerator,Data_Inertia,Data_Initiative,Data_InspirationAmount,Data_IsSoftUnit,Data_LocalizedName_stringkey,Data_MaxHP,Data_MilitaryRating,Data_Morale,Data_OldWarhammerAgility,Data_OldWarhammerBallisticSkill,Data_OldWarhammerFellowship,Data_OldWarhammerIntelligence,Data_OldWarhammerPerception,Data_OldWarhammerStrength,Data_OldWarhammerToughness,Data_OldWarhammerWeaponSkill,Data_OldWarhammerWillpower,Data_Prefab_AssetId,Data_Size,Data_SpeedOnStarSystemMap,Data_Speed_m_Value,Data_StarshipSpeed,Data_Subtype,Data_WarhammerAgilitySetting_Modifier,Data_WarhammerAgilitySetting_NotModified,Data_WarhammerBallisticSkillSetting_NotModified,Data_WarhammerFellowshipSetting_NotModified,Data_WarhammerInitialAPBlue,Data_WarhammerInitialAPYellow,Data_WarhammerIntelligenceSetting_NotModified,Data_WarhammerMovementApPerCell,Data_WarhammerMovementApPerCellThreateningArea,Data_WarhammerPerceptionSetting_Modifier,Data_WarhammerPerceptionSetting_NotModified,Data_WarhammerStrengthSetting_NotModified,Data_WarhammerToughnessSetting_NotModified,Data_WarhammerWeaponSkillSett

Damage resistance columns:

In [2035]:
columns = ['Data_HullSlots_m_VoidShieldGenerator']
#english version
enemies_df = get_text_info(df = enemies_df, 
                                  map_df = assets_df,
                                  index_df=index_df,
                                  columns_in=columns, suffix='shield_dmg_resist',
                                  keylist=['Data','damageExtraResistance'], return_temp=True )
enemies_df = get_text_info(df = enemies_df, 
                                  map_df = assets_df,
                                  index_df=index_df,
                                  columns_in=columns, suffix='shield_dmg_resist_percent',
                                  keylist=['Data','extraResistanceDamageReductionPercent'], return_temp=True )


# russian version
enemies_df_ru = get_text_info(df = enemies_df_ru, 
                                  map_df = assets_df,
                                  index_df=index_df_ru,
                                  columns_in=columns, suffix='shield_dmg_resist',
                                  keylist=['Data','damageExtraResistance'], return_temp=True )
enemies_df_ru = get_text_info(df = enemies_df_ru, 
                                  map_df = assets_df,
                                  index_df=index_df_ru,
                                  columns_in=columns, suffix='shield_dmg_resist_percent',
                                  keylist=['Data','extraResistanceDamageReductionPercent'], return_temp=True )

In [2036]:
display(enemies_df.head(2), enemies_df_ru.head(2))

,AssetId,Data_$type,Data_AiDesiredDistanceToPlayer,Data_Alignment,Data_AlternativeBrains_0,Data_ApproachStarSystemObjectRadius,Data_ArmourAft,Data_ArmourFore,Data_ArmourPort,Data_ArmourStarboard,Data_Body_DisableHands,Data_Color_a,Data_Color_b,Data_Color_g,Data_Color_r,Data_Components_0_$type,Data_Components_0_CR,Data_Components_0_Encounter,Data_Components_0_HpPerUnit,Data_Components_0_LandingActions_Actions_0_$type,Data_Components_0_LandingActions_Actions_0_DC_Property,Data_Components_0_LandingActions_Actions_0_DC_PropertyName,Data_Components_0_LandingActions_Actions_0_DC_ValueRank,Data_Components_0_LandingActions_Actions_0_DC_ValueShared,Data_Components_0_LandingActions_Actions_0_DC_ValueType,Data_Components_0_LandingActions_Actions_0_SpellLevel_Property,Data_Components_0_LandingActions_Actions_0_SpellLevel_PropertyName,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueRank,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueShared,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueType,Data_Components_0_LandingActions_Actions_0_m_Spell,Data_Components_0_LandingActions_Actions_0_name,Data_Components_0_LandingActions_Actions_1_$type,Data_Components_0_LandingActions_Actions_1_NoLog,Data_Components_0_LandingActions_Actions_1_name,Data_Components_0_LandingDistance,Data_Components_0_Modifier,Data_Components_0_name,Data_Components_1_$type,Data_Components_1_ExpirationActions_Actions_0_$type,Data_Components_1_ExpirationActions_Actions_0_PrefabLink_AssetId,Data_Components_1_ExpirationActions_Actions_0_name,Data_Components_1_ExpirationActions_Actions_1_$type,Data_Components_1_ExpirationActions_Actions_1_NoExp,Data_Components_1_ExpirationActions_Actions_1_name,Data_Components_1_Rewards_0_Condition_0_HasBuff,Data_Components_1_Rewards_0_Condition_0_Not,Data_Components_1_Rewards_0_Condition_0_m_Buff,Data_Components_1_Rewards_0_ItemCounts_0,Data_Components_1_Rewards_0_ItemCounts_1,Data_Components_1_Rewards_0_Items_0,Data_Components_1_Rewards_0_Items_1,Data_Components_1_Rewards_0_Scrap,Data_Components_1_m_ExpendAllFuelOnAttack,Data_Components_1_m_FuelBuff,Data_Components_1_m_LandingBuff,Data_Components_1_m_Overrides_0,Data_Components_1_name,Data_CrewQuartersThroughput,Data_DifficultyType,Data_Evasion,Data_Gender,Data_HullIntegrity,Data_HullSlots_Weapons_0_OffsetFromProw,Data_HullSlots_Weapons_0_Type,Data_HullSlots_Weapons_0_Weapon,Data_HullSlots_Weapons_1_OffsetFromProw,Data_HullSlots_Weapons_1_Type,Data_HullSlots_Weapons_1_Weapon,Data_HullSlots_Weapons_2_OffsetFromProw,Data_HullSlots_Weapons_2_Type,Data_HullSlots_Weapons_2_Weapon,Data_HullSlots_Weapons_3_OffsetFromProw,Data_HullSlots_Weapons_3_Type,Data_HullSlots_Weapons_3_Weapon,Data_HullSlots_Weapons_4_Type,Data_HullSlots_Weapons_4_Weapon,Data_HullSlots_Weapons_5_OffsetFromProw,Data_HullSlots_Weapons_5_Type,Data_HullSlots_Weapons_5_Weapon,Data_HullSlots_m_AugerArray,Data_HullSlots_m_PlasmaDrives,Data_HullSlots_m_VoidShieldGenerator,Data_Inertia,Data_Initiative,Data_InspirationAmount,Data_IsSoftUnit,Data_LocalizedName_stringkey,Data_MaxHP,Data_MilitaryRating,Data_Morale,Data_OldWarhammerAgility,Data_OldWarhammerBallisticSkill,Data_OldWarhammerFellowship,Data_OldWarhammerIntelligence,Data_OldWarhammerPerception,Data_OldWarhammerStrength,Data_OldWarhammerToughness,Data_OldWarhammerWeaponSkill,Data_OldWarhammerWillpower,Data_Prefab_AssetId,Data_Size,Data_SpeedOnStarSystemMap,Data_Speed_m_Value,Data_StarshipSpeed,Data_Subtype,Data_WarhammerAgilitySetting_Modifier,Data_WarhammerAgilitySetting_NotModified,Data_WarhammerBallisticSkillSetting_NotModified,Data_WarhammerFellowshipSetting_NotModified,Data_WarhammerInitialAPBlue,Data_WarhammerInitialAPYellow,Data_WarhammerIntelligenceSetting_NotModified,Data_WarhammerMovementApPerCell,Data_WarhammerMovementApPerCellThreateningArea,Data_WarhammerPerceptionSetting_Modifier,Data_WarhammerPerceptionSetting_NotModified,Data_WarhammerStrengthSetting_NotModified,Data_WarhammerToughnessSetting_NotModified,Data_WarhammerWeaponSkillSett

,AssetId,Data_$type,Data_AiDesiredDistanceToPlayer,Data_Alignment,Data_AlternativeBrains_0,Data_ApproachStarSystemObjectRadius,Data_ArmourAft,Data_ArmourFore,Data_ArmourPort,Data_ArmourStarboard,Data_Body_DisableHands,Data_Color_a,Data_Color_b,Data_Color_g,Data_Color_r,Data_Components_0_$type,Data_Components_0_CR,Data_Components_0_Encounter,Data_Components_0_HpPerUnit,Data_Components_0_LandingActions_Actions_0_$type,Data_Components_0_LandingActions_Actions_0_DC_Property,Data_Components_0_LandingActions_Actions_0_DC_PropertyName,Data_Components_0_LandingActions_Actions_0_DC_ValueRank,Data_Components_0_LandingActions_Actions_0_DC_ValueShared,Data_Components_0_LandingActions_Actions_0_DC_ValueType,Data_Components_0_LandingActions_Actions_0_SpellLevel_Property,Data_Components_0_LandingActions_Actions_0_SpellLevel_PropertyName,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueRank,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueShared,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueType,Data_Components_0_LandingActions_Actions_0_m_Spell,Data_Components_0_LandingActions_Actions_0_name,Data_Components_0_LandingActions_Actions_1_$type,Data_Components_0_LandingActions_Actions_1_NoLog,Data_Components_0_LandingActions_Actions_1_name,Data_Components_0_LandingDistance,Data_Components_0_Modifier,Data_Components_0_name,Data_Components_1_$type,Data_Components_1_ExpirationActions_Actions_0_$type,Data_Components_1_ExpirationActions_Actions_0_PrefabLink_AssetId,Data_Components_1_ExpirationActions_Actions_0_name,Data_Components_1_ExpirationActions_Actions_1_$type,Data_Components_1_ExpirationActions_Actions_1_NoExp,Data_Components_1_ExpirationActions_Actions_1_name,Data_Components_1_Rewards_0_Condition_0_HasBuff,Data_Components_1_Rewards_0_Condition_0_Not,Data_Components_1_Rewards_0_Condition_0_m_Buff,Data_Components_1_Rewards_0_ItemCounts_0,Data_Components_1_Rewards_0_ItemCounts_1,Data_Components_1_Rewards_0_Items_0,Data_Components_1_Rewards_0_Items_1,Data_Components_1_Rewards_0_Scrap,Data_Components_1_m_ExpendAllFuelOnAttack,Data_Components_1_m_FuelBuff,Data_Components_1_m_LandingBuff,Data_Components_1_m_Overrides_0,Data_Components_1_name,Data_CrewQuartersThroughput,Data_DifficultyType,Data_Evasion,Data_Gender,Data_HullIntegrity,Data_HullSlots_Weapons_0_OffsetFromProw,Data_HullSlots_Weapons_0_Type,Data_HullSlots_Weapons_0_Weapon,Data_HullSlots_Weapons_1_OffsetFromProw,Data_HullSlots_Weapons_1_Type,Data_HullSlots_Weapons_1_Weapon,Data_HullSlots_Weapons_2_OffsetFromProw,Data_HullSlots_Weapons_2_Type,Data_HullSlots_Weapons_2_Weapon,Data_HullSlots_Weapons_3_OffsetFromProw,Data_HullSlots_Weapons_3_Type,Data_HullSlots_Weapons_3_Weapon,Data_HullSlots_Weapons_4_Type,Data_HullSlots_Weapons_4_Weapon,Data_HullSlots_Weapons_5_OffsetFromProw,Data_HullSlots_Weapons_5_Type,Data_HullSlots_Weapons_5_Weapon,Data_HullSlots_m_AugerArray,Data_HullSlots_m_PlasmaDrives,Data_HullSlots_m_VoidShieldGenerator,Data_Inertia,Data_Initiative,Data_InspirationAmount,Data_IsSoftUnit,Data_LocalizedName_stringkey,Data_MaxHP,Data_MilitaryRating,Data_Morale,Data_OldWarhammerAgility,Data_OldWarhammerBallisticSkill,Data_OldWarhammerFellowship,Data_OldWarhammerIntelligence,Data_OldWarhammerPerception,Data_OldWarhammerStrength,Data_OldWarhammerToughness,Data_OldWarhammerWeaponSkill,Data_OldWarhammerWillpower,Data_Prefab_AssetId,Data_Size,Data_SpeedOnStarSystemMap,Data_Speed_m_Value,Data_StarshipSpeed,Data_Subtype,Data_WarhammerAgilitySetting_Modifier,Data_WarhammerAgilitySetting_NotModified,Data_WarhammerBallisticSkillSetting_NotModified,Data_WarhammerFellowshipSetting_NotModified,Data_WarhammerInitialAPBlue,Data_WarhammerInitialAPYellow,Data_WarhammerIntelligenceSetting_NotModified,Data_WarhammerMovementApPerCell,Data_WarhammerMovementApPerCellThreateningArea,Data_WarhammerPerceptionSetting_Modifier,Data_WarhammerPerceptionSetting_NotModified,Data_WarhammerStrengthSetting_NotModified,Data_WarhammerToughnessSetting_NotModified,Data_WarhammerWeaponSkillSett

In [2037]:
for col in enemies_df.columns[-6:]:
    print(col, enemies_df[col].unique())

Data_HullSlots_m_VoidShieldGenerator_shield_port [0 105 180 75 225 30 160 210 85 95 250 35 50 70]
Data_HullSlots_m_VoidShieldGenerator_shield_starboard [0 105 180 75 225 30 160 210 85 95 250 35 50 70]
Data_HullSlots_m_VoidShieldGenerator_shield_aft [0 105 180 75 225 30 160 210 85 95 250 35 50 70]
Shields_Stats ['Fore: 0, Port: 0, Starboard: 0, Aft: 0, '
 'Fore: 105, Port: 105, Starboard: 105, Aft: 105, '
 'Fore: 180, Port: 180, Starboard: 180, Aft: 180, '
 'Fore: 75, Port: 75, Starboard: 75, Aft: 75, '
 'Fore: 225, Port: 225, Starboard: 225, Aft: 225, '
 'Fore: 30, Port: 30, Starboard: 30, Aft: 30, '
 'Fore: 160, Port: 160, Starboard: 160, Aft: 160, '
 'Fore: 210, Port: 210, Starboard: 210, Aft: 210, '
 'Fore: 85, Port: 85, Starboard: 85, Aft: 85, '
 'Fore: 95, Port: 95, Starboard: 95, Aft: 95, '
 'Fore: 250, Port: 250, Starboard: 250, Aft: 250, '
 'Fore: 35, Port: 35, Starboard: 35, Aft: 35, '
 'Fore: 50, Port: 50, Starboard: 50, Aft: 50, '
 'Fore: 70, Port: 70, Starboard: 70, Aft: 70

#### Starship weapons

**Notes**:  
Data type==```"d2513f4af02e4554bb8f9f79a5c35a46, BlueprintStarshipWeapon"``` contains information about ship weapon systems

In [2038]:
ship_weapons_df = space_combat_data[space_combat_data['Data_$type']=='d2513f4af02e4554bb8f9f79a5c35a46, BlueprintStarshipWeapon']
ship_weapons_df = ship_weapons_df.replace('[]', np.nan)
ship_weapons_df = drop_trash_columns(ship_weapons_df)
#drop unused columns
ship_weapons_df = ship_weapons_df.drop(columns = ['Data_StarshipEE_fileid',	'Data_StarshipEE_guid'], axis = 1)
#drop columns containing oonly one value or nan
dropped_cols = []
for col in ship_weapons_df:
        if(len(ship_weapons_df.loc[:,col].unique())==1):
            dropped_cols.append(col)
            ship_weapons_df.pop(col)  
print(ship_weapons_df.shape)
display(ship_weapons_df.head())

(77, 40)


,AssetId,Data_AllowedSlots_0,Data_AllowedSlots_1,Data_AllowedSlots_2,Data_Components_0_$type,Data_Components_0_m_Fact,Data_Components_0_name,Data_DamageInstances,Data_DelayBetweenProjectiles,Data_DelayInSeries,Data_RateOfFire,Data_RemoveFromSlotWhenNoCharges,Data_ShotsInSeries,Data_WeaponAbilities_Ability1_Type,Data_WeaponAbilities_Ability1_m_Ability,Data_WeaponAbilities_Ability2_Type,Data_WeaponAbilities_Ability2_m_Ability,Data_WeaponType,Data_m_AlternateAmmo,Data_m_CargoVolumePercent,Data_m_DefaultAmmo,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey,Data_m_Origin,Data_m_Overrides_0,Data_m_Overrides_1,Data_m_Overrides_10,Data_m_Overrides_2,Data_m_Overrides_3,Data_m_Overrides_4,Data_m_Overrides_5,Data_m_Overrides_6,Data_m_Overrides_7,Data_m_Overrides_8,Data_m_Overrides_9,Data_m_ProfitFactorCost,Filepath
136,c74a5fe81846426fb688f45ec9de0c99,None,NaN,NaN,NaN,NaN,NaN,1.0,0.00,NaN,1.0,True,NaN,None,32ed5ed1cf1b44059ec85ccf289e0e74,None,NaN,Other,NaN,10.0,b14a946d4cdb414ca65b430ac395f452,NaN,NaN,1815c7de-13e3-45af-9627-2e11408b215e,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Abilities\Ultabilities\UA_Astropatic_EmpyreanStorm_Weapon.jbp
204,95200d41fd964283b29c9c67880fe12a,Prow,Dorsal,NaN,NaN,NaN,NaN,4.0,0.23,0.0,NaN,False,1.0,Custom,e0a57e5237fa4bd6ad30ccb3e5c3332a,None,NaN,Lances,NaN,10.0,ec40d1ba706c45d2960d2874cdd9a472,b4933a60-cd33-4547-9991-50598f847450,NaN,e245bdce-0ac8-484c-b25d-74ea34236b6e,NaN,NaN,ShipComponents,m_Description,m_DisplayName,NaN,WeaponAbilities.Ability1.m_Ability,m_DefaultAmmo,NaN,NaN,NaN,NaN,NaN,NaN,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Components\Lances\LanceLight0_Weapon.jbp
207,ef43b2e7bba440a68931ab2c454aaded,Dorsal,NaN,NaN,NaN,NaN,NaN,4.0,0.23,0.0,NaN,False,1.0,Custom,849c9fb22efb46c6b30611302aab9ce7,None,NaN,Lances,NaN,10.0,51e409b477864bd0974ccf6743f87f9c,92f2234b-7c4e-4ed4-8c43-6b76cc8fe415,NaN,ed0c2e56-3ea4-42e2-87db-e54d28d22a48,NaN,NaN,ShipComponents,m_DisplayName,Abilities,m_Icon,m_DefaultAmmo,StarshipEE,m_Description,WeaponAbilities.Ability1.m_Ability,IsBroken,DamageInstances,DelayBetweenProjectiles,AllowedSlots,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Components\Lances\LanceLight1_Weapon.jbp
210,0f6feb3b8ad14b61aa66cedf3f780805,Dorsal,NaN,NaN,NaN,NaN,NaN,4.0,0.23,0.0,NaN,False,1.0,Custom,cfac962f1dbe4483b59411e819791fd2,None,NaN,Lances,NaN,10.0,1e887603ba4a4aaa82ad458b0bf993a2,63198466-55e4-478c-8190-db6db2ce92aa,NaN,65dc8d4a-ef78-427e-b6b4-65fbe5758954,NaN,NaN,ShipComponents,m_DisplayName,Abilities,NaN,m_DefaultAmmo,m_Description,WeaponAbilities.Ability1.m_Ability,m_Icon,NaN,NaN,NaN,NaN,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Components\Lances\LanceLight2_Weapon.jbp
213,be242f51ce154c2ca3f80bce46ddde1c,Prow,NaN,NaN,NaN,NaN,NaN,5.0,0.20,0.0,NaN,False,1.0,Custom,0277b7ed853b45bd8ced19152549fffb,None,NaN,Lances,NaN,10.0,d3a7694b208c4bf0b94b3bacb6f57032,a5218f6d-5c24-4f67-abc3-e7ffa5849096,NaN,9d6756c8-fe0f-4a41-bee7-bc4d80ad4233,NaN,NaN,ShipComponents,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Components\Lances\LanceLong1_Weapon.jbp


Check dataframe for missing values:

In [2039]:
missing_values_tab(ship_weapons_df)

Dataframe contains 40 columns and 77 strings.

It has  28 columns with missed values


,missed_values,%_of_all,data_type
Data_Components_0_$type,76,98.70,object
Data_Components_0_m_Fact,76,98.70,object
Data_Components_0_name,76,98.70,object
Data_m_DisplayName_m_Key,75,97.40,object
Data_m_Overrides_10,75,97.40,object
Data_m_Overrides_9,74,96.10,object
Data_AllowedSlots_2,74,96.10,object
Data_m_FlavorText_Shared_stringkey,72,93.51,object
Data_m_Overrides_8,71,92.21,object
Data_m_Overrides_7,69,89.61,object


Calculate column groups:

In [2040]:
keywords = []
selected_cols, assetIds_cols, stringkey_cols, property_cols, misc_cols = select_columns(ship_weapons_df, keywords)
#fill nans in string key columns with ''
ship_weapons_df[stringkey_cols] = ship_weapons_df[stringkey_cols].fillna('')

Create columns with weapon names and descriptions.   
Fill missed values in ```Data_m_DisplayName_Shared_stringkey``` by the values from ```'Data_m_DisplayName_m_Key'``` if possible.

In [2041]:
ship_weapons_df['Data_m_DisplayName_Shared_stringkey'] = np.where(ship_weapons_df['Data_m_DisplayName_Shared_stringkey']=="",
                                                                  ship_weapons_df['Data_m_DisplayName_m_Key'],
                                                                  ship_weapons_df['Data_m_DisplayName_Shared_stringkey'])
# for sescription columns
ship_weapons_df['Data_m_Description_Shared_stringkey'] = np.where(ship_weapons_df['Data_m_Description_Shared_stringkey']=="",
                                                                  ship_weapons_df['Data_m_Description_m_Key'],
                                                                  ship_weapons_df['Data_m_Description_Shared_stringkey'])
display(ship_weapons_df[(ship_weapons_df['Data_m_DisplayName_Shared_stringkey']!="") & 
                        (ship_weapons_df['Data_m_DisplayName_m_Key']!="")][stringkey_cols].head())
print('The number of rows without names: ', ship_weapons_df[ship_weapons_df['Data_m_DisplayName_Shared_stringkey']==""].shape[0])

,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey
582,01328751-ce17-48f9-9517-f7c68b51a20c,,a3dea405-8807-4871-acb3-e390ed1885ad,a3dea405-8807-4871-acb3-e390ed1885ad,
595,01328751-ce17-48f9-9517-f7c68b51a20c,,2a21acde-0696-4ca9-b89b-ca1d685a4fb8,2a21acde-0696-4ca9-b89b-ca1d685a4fb8,


The number of rows without names:  2


In [2042]:
#make separate copy for russian version
ship_weapons_df_ru = ship_weapons_df.copy(deep=True)

In [2043]:
#english version, weapon names
ship_weapons_df = create_text_columns(df = ship_weapons_df,
                                        index_df = index_df,
                                        columns_in=['Data_m_DisplayName_Shared_stringkey'],
                                        column_out='Weapon_Name', 
                                        format_text=True)
#english version, description
ship_weapons_df = create_text_columns(df = ship_weapons_df,
                                        index_df = index_df,
                                        columns_in=['Data_m_Description_Shared_stringkey'],
                                        column_out='Weapon_Descr', 
                                        format_text=True)

Create ship  weapon names for russian version:

In [2044]:
#english version, weapon names
ship_weapons_df_ru = create_text_columns(df = ship_weapons_df_ru,
                                        index_df = index_df_ru,
                                        columns_in=['Data_m_DisplayName_Shared_stringkey'],
                                        column_out='Weapon_Name', 
                                        format_text=True)
#english version, description
ship_weapons_df_ru = create_text_columns(df = ship_weapons_df_ru,
                                        index_df = index_df_ru,
                                        columns_in=['Data_m_Description_Shared_stringkey'],
                                        column_out='Weapon_Descr', 
                                        format_text=True)

In [2045]:
print('ship_weapons_df'.center(30))
display(ship_weapons_df.head(3))
print('ship-weapons_df_ru'.center(30))
display(ship_weapons_df_ru.head(3))

       ship_weapons_df        


,AssetId,Data_AllowedSlots_0,Data_AllowedSlots_1,Data_AllowedSlots_2,Data_Components_0_$type,Data_Components_0_m_Fact,Data_Components_0_name,Data_DamageInstances,Data_DelayBetweenProjectiles,Data_DelayInSeries,Data_RateOfFire,Data_RemoveFromSlotWhenNoCharges,Data_ShotsInSeries,Data_WeaponAbilities_Ability1_Type,Data_WeaponAbilities_Ability1_m_Ability,Data_WeaponAbilities_Ability2_Type,Data_WeaponAbilities_Ability2_m_Ability,Data_WeaponType,Data_m_AlternateAmmo,Data_m_CargoVolumePercent,Data_m_DefaultAmmo,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey,Data_m_Origin,Data_m_Overrides_0,Data_m_Overrides_1,Data_m_Overrides_10,Data_m_Overrides_2,Data_m_Overrides_3,Data_m_Overrides_4,Data_m_Overrides_5,Data_m_Overrides_6,Data_m_Overrides_7,Data_m_Overrides_8,Data_m_Overrides_9,Data_m_ProfitFactorCost,Filepath,Weapon_Name,Weapon_Descr
136,c74a5fe81846426fb688f45ec9de0c99,None,NaN,NaN,NaN,NaN,NaN,1.0,0.00,NaN,1.0,True,NaN,None,32ed5ed1cf1b44059ec85ccf289e0e74,None,NaN,Other,NaN,10.0,b14a946d4cdb414ca65b430ac395f452,,,1815c7de-13e3-45af-9627-2e11408b215e,,,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Abilities\Ultabilities\UA_Astropatic_EmpyreanStorm_Weapon.jbp,Empyrean Storm,
204,95200d41fd964283b29c9c67880fe12a,Prow,Dorsal,NaN,NaN,NaN,NaN,4.0,0.23,0.0,NaN,False,1.0,Custom,e0a57e5237fa4bd6ad30ccb3e5c3332a,None,NaN,Lances,NaN,10.0,ec40d1ba706c45d2960d2874cdd9a472,b4933a60-cd33-4547-9991-50598f847450,,e245bdce-0ac8-484c-b25d-74ea34236b6e,,,ShipComponents,m_Description,m_DisplayName,NaN,WeaponAbilities.Ability1.m_Ability,m_DefaultAmmo,NaN,NaN,NaN,NaN,NaN,NaN,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Components\Lances\LanceLight0_Weapon.jbp,Starbreaker Lance Weapon (Prototype),Medium-range dorsal lance weapon dealing (from 28 to 32) damage.
207,ef43b2e7bba440a68931ab2c454aaded,Dorsal,NaN,NaN,NaN,NaN,NaN,4.0,0.23,0.0,NaN,False,1.0,Custom,849c9fb22efb46c6b30611302aab9ce7,None,NaN,Lances,NaN,10.0,51e409b477864bd0974ccf6743f87f9c,92f2234b-7c4e-4ed4-8c43-6b76cc8fe415,,ed0c2e56-3ea4-42e2-87db-e54d28d22a48,,,ShipComponents,m_DisplayName,Abilities,m_Icon,m_DefaultAmmo,StarshipEE,m_Description,WeaponAbilities.Ability1.m_Ability,IsBroken,DamageInstances,DelayBetweenProjectiles,AllowedSlots,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Components\Lances\LanceLight1_Weapon.jbp,Starbreaker Lance Weapon,Medium-range dorsal lance weapon dealing (from 32 to 40) damage.


      ship-weapons_df_ru      


,AssetId,Data_AllowedSlots_0,Data_AllowedSlots_1,Data_AllowedSlots_2,Data_Components_0_$type,Data_Components_0_m_Fact,Data_Components_0_name,Data_DamageInstances,Data_DelayBetweenProjectiles,Data_DelayInSeries,Data_RateOfFire,Data_RemoveFromSlotWhenNoCharges,Data_ShotsInSeries,Data_WeaponAbilities_Ability1_Type,Data_WeaponAbilities_Ability1_m_Ability,Data_WeaponAbilities_Ability2_Type,Data_WeaponAbilities_Ability2_m_Ability,Data_WeaponType,Data_m_AlternateAmmo,Data_m_CargoVolumePercent,Data_m_DefaultAmmo,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey,Data_m_Origin,Data_m_Overrides_0,Data_m_Overrides_1,Data_m_Overrides_10,Data_m_Overrides_2,Data_m_Overrides_3,Data_m_Overrides_4,Data_m_Overrides_5,Data_m_Overrides_6,Data_m_Overrides_7,Data_m_Overrides_8,Data_m_Overrides_9,Data_m_ProfitFactorCost,Filepath,Weapon_Name,Weapon_Descr
136,c74a5fe81846426fb688f45ec9de0c99,None,NaN,NaN,NaN,NaN,NaN,1.0,0.00,NaN,1.0,True,NaN,None,32ed5ed1cf1b44059ec85ccf289e0e74,None,NaN,Other,NaN,10.0,b14a946d4cdb414ca65b430ac395f452,,,1815c7de-13e3-45af-9627-2e11408b215e,,,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Abilities\Ultabilities\UA_Astropatic_EmpyreanStorm_Weapon.jbp,Шторм Эмпирей,
204,95200d41fd964283b29c9c67880fe12a,Prow,Dorsal,NaN,NaN,NaN,NaN,4.0,0.23,0.0,NaN,False,1.0,Custom,e0a57e5237fa4bd6ad30ccb3e5c3332a,None,NaN,Lances,NaN,10.0,ec40d1ba706c45d2960d2874cdd9a472,b4933a60-cd33-4547-9991-50598f847450,,e245bdce-0ac8-484c-b25d-74ea34236b6e,,,ShipComponents,m_Description,m_DisplayName,NaN,WeaponAbilities.Ability1.m_Ability,m_DefaultAmmo,NaN,NaN,NaN,NaN,NaN,NaN,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Components\Lances\LanceLight0_Weapon.jbp,"Лэнс-излучатель ""Звездолом"" (прототип)","Башенный лэнс-излучатель средней дальности, который наносит (от 28 до 32) урона."
207,ef43b2e7bba440a68931ab2c454aaded,Dorsal,NaN,NaN,NaN,NaN,NaN,4.0,0.23,0.0,NaN,False,1.0,Custom,849c9fb22efb46c6b30611302aab9ce7,None,NaN,Lances,NaN,10.0,51e409b477864bd0974ccf6743f87f9c,92f2234b-7c4e-4ed4-8c43-6b76cc8fe415,,ed0c2e56-3ea4-42e2-87db-e54d28d22a48,,,ShipComponents,m_DisplayName,Abilities,m_Icon,m_DefaultAmmo,StarshipEE,m_Description,WeaponAbilities.Ability1.m_Ability,IsBroken,DamageInstances,DelayBetweenProjectiles,AllowedSlots,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Components\Lances\LanceLight1_Weapon.jbp,"Лэнс-излучатель ""Звездолом""","Башенный лэнс-излучатель средней дальности, который наносит (от 32 до 40) урона."


Create columns for other properties:

In [2046]:
#english version
columns = ['Data_WeaponAbilities_Ability1_m_Ability', 'Data_WeaponAbilities_Ability2_m_Ability','Data_m_DefaultAmmo']
ship_weapons_df, created_columns_eng = get_text_info(df = ship_weapons_df, 
                                  map_df = assets_df,
                                  index_df=index_df,
                                  columns_in=columns,
                                  keylist=None, format_text=True)
print('How many weapons have two abilities: ',ship_weapons_df.query('Data_WeaponAbilities_Ability2_m_Ability_text!=""').shape[0])
display(ship_weapons_df.head(2))

#russian version
columns = ['Data_WeaponAbilities_Ability1_m_Ability', 'Data_WeaponAbilities_Ability2_m_Ability','Data_m_DefaultAmmo']
ship_weapons_df_ru, created_columns_ru = get_text_info(df = ship_weapons_df_ru, 
                                  map_df = assets_df,
                                  index_df=index_df_ru,
                                  columns_in=columns,
                                  keylist=None, format_text=True)
display(ship_weapons_df_ru.head(2))

How many weapons have two abilities:  9


,AssetId,Data_AllowedSlots_0,Data_AllowedSlots_1,Data_AllowedSlots_2,Data_Components_0_$type,Data_Components_0_m_Fact,Data_Components_0_name,Data_DamageInstances,Data_DelayBetweenProjectiles,Data_DelayInSeries,Data_RateOfFire,Data_RemoveFromSlotWhenNoCharges,Data_ShotsInSeries,Data_WeaponAbilities_Ability1_Type,Data_WeaponAbilities_Ability1_m_Ability,Data_WeaponAbilities_Ability2_Type,Data_WeaponAbilities_Ability2_m_Ability,Data_WeaponType,Data_m_AlternateAmmo,Data_m_CargoVolumePercent,Data_m_DefaultAmmo,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey,Data_m_Origin,Data_m_Overrides_0,Data_m_Overrides_1,Data_m_Overrides_10,Data_m_Overrides_2,Data_m_Overrides_3,Data_m_Overrides_4,Data_m_Overrides_5,Data_m_Overrides_6,Data_m_Overrides_7,Data_m_Overrides_8,Data_m_Overrides_9,Data_m_ProfitFactorCost,Filepath,Weapon_Name,Weapon_Descr,Data_WeaponAbilities_Ability1_m_Ability_text,Data_WeaponAbilities_Ability2_m_Ability_text,Data_m_DefaultAmmo_text
136,c74a5fe81846426fb688f45ec9de0c99,None,NaN,NaN,NaN,NaN,NaN,1.0,0.00,NaN,1.0,True,NaN,None,32ed5ed1cf1b44059ec85ccf289e0e74,None,NaN,Other,NaN,10.0,b14a946d4cdb414ca65b430ac395f452,,,1815c7de-13e3-45af-9627-2e11408b215e,,,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Abilities\Ultabilities\UA_Astropatic_EmpyreanStorm_Weapon.jbp,Empyrean Storm,,Empyrean Storm,,Empyrean Storm
204,95200d41fd964283b29c9c67880fe12a,Prow,Dorsal,NaN,NaN,NaN,NaN,4.0,0.23,0.0,NaN,False,1.0,Custom,e0a57e5237fa4bd6ad30ccb3e5c3332a,None,NaN,Lances,NaN,10.0,ec40d1ba706c45d2960d2874cdd9a472,b4933a60-cd33-4547-9991-50598f847450,,e245bdce-0ac8-484c-b25d-74ea34236b6e,,,ShipComponents,m_Description,m_DisplayName,NaN,WeaponAbilities.Ability1.m_Ability,m_DefaultAmmo,NaN,NaN,NaN,NaN,NaN,NaN,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Components\Lances\LanceLight0_Weapon.jbp,Starbreaker Lance Weapon (Prototype),Medium-range dorsal lance weapon dealing (from 28 to 32) damage.,Starbreaker Lance Weapon (Prototype),,Starbreaker Lance Weapon


,AssetId,Data_AllowedSlots_0,Data_AllowedSlots_1,Data_AllowedSlots_2,Data_Components_0_$type,Data_Components_0_m_Fact,Data_Components_0_name,Data_DamageInstances,Data_DelayBetweenProjectiles,Data_DelayInSeries,Data_RateOfFire,Data_RemoveFromSlotWhenNoCharges,Data_ShotsInSeries,Data_WeaponAbilities_Ability1_Type,Data_WeaponAbilities_Ability1_m_Ability,Data_WeaponAbilities_Ability2_Type,Data_WeaponAbilities_Ability2_m_Ability,Data_WeaponType,Data_m_AlternateAmmo,Data_m_CargoVolumePercent,Data_m_DefaultAmmo,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey,Data_m_Origin,Data_m_Overrides_0,Data_m_Overrides_1,Data_m_Overrides_10,Data_m_Overrides_2,Data_m_Overrides_3,Data_m_Overrides_4,Data_m_Overrides_5,Data_m_Overrides_6,Data_m_Overrides_7,Data_m_Overrides_8,Data_m_Overrides_9,Data_m_ProfitFactorCost,Filepath,Weapon_Name,Weapon_Descr,Data_WeaponAbilities_Ability1_m_Ability_text,Data_WeaponAbilities_Ability2_m_Ability_text,Data_m_DefaultAmmo_text
136,c74a5fe81846426fb688f45ec9de0c99,None,NaN,NaN,NaN,NaN,NaN,1.0,0.00,NaN,1.0,True,NaN,None,32ed5ed1cf1b44059ec85ccf289e0e74,None,NaN,Other,NaN,10.0,b14a946d4cdb414ca65b430ac395f452,,,1815c7de-13e3-45af-9627-2e11408b215e,,,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Abilities\Ultabilities\UA_Astropatic_EmpyreanStorm_Weapon.jbp,Шторм Эмпирей,,Шторм Эмпирей,,Шторм Эмпирей
204,95200d41fd964283b29c9c67880fe12a,Prow,Dorsal,NaN,NaN,NaN,NaN,4.0,0.23,0.0,NaN,False,1.0,Custom,e0a57e5237fa4bd6ad30ccb3e5c3332a,None,NaN,Lances,NaN,10.0,ec40d1ba706c45d2960d2874cdd9a472,b4933a60-cd33-4547-9991-50598f847450,,e245bdce-0ac8-484c-b25d-74ea34236b6e,,,ShipComponents,m_Description,m_DisplayName,NaN,WeaponAbilities.Ability1.m_Ability,m_DefaultAmmo,NaN,NaN,NaN,NaN,NaN,NaN,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Components\Lances\LanceLight0_Weapon.jbp,"Лэнс-излучатель ""Звездолом"" (прототип)","Башенный лэнс-излучатель средней дальности, который наносит (от 28 до 32) урона.","Лэнс-излучатель ""Звездолом"" (прототип)",,"Лэнс-излучатель ""Звездолом"""


Add damage and damage type values for ```Data_m_DefaultAmmo``` and ```Data_m_AlternateAmmo```:

In [2047]:
#english version
columns = ['Data_m_DefaultAmmo', 'Data_m_AlternateAmmo']
ship_weapons_df = get_text_info(df = ship_weapons_df, 
                                  map_df = assets_df,
                                  index_df=index_df,
                                  columns_in=columns, suffix='damage',
                                  keylist=['Data','Damage'], return_temp=True)
ship_weapons_df = get_text_info(df = ship_weapons_df, 
                                  map_df = assets_df,
                                  index_df=index_df,
                                  columns_in=columns, suffix='max_damage',
                                  keylist=['Data','MaxDamage'], return_temp=True)


ship_weapons_df = get_text_info(df = ship_weapons_df, 
                                  map_df = assets_df,
                                  index_df=index_df,
                                  columns_in=columns, suffix='damage_type',
                                  keylist=['Data','DamageType','Type'], return_temp=True)
#russian version
ship_weapons_df_ru = get_text_info(df = ship_weapons_df_ru, 
                                  map_df = assets_df,
                                  index_df=index_df_ru,
                                  columns_in=columns, suffix='damage',
                                  keylist=['Data','Damage'], return_temp=True)
ship_weapons_df_ru = get_text_info(df = ship_weapons_df_ru, 
                                  map_df = assets_df,
                                  index_df=index_df_ru,
                                  columns_in=columns, suffix='max_damage',
                                  keylist=['Data','MaxDamage'], return_temp=True)

ship_weapons_df_ru = get_text_info(df = ship_weapons_df_ru, 
                                  map_df = assets_df,
                                  index_df=index_df_ru,
                                  columns_in=columns, suffix='damage_type',
                                  keylist=['Data','DamageType','Type'], return_temp=True)

Output created columns ```ship_weapons_df```

In [2048]:
display(ship_weapons_df[['Data_m_DefaultAmmo',
                                                               'Data_m_DefaultAmmo_damage',
                                                               'Data_m_DefaultAmmo_max_damage',
                                                               'Data_m_DefaultAmmo_damage_type']].head(2))
display(ship_weapons_df_ru[['Data_m_DefaultAmmo',
                                                               'Data_m_DefaultAmmo_damage',
                                                               'Data_m_DefaultAmmo_max_damage',
                                                               'Data_m_DefaultAmmo_damage_type']].head(2))

,Data_m_DefaultAmmo,Data_m_DefaultAmmo_damage,Data_m_DefaultAmmo_max_damage,Data_m_DefaultAmmo_damage_type
136,b14a946d4cdb414ca65b430ac395f452,8,14,Warp
204,ec40d1ba706c45d2960d2874cdd9a472,7,8,Energy


,Data_m_DefaultAmmo,Data_m_DefaultAmmo_damage,Data_m_DefaultAmmo_max_damage,Data_m_DefaultAmmo_damage_type
136,b14a946d4cdb414ca65b430ac395f452,8,14,Warp
204,ec40d1ba706c45d2960d2874cdd9a472,7,8,Energy


In [2049]:
display(ship_weapons_df.query('Data_m_AlternateAmmo.notna()')[['Data_m_AlternateAmmo',
                                                               'Data_m_AlternateAmmo_damage',
                                                               'Data_m_AlternateAmmo_max_damage',
                                                               'Data_m_AlternateAmmo_damage_type']].head(2))
display(ship_weapons_df_ru.query('Data_m_AlternateAmmo.notna()')[['Data_m_AlternateAmmo',
                                                               'Data_m_AlternateAmmo_damage',
                                                               'Data_m_AlternateAmmo_max_damage',
                                                               'Data_m_AlternateAmmo_damage_type']].head(2))

,Data_m_AlternateAmmo,Data_m_AlternateAmmo_damage,Data_m_AlternateAmmo_max_damage,Data_m_AlternateAmmo_damage_type
292,aed73086c5a148828ab647eb856d3dc2,18,0,Energy
294,aed73086c5a148828ab647eb856d3dc2,18,0,Energy


,Data_m_AlternateAmmo,Data_m_AlternateAmmo_damage,Data_m_AlternateAmmo_max_damage,Data_m_AlternateAmmo_damage_type
292,aed73086c5a148828ab647eb856d3dc2,18,0,Energy
294,aed73086c5a148828ab647eb856d3dc2,18,0,Energy


Extract the value of the key```"m_ShotProjectile"``` of ```Data_m_AlternateAmmo```:

In [2050]:
#english version
columns = ['Data_m_DefaultAmmo','Data_m_AlternateAmmo']
ship_weapons_df = get_text_info(df = ship_weapons_df, 
                                  map_df = assets_df,
                                  index_df=index_df,
                                  columns_in=columns, suffix='proj_id',
                                  keylist=['Data','m_ShotProjectile'], return_temp=True )
ship_weapons_df['Data_m_AlternateAmmo_proj_id'] = ship_weapons_df['Data_m_AlternateAmmo_proj_id'].apply(clear_rows)
ship_weapons_df['Data_m_DefaultAmmo_proj_id'] = ship_weapons_df['Data_m_DefaultAmmo_proj_id'].apply(clear_rows)

#russian version
ship_weapons_df_ru = get_text_info(df = ship_weapons_df_ru, 
                                  map_df = assets_df,
                                  index_df=index_df_ru,
                                  columns_in=columns, suffix='proj_id',
                                  keylist=['Data','m_ShotProjectile'], return_temp=True)
ship_weapons_df_ru['Data_m_AlternateAmmo_proj_id'] = ship_weapons_df_ru['Data_m_AlternateAmmo_proj_id'].apply(clear_rows)
ship_weapons_df_ru['Data_m_DefaultAmmo_proj_id'] = ship_weapons_df_ru['Data_m_DefaultAmmo_proj_id'].apply(clear_rows)

In [2051]:
display(ship_weapons_df.query('Data_m_AlternateAmmo.notna()')[['Data_m_AlternateAmmo',
                                                               'Data_m_AlternateAmmo_proj_id','Data_m_DefaultAmmo_proj_id']].head(2))
display(ship_weapons_df_ru.query('Data_m_AlternateAmmo.notna()')[['Data_m_AlternateAmmo',
                                                               'Data_m_AlternateAmmo_proj_id','Data_m_DefaultAmmo_proj_id']].head(2))

,Data_m_AlternateAmmo,Data_m_AlternateAmmo_proj_id,Data_m_DefaultAmmo_proj_id
292,aed73086c5a148828ab647eb856d3dc2,5b4a75808fa04da5b1d1587bc70be028,5b4a75808fa04da5b1d1587bc70be028
294,aed73086c5a148828ab647eb856d3dc2,5b4a75808fa04da5b1d1587bc70be028,5b4a75808fa04da5b1d1587bc70be028


,Data_m_AlternateAmmo,Data_m_AlternateAmmo_proj_id,Data_m_DefaultAmmo_proj_id
292,aed73086c5a148828ab647eb856d3dc2,5b4a75808fa04da5b1d1587bc70be028,5b4a75808fa04da5b1d1587bc70be028
294,aed73086c5a148828ab647eb856d3dc2,5b4a75808fa04da5b1d1587bc70be028,5b4a75808fa04da5b1d1587bc70be028


Add value "range" for weapon abilities:

In [2052]:
#english version
columns = ['Data_WeaponAbilities_Ability1_m_Ability','Data_WeaponAbilities_Ability2_m_Ability']
ship_weapons_df = get_text_info(df = ship_weapons_df, 
                                  map_df = assets_df,
                                  index_df=index_df,
                                  columns_in=columns, suffix='range',
                                  keylist=['Data','CustomRange'], return_temp=True )
ship_weapons_df_ru = get_text_info(df = ship_weapons_df_ru, 
                                  map_df = assets_df,
                                  index_df=index_df_ru,
                                  columns_in=columns, suffix='range',
                                  keylist=['Data','CustomRange'], return_temp=True )

In [2053]:
display(
    ship_weapons_df.query('Data_WeaponAbilities_Ability2_m_Ability.notna()')
[['Data_WeaponAbilities_Ability1_m_Ability','Data_WeaponAbilities_Ability2_m_Ability',
                 'Data_WeaponAbilities_Ability1_m_Ability_range','Data_WeaponAbilities_Ability2_m_Ability_range']
                 ].head(10)
)


,Data_WeaponAbilities_Ability1_m_Ability,Data_WeaponAbilities_Ability2_m_Ability,Data_WeaponAbilities_Ability1_m_Ability_range,Data_WeaponAbilities_Ability2_m_Ability_range
307,08edc54d53124416b63f5a6c638da50d,83be4f7d3cfe4acd8c59b23e7153ec1b,0,1
308,08edc54d53124416b63f5a6c638da50d,83be4f7d3cfe4acd8c59b23e7153ec1b,0,1
309,08edc54d53124416b63f5a6c638da50d,83be4f7d3cfe4acd8c59b23e7153ec1b,0,1
425,40b0baf127574b21a3c8b6a6d67ead74,40b0baf127574b21a3c8b6a6d67ead74,14,14
534,4e3af616791d43d18cca1c3094dcfc7b,28521625389c433abd6f00571d4e3642,0,1
558,4e3af616791d43d18cca1c3094dcfc7b,28521625389c433abd6f00571d4e3642,0,1
586,4e3af616791d43d18cca1c3094dcfc7b,28521625389c433abd6f00571d4e3642,0,1
591,4e3af616791d43d18cca1c3094dcfc7b,28521625389c433abd6f00571d4e3642,0,1
610,4e3af616791d43d18cca1c3094dcfc7b,28521625389c433abd6f00571d4e3642,0,1


Change column type of ```Data_DamageInstances``` to int:

In [2054]:
#ship_weapons_df['Data_DamageInstances'].unique()
ship_weapons_df['Data_DamageInstances'] = ship_weapons_df['Data_DamageInstances'].astype('int')
ship_weapons_df_ru['Data_DamageInstances'] = ship_weapons_df_ru['Data_DamageInstances'].astype('int')

Final look of ```ship_weapons_df```:

In [2055]:
display(ship_weapons_df_ru.head(3))

,AssetId,Data_AllowedSlots_0,Data_AllowedSlots_1,Data_AllowedSlots_2,Data_Components_0_$type,Data_Components_0_m_Fact,Data_Components_0_name,Data_DamageInstances,Data_DelayBetweenProjectiles,Data_DelayInSeries,Data_RateOfFire,Data_RemoveFromSlotWhenNoCharges,Data_ShotsInSeries,Data_WeaponAbilities_Ability1_Type,Data_WeaponAbilities_Ability1_m_Ability,Data_WeaponAbilities_Ability2_Type,Data_WeaponAbilities_Ability2_m_Ability,Data_WeaponType,Data_m_AlternateAmmo,Data_m_CargoVolumePercent,Data_m_DefaultAmmo,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey,Data_m_Origin,Data_m_Overrides_0,Data_m_Overrides_1,Data_m_Overrides_10,Data_m_Overrides_2,Data_m_Overrides_3,Data_m_Overrides_4,Data_m_Overrides_5,Data_m_Overrides_6,Data_m_Overrides_7,Data_m_Overrides_8,Data_m_Overrides_9,Data_m_ProfitFactorCost,Filepath,Weapon_Name,Weapon_Descr,Data_WeaponAbilities_Ability1_m_Ability_text,Data_WeaponAbilities_Ability2_m_Ability_text,Data_m_DefaultAmmo_text,Data_m_DefaultAmmo_damage,Data_m_AlternateAmmo_damage,Data_m_DefaultAmmo_max_damage,Data_m_AlternateAmmo_max_damage,Data_m_DefaultAmmo_damage_type,Data_m_AlternateAmmo_damage_type,Data_m_DefaultAmmo_proj_id,Data_m_AlternateAmmo_proj_id,Data_WeaponAbilities_Ability1_m_Ability_range,Data_WeaponAbilities_Ability2_m_Ability_range
136,c74a5fe81846426fb688f45ec9de0c99,None,NaN,NaN,NaN,NaN,NaN,1,0.00,NaN,1.0,True,NaN,None,32ed5ed1cf1b44059ec85ccf289e0e74,None,NaN,Other,NaN,10.0,b14a946d4cdb414ca65b430ac395f452,,,1815c7de-13e3-45af-9627-2e11408b215e,,,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Abilities\Ultabilities\UA_Astropatic_EmpyreanStorm_Weapon.jbp,Шторм Эмпирей,,Шторм Эмпирей,,Шторм Эмпирей,8,,14,,Warp,,668e782663fa4a259cf705e98afeacdf,,8,
204,95200d41fd964283b29c9c67880fe12a,Prow,Dorsal,NaN,NaN,NaN,NaN,4,0.23,0.0,NaN,False,1.0,Custom,e0a57e5237fa4bd6ad30ccb3e5c3332a,None,NaN,Lances,NaN,10.0,ec40d1ba706c45d2960d2874cdd9a472,b4933a60-cd33-4547-9991-50598f847450,,e245bdce-0ac8-484c-b25d-74ea34236b6e,,,ShipComponents,m_Description,m_DisplayName,NaN,WeaponAbilities.Ability1.m_Ability,m_DefaultAmmo,NaN,NaN,NaN,NaN,NaN,NaN,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Components\Lances\LanceLight0_Weapon.jbp,"Лэнс-излучатель ""Звездолом"" (прототип)","Башенный лэнс-излучатель средней дальности, который наносит (от 28 до 32) урона.","Лэнс-излучатель ""Звездолом"" (прототип)",,"Лэнс-излучатель ""Звездолом""",7,,8,,Energy,,037a72412a3d4f849485828686385eba,,10,
207,ef43b2e7bba440a68931ab2c454aaded,Dorsal,NaN,NaN,NaN,NaN,NaN,4,0.23,0.0,NaN,False,1.0,Custom,849c9fb22efb46c6b30611302aab9ce7,None,NaN,Lances,NaN,10.0,51e409b477864bd0974ccf6743f87f9c,92f2234b-7c4e-4ed4-8c43-6b76cc8fe415,,ed0c2e56-3ea4-42e2-87db-e54d28d22a48,,,ShipComponents,m_DisplayName,Abilities,m_Icon,m_DefaultAmmo,StarshipEE,m_Description,WeaponAbilities.Ability1.m_Ability,IsBroken,DamageInstances,DelayBetweenProjectiles,AllowedSlots,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Components\Lances\LanceLight1_Weapon.jbp,"Лэнс-излучатель ""Звездолом""","Башенный лэнс-излучатель средней дальности, который наносит (от 32 до 40) урона.","Лэнс-излучатель ""Звездолом""",,"Лэнс-излучатель ""Звездолом""",8,,10,,Energy,,037a72412a3d4f849485828686385eba,,12,


In [2056]:
# list of columns of ship_weapons_df for merge
shipweapons_merge_columns = ['AssetId','Weapon_Name',	'Weapon_Descr',	'Data_WeaponType',
                            'Data_WeaponAbilities_Ability1_m_Ability_range',                            
                            'Data_m_DefaultAmmo_damage','Data_m_DefaultAmmo_max_damage',
                            'Data_DamageInstances','Data_m_DefaultAmmo_damage_type'                            
                                ]

### Player ship shields

In [2057]:
'427d75a81e7bb9a40ac9b7eed773b946, BlueprintItemVoidShieldGenerator'
shields_df = space_combat_data[space_combat_data['Data_$type']=="427d75a81e7bb9a40ac9b7eed773b946, BlueprintItemVoidShieldGenerator"]
shields_df = shields_df.replace('[]', np.nan)
shields_df = drop_trash_columns(shields_df)
#drop unused columns
shields_df = shields_df.drop(columns = ['Data_m_CargoVolumePercent', 'Data_$type', 'Data_SpendCharges', 
                                        'Data_m_ForcedRampColorPresetIndex', 'Data_m_Rarity', 'Data_Charges',
                                            	'Data_m_Origin', 'Data_m_ProfitFactorCost'], axis = 1)
display(shields_df.head())
#create separate russian version
shields_df_ru = shields_df.copy(deep = True)

,AssetId,Data_Aft,Data_Fore,Data_Port,Data_Starboard,Data_damageExtraResistance,Data_extraResistanceDamageReductionPercent,Data_m_Description_m_Key,Data_m_DisplayName_m_Key,Filepath
275,32bc21de3caf412687e4f4de3a23e864,80.0,80.0,80.0,80.0,Direct,0.0,dd5f7df3-727f-46c2-ac43-63aba887af0e,4665bba4-423a-430e-90df-42c88a2d81d8,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Components\Shields\ShieldsCommon1.jbp
276,d53287355883474ca75d5ee1d504b59f,120.0,120.0,120.0,120.0,Direct,0.0,b40e2d83-2126-4536-b2ce-b5309bc97a14,54dcfdea-73ef-4bb8-9f94-f75ffcfb403d,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Components\Shields\ShieldsCommon2.jbp
277,9e57452e94c74ff780ddd7e4ed456a62,190.0,190.0,190.0,190.0,Direct,0.0,ee071839-1ecd-4df0-aa90-bfb28aa6d473,6b919be7-036f-43b9-b10b-4f69ab474eee,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Components\Shields\ShieldsCommon3.jbp
278,8c430881d05148c983d90cb63723dfe4,70.0,70.0,70.0,70.0,Energy,50.0,f2bc98ec-b3eb-4314-b371-807f4a225cca,326ebbb7-96a8-4a76-bc4f-a9644024919e,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Components\Shields\ShieldsEnergy0.jbp
279,b43c611fbcdc48c9a35bb2f8ac96bc73,95.0,95.0,95.0,95.0,Energy,50.0,3e55944e-9a0c-43ed-969c-432810373bb7,bb2f0ee0-1afc-4521-ab78-36e3d99fcd5d,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Components\Shields\ShieldsEnergy1.jbp


Create text columns:

In [2058]:
#Fill missed values with empty rows to function to work
stringkey_cols = ['Data_m_Description_m_Key',	'Data_m_DisplayName_m_Key']
shields_df[stringkey_cols] = shields_df[stringkey_cols].fillna('')
shields_df_ru[stringkey_cols] = shields_df_ru[stringkey_cols].fillna('')

In [2059]:
#english version, names
shields_df = create_text_columns(df = shields_df,
                                        index_df = index_df,
                                        columns_in=['Data_m_DisplayName_m_Key'],
                                        column_out='Shield_Name', 
                                        format_text=True)
#english version, description
shields_df = create_text_columns(df = shields_df,
                                        index_df = index_df,
                                        columns_in=['Data_m_Description_m_Key'],
                                        column_out='Shield_Descr', 
                                        format_text=True)
#russian version,  names
shields_df_ru = create_text_columns(df = shields_df_ru,
                                        index_df = index_df_ru,
                                        columns_in=['Data_m_DisplayName_m_Key'],
                                        column_out='Shield_Name', 
                                        format_text=True)
#russian version, description
shields_df_ru = create_text_columns(df = shields_df_ru,
                                        index_df = index_df_ru,
                                        columns_in=['Data_m_Description_m_Key'],
                                        column_out='Shield_Descr', 
                                        format_text=True)


In [2060]:
display(shields_df.head(2), shields_df_ru.head(2))

,AssetId,Data_Aft,Data_Fore,Data_Port,Data_Starboard,Data_damageExtraResistance,Data_extraResistanceDamageReductionPercent,Data_m_Description_m_Key,Data_m_DisplayName_m_Key,Filepath,Shield_Name,Shield_Descr
275,32bc21de3caf412687e4f4de3a23e864,80.0,80.0,80.0,80.0,Direct,0.0,dd5f7df3-727f-46c2-ac43-63aba887af0e,4665bba4-423a-430e-90df-42c88a2d81d8,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Components\Shields\ShieldsCommon1.jbp,Single Void Shield Array,Provides 80 points of protection for each sector.
276,d53287355883474ca75d5ee1d504b59f,120.0,120.0,120.0,120.0,Direct,0.0,b40e2d83-2126-4536-b2ce-b5309bc97a14,54dcfdea-73ef-4bb8-9f94-f75ffcfb403d,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Components\Shields\ShieldsCommon2.jbp,Compound Void Shield Array,Provides 120 points of protection for each sector.


,AssetId,Data_Aft,Data_Fore,Data_Port,Data_Starboard,Data_damageExtraResistance,Data_extraResistanceDamageReductionPercent,Data_m_Description_m_Key,Data_m_DisplayName_m_Key,Filepath,Shield_Name,Shield_Descr
275,32bc21de3caf412687e4f4de3a23e864,80.0,80.0,80.0,80.0,Direct,0.0,dd5f7df3-727f-46c2-ac43-63aba887af0e,4665bba4-423a-430e-90df-42c88a2d81d8,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Components\Shields\ShieldsCommon1.jbp,Одинарный пустотный щит,Обеспечивает 60 защиты каждому сектору.
276,d53287355883474ca75d5ee1d504b59f,120.0,120.0,120.0,120.0,Direct,0.0,b40e2d83-2126-4536-b2ce-b5309bc97a14,54dcfdea-73ef-4bb8-9f94-f75ffcfb403d,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Components\Shields\ShieldsCommon2.jbp,Множественный пустотный щит,Обеспечивает 120 защиты каждому сектору.


Create tables of shields with resist bonuses to type of damage:

In [2061]:
resist_shields = shields_df[shields_df['Data_extraResistanceDamageReductionPercent']!=0]
resist_shields_ru = shields_df_ru[shields_df_ru['Data_extraResistanceDamageReductionPercent']!=0]

In [2062]:
# types of damages
resist_shields['Data_damageExtraResistance'].unique()

array(['Energy', 'Impact'], dtype=object)

Create lists of shields good against a type of damage. These lists will be mapped to ship weapons columns to create column with advisable shields against that weapon.

In [2063]:
# list of shields good against energy damage
#english 
list_shields_vs_energy = resist_shields.query('Data_damageExtraResistance=="Energy"')['Shield_Name'].unique().tolist()
# list of shields good against kinetic damage
list_shields_vs_kinetic = resist_shields.query('Data_damageExtraResistance=="Impact"')['Shield_Name'].unique().tolist()
#russian
list_shields_vs_energy_ru = resist_shields_ru.query('Data_damageExtraResistance=="Energy"')['Shield_Name'].unique().tolist()
# list of shields good against kinetic damage
list_shields_vs_kinetic_ru = resist_shields_ru.query('Data_damageExtraResistance=="Impact"')['Shield_Name'].unique().tolist()

#### Merging ```ship_weapons_df``` and ```enemies_df```

In [2064]:
display(enemies_df.head(2))

,AssetId,Data_$type,Data_AiDesiredDistanceToPlayer,Data_Alignment,Data_AlternativeBrains_0,Data_ApproachStarSystemObjectRadius,Data_ArmourAft,Data_ArmourFore,Data_ArmourPort,Data_ArmourStarboard,Data_Body_DisableHands,Data_Color_a,Data_Color_b,Data_Color_g,Data_Color_r,Data_Components_0_$type,Data_Components_0_CR,Data_Components_0_Encounter,Data_Components_0_HpPerUnit,Data_Components_0_LandingActions_Actions_0_$type,Data_Components_0_LandingActions_Actions_0_DC_Property,Data_Components_0_LandingActions_Actions_0_DC_PropertyName,Data_Components_0_LandingActions_Actions_0_DC_ValueRank,Data_Components_0_LandingActions_Actions_0_DC_ValueShared,Data_Components_0_LandingActions_Actions_0_DC_ValueType,Data_Components_0_LandingActions_Actions_0_SpellLevel_Property,Data_Components_0_LandingActions_Actions_0_SpellLevel_PropertyName,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueRank,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueShared,Data_Components_0_LandingActions_Actions_0_SpellLevel_ValueType,Data_Components_0_LandingActions_Actions_0_m_Spell,Data_Components_0_LandingActions_Actions_0_name,Data_Components_0_LandingActions_Actions_1_$type,Data_Components_0_LandingActions_Actions_1_NoLog,Data_Components_0_LandingActions_Actions_1_name,Data_Components_0_LandingDistance,Data_Components_0_Modifier,Data_Components_0_name,Data_Components_1_$type,Data_Components_1_ExpirationActions_Actions_0_$type,Data_Components_1_ExpirationActions_Actions_0_PrefabLink_AssetId,Data_Components_1_ExpirationActions_Actions_0_name,Data_Components_1_ExpirationActions_Actions_1_$type,Data_Components_1_ExpirationActions_Actions_1_NoExp,Data_Components_1_ExpirationActions_Actions_1_name,Data_Components_1_Rewards_0_Condition_0_HasBuff,Data_Components_1_Rewards_0_Condition_0_Not,Data_Components_1_Rewards_0_Condition_0_m_Buff,Data_Components_1_Rewards_0_ItemCounts_0,Data_Components_1_Rewards_0_ItemCounts_1,Data_Components_1_Rewards_0_Items_0,Data_Components_1_Rewards_0_Items_1,Data_Components_1_Rewards_0_Scrap,Data_Components_1_m_ExpendAllFuelOnAttack,Data_Components_1_m_FuelBuff,Data_Components_1_m_LandingBuff,Data_Components_1_m_Overrides_0,Data_Components_1_name,Data_CrewQuartersThroughput,Data_DifficultyType,Data_Evasion,Data_Gender,Data_HullIntegrity,Data_HullSlots_Weapons_0_OffsetFromProw,Data_HullSlots_Weapons_0_Type,Data_HullSlots_Weapons_0_Weapon,Data_HullSlots_Weapons_1_OffsetFromProw,Data_HullSlots_Weapons_1_Type,Data_HullSlots_Weapons_1_Weapon,Data_HullSlots_Weapons_2_OffsetFromProw,Data_HullSlots_Weapons_2_Type,Data_HullSlots_Weapons_2_Weapon,Data_HullSlots_Weapons_3_OffsetFromProw,Data_HullSlots_Weapons_3_Type,Data_HullSlots_Weapons_3_Weapon,Data_HullSlots_Weapons_4_Type,Data_HullSlots_Weapons_4_Weapon,Data_HullSlots_Weapons_5_OffsetFromProw,Data_HullSlots_Weapons_5_Type,Data_HullSlots_Weapons_5_Weapon,Data_HullSlots_m_AugerArray,Data_HullSlots_m_PlasmaDrives,Data_HullSlots_m_VoidShieldGenerator,Data_Inertia,Data_Initiative,Data_InspirationAmount,Data_IsSoftUnit,Data_LocalizedName_stringkey,Data_MaxHP,Data_MilitaryRating,Data_Morale,Data_OldWarhammerAgility,Data_OldWarhammerBallisticSkill,Data_OldWarhammerFellowship,Data_OldWarhammerIntelligence,Data_OldWarhammerPerception,Data_OldWarhammerStrength,Data_OldWarhammerToughness,Data_OldWarhammerWeaponSkill,Data_OldWarhammerWillpower,Data_Prefab_AssetId,Data_Size,Data_SpeedOnStarSystemMap,Data_Speed_m_Value,Data_StarshipSpeed,Data_Subtype,Data_WarhammerAgilitySetting_Modifier,Data_WarhammerAgilitySetting_NotModified,Data_WarhammerBallisticSkillSetting_NotModified,Data_WarhammerFellowshipSetting_NotModified,Data_WarhammerInitialAPBlue,Data_WarhammerInitialAPYellow,Data_WarhammerIntelligenceSetting_NotModified,Data_WarhammerMovementApPerCell,Data_WarhammerMovementApPerCellThreateningArea,Data_WarhammerPerceptionSetting_Modifier,Data_WarhammerPerceptionSetting_NotModified,Data_WarhammerStrengthSetting_NotModified,Data_WarhammerToughnessSetting_NotModified,Data_WarhammerWeaponSkillSett

Create separate dataframe from ```enemies_df``` for merge with ```ship_weapons_df```:

In [2065]:
enemies_table = enemies_df.copy(deep=True)
enemies_table_ru =enemies_df_ru.copy(deep = True)

Select columns:

In [2066]:
columns_select = ['AssetId','Ship_Name', 'Data_AiDesiredDistanceToPlayer', 'Data_AlternativeBrains_0',
       'Data_ArmourAft', 'Data_ArmourFore', 'Data_ArmourPort',
       'Data_ArmourStarboard', 
       'Data_Components_0_$type', 'Data_Components_0_CR',
       'Data_Components_0_Encounter', 'Data_Components_0_HpPerUnit',
       'Data_Components_0_Modifier', 'Data_Components_0_name',
       'Data_Components_1_$type',       
       'Data_Components_1_Rewards_0_ItemCounts_0',
       'Data_Components_1_Rewards_0_ItemCounts_1',
       'Data_Components_1_Rewards_0_Items_0',
       'Data_Components_1_Rewards_0_Items_1',
       'Data_Components_1_Rewards_0_Scrap',       
       'Data_DifficultyType', 'Data_Evasion', 'Data_HullIntegrity',       
       'Data_HullSlots_Weapons_0_Type', 'Data_HullSlots_Weapons_0_Weapon',       
       'Data_HullSlots_Weapons_1_Type', 'Data_HullSlots_Weapons_1_Weapon',       
       'Data_HullSlots_Weapons_2_Type', 'Data_HullSlots_Weapons_2_Weapon',
       'Data_HullSlots_Weapons_3_Type', 'Data_HullSlots_Weapons_3_Weapon',
       'Data_HullSlots_Weapons_4_Type', 'Data_HullSlots_Weapons_4_Weapon',
       'Data_HullSlots_Weapons_5_Type', 'Data_HullSlots_Weapons_5_Weapon',
       'Data_HullSlots_m_AugerArray',	'Data_HullSlots_m_PlasmaDrives',
	'Data_HullSlots_m_VoidShieldGenerator',
	'Data_Inertia',	'Data_Initiative',	'Data_InspirationAmount',	
       'Data_LocalizedName_stringkey',	'Data_MaxHP',	'Data_MilitaryRating',
       'Data_Size',	'Data_SpeedOnStarSystemMap',	'Data_Speed_m_Value',	'Data_StarshipSpeed',	
       'Data_m_Brain',	'Data_m_Faction',	'Data_m_ShipType',	       		
       'Data_HullSlots_Weapons_0_Weapon_text',	'Data_HullSlots_Weapons_1_Weapon_text',	
       'Data_HullSlots_Weapons_2_Weapon_text',   'Data_HullSlots_Weapons_3_Weapon_text',
       'Data_HullSlots_Weapons_4_Weapon_text',   'Data_HullSlots_Weapons_5_Weapon_text', 
       'Data_HullSlots_m_AugerArray_hitChance',	'Data_HullSlots_m_AugerArray_critChance',
       'Shields_Stats',
       'Filepath']

enemies_table = enemies_table[columns_select]
enemies_table_ru = enemies_df_ru[columns_select]

In [2067]:
#list of columns with weapon stats except weapons
ship_stats_columns = [ 'AssetId','Ship_Name', 'Data_DifficultyType', 'Data_MilitaryRating',  
                   'Data_StarshipSpeed','Data_Initiative',
                    'Data_Evasion','Data_HullIntegrity',
                    'Data_ArmourAft',	'Data_ArmourFore',	'Data_ArmourPort',	'Data_ArmourStarboard',
                    'Data_HullSlots_m_AugerArray_hitChance',	'Data_HullSlots_m_AugerArray_critChance',
                    'Shields_Stats',
                        	'Data_AiDesiredDistanceToPlayer', 'Data_Components_1_Rewards_0_Scrap','Data_Size']
enemies_stats = enemies_table[ship_stats_columns]
enemies_stats_ru = enemies_table_ru[ship_stats_columns]

In [2068]:
enemies_table.head(2)

,AssetId,Ship_Name,Data_AiDesiredDistanceToPlayer,Data_AlternativeBrains_0,Data_ArmourAft,Data_ArmourFore,Data_ArmourPort,Data_ArmourStarboard,Data_Components_0_$type,Data_Components_0_CR,Data_Components_0_Encounter,Data_Components_0_HpPerUnit,Data_Components_0_Modifier,Data_Components_0_name,Data_Components_1_$type,Data_Components_1_Rewards_0_ItemCounts_0,Data_Components_1_Rewards_0_ItemCounts_1,Data_Components_1_Rewards_0_Items_0,Data_Components_1_Rewards_0_Items_1,Data_Components_1_Rewards_0_Scrap,Data_DifficultyType,Data_Evasion,Data_HullIntegrity,Data_HullSlots_Weapons_0_Type,Data_HullSlots_Weapons_0_Weapon,Data_HullSlots_Weapons_1_Type,Data_HullSlots_Weapons_1_Weapon,Data_HullSlots_Weapons_2_Type,Data_HullSlots_Weapons_2_Weapon,Data_HullSlots_Weapons_3_Type,Data_HullSlots_Weapons_3_Weapon,Data_HullSlots_Weapons_4_Type,Data_HullSlots_Weapons_4_Weapon,Data_HullSlots_Weapons_5_Type,Data_HullSlots_Weapons_5_Weapon,Data_HullSlots_m_AugerArray,Data_HullSlots_m_PlasmaDrives,Data_HullSlots_m_VoidShieldGenerator,Data_Inertia,Data_Initiative,Data_InspirationAmount,Data_LocalizedName_stringkey,Data_MaxHP,Data_MilitaryRating,Data_Size,Data_SpeedOnStarSystemMap,Data_Speed_m_Value,Data_StarshipSpeed,Data_m_Brain,Data_m_Faction,Data_m_ShipType,Data_HullSlots_Weapons_0_Weapon_text,Data_HullSlots_Weapons_1_Weapon_text,Data_HullSlots_Weapons_2_Weapon_text,Data_HullSlots_Weapons_3_Weapon_text,Data_HullSlots_Weapons_4_Weapon_text,Data_HullSlots_Weapons_5_Weapon_text,Data_HullSlots_m_AugerArray_hitChance,Data_HullSlots_m_AugerArray_critChance,Shields_Stats,Filepath
0,817b866ad5504ecbbc38d4c61dcc415c,Aeldari Aconite,6.0,NaN,4,4,4,4,"011e862b513000f4bae31886f3489ace, Experience",6,Mob,NaN,21.5,$Experience$8fd03507-8a60-439b-9660-38d1b4174c74,"a1ed0cc8887c45d299b2774d4c525329, SpaceCombatReward",2,1,42d9df9d31994e5c8cb1969c23dd587c,4d4c0c0ed72e43809f1f6d6a0cfc5c09,33,Hard,25,146,Prow,211a20ba46b94eecb5bc0543cf912b13,Prow,21606724feed484cae18ce96f3940bc9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3e2acfd98611424b8bdf7b45ff03866f,8168ec1542ab447ea8640de56341eccc,None,3,40,3,2620ee9f-4f3d-473b-b7a5-eed3a89e1697,40,3,Frigate_1x2,6.0,30.0,16,0861b1718fbe415aa949370994cea675,0f539babafb47fe4586b719d02aff7c4,SwordClassFrigate,Pulsar Lance,Aspect Torpedo Tubes,,,,,95,5,"Fore: 0, Port: 0, Starboard: 0, Aft: 0,",e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Units\Enemies\Aeldari\AeldariFrigate6.jbp
2,148730e8be2a47f682d87c5f78602758,Aeldari Shadowhunter,6.0,NaN,0,0,0,0,"011e862b513000f4bae31886f3489ace, Experience",3,Mob,NaN,10.0,$Experience$dc38e691-7ec8-46e5-85e8-f52cec22bb34,"a1ed0cc8887c45d299b2774d4c525329, SpaceCombatReward",1,NaN,42d9df9d31994e5c8cb1969c23dd587c,NaN,15,Common,35,55,Prow,4e6b51451b2948ad9ecc97f14053e0c2,Prow,3cc4a23f89bf45eba192f172ef5df731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88cd20a9f62843f7a208434a82e2e8d7,ab6f9eb6cc0a4889b9546c840a93da87,None,2,90,3,6dc992e2-f23f-4d5f-a05e-d5f7d62e7dff,40,3,Raider_1x1,6.0,30.0,12,9eeb068399cd4f59bc5ea6662710e655,0f539babafb47fe4586b719d02aff7c4,SwordClassFrigate,Shadow Lance,Aeldari Torpedo Tubes,,,,,90,5,"Fore: 0, Port: 0, Starboard: 0, Aft: 0,",e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Units\Enemies\Aeldari\AeldariRaider3.jbp


In [2069]:
weapon_columns = ['Data_HullSlots_Weapons_0_Weapon',	'Data_HullSlots_Weapons_1_Weapon',		
                  'Data_HullSlots_Weapons_2_Weapon',   'Data_HullSlots_Weapons_3_Weapon',		
                  'Data_HullSlots_Weapons_4_Weapon',   'Data_HullSlots_Weapons_5_Weapon']
weapon_types_columns = ['Data_HullSlots_Weapons_0_Type', 'Data_HullSlots_Weapons_1_Type', 
                 'Data_HullSlots_Weapons_2_Type', 'Data_HullSlots_Weapons_3_Type', 
                   'Data_HullSlots_Weapons_4_Type',  'Data_HullSlots_Weapons_5_Type' ]

Create table of enemies equipment:

In [2070]:
# english version
enemies_equipment = pd.melt(enemies_table,id_vars=['AssetId','Ship_Name'], value_vars=weapon_columns ,
                          var_name = 'Ship_Weapon', value_name = 'Ship_WeaponId')
# Left only rows with 'Ship_WeaponID' non nan:
enemies_equipment = enemies_equipment.dropna(subset=['Ship_WeaponId'])
display(enemies_equipment.head(6))
display(enemies_equipment.query('Ship_Name=="Styx-Class Heavy Cruiser"'))

#russian version
enemies_equipment_ru = pd.melt(enemies_table_ru,id_vars=['AssetId','Ship_Name'], value_vars=weapon_columns ,
                          var_name = 'Ship_Weapon', value_name = 'Ship_WeaponId')
# Left only rows with 'Ship_WeaponID' non nan:
enemies_equipment_ru = enemies_equipment_ru.dropna(subset=['Ship_WeaponId'])

,AssetId,Ship_Name,Ship_Weapon,Ship_WeaponId
0,817b866ad5504ecbbc38d4c61dcc415c,Aeldari Aconite,Data_HullSlots_Weapons_0_Weapon,211a20ba46b94eecb5bc0543cf912b13
1,148730e8be2a47f682d87c5f78602758,Aeldari Shadowhunter,Data_HullSlots_Weapons_0_Weapon,4e6b51451b2948ad9ecc97f14053e0c2
2,ddf557a81bca41f79c04b848ac4406bb,Murder-Class Cruiser,Data_HullSlots_Weapons_0_Weapon,8a0d0c07c7ea42eea126dbefe9dc7cfc
4,2074b0cda9de41f2a446483f98605215,Hellbringer-Class Light Cruiser,Data_HullSlots_Weapons_0_Weapon,657b911e90f943b582983866f43c2082
5,c8537d688ad243b7a1c7d9f5c4c637c9,Styx-Class Heavy Cruiser,Data_HullSlots_Weapons_0_Weapon,8a0d0c07c7ea42eea126dbefe9dc7cfc
6,a3d47366902443afb53a116ee40113cb,Blasphemous Styx-Class Heavy Cruiser,Data_HullSlots_Weapons_0_Weapon,8a0d0c07c7ea42eea126dbefe9dc7cfc


,AssetId,Ship_Name,Ship_Weapon,Ship_WeaponId
5,c8537d688ad243b7a1c7d9f5c4c637c9,Styx-Class Heavy Cruiser,Data_HullSlots_Weapons_0_Weapon,8a0d0c07c7ea42eea126dbefe9dc7cfc
38,c8537d688ad243b7a1c7d9f5c4c637c9,Styx-Class Heavy Cruiser,Data_HullSlots_Weapons_1_Weapon,8a0d0c07c7ea42eea126dbefe9dc7cfc
71,c8537d688ad243b7a1c7d9f5c4c637c9,Styx-Class Heavy Cruiser,Data_HullSlots_Weapons_2_Weapon,4c642f9e138b49e7bc1147a25a04d79a
104,c8537d688ad243b7a1c7d9f5c4c637c9,Styx-Class Heavy Cruiser,Data_HullSlots_Weapons_3_Weapon,4c642f9e138b49e7bc1147a25a04d79a
137,c8537d688ad243b7a1c7d9f5c4c637c9,Styx-Class Heavy Cruiser,Data_HullSlots_Weapons_4_Weapon,4c642f9e138b49e7bc1147a25a04d79a
170,c8537d688ad243b7a1c7d9f5c4c637c9,Styx-Class Heavy Cruiser,Data_HullSlots_Weapons_5_Weapon,4c642f9e138b49e7bc1147a25a04d79a


Create dataframe of ship weapon slot locations:

In [2071]:
weapon_slots = pd.melt(enemies_table,id_vars=['AssetId','Ship_Name'], value_vars=weapon_types_columns,
                          var_name = 'Ship_Weapon', value_name = 'Weapon_Slot')
# Left only rows with 'Ship_WeaponID' non nan:
weapon_slots = weapon_slots.dropna(subset=['Weapon_Slot'])
display(weapon_slots.head())
display(weapon_slots.query('Ship_Name=="Styx-Class Heavy Cruiser"'))

#russian version
weapon_slots_ru = pd.melt(enemies_table_ru,id_vars=['AssetId','Ship_Name'], value_vars=weapon_types_columns,
                          var_name = 'Ship_Weapon', value_name = 'Weapon_Slot')
# Left only rows with 'Ship_WeaponID' non nan:
weapon_slots_ru = weapon_slots_ru.dropna(subset=['Weapon_Slot'])

,AssetId,Ship_Name,Ship_Weapon,Weapon_Slot
0,817b866ad5504ecbbc38d4c61dcc415c,Aeldari Aconite,Data_HullSlots_Weapons_0_Type,Prow
1,148730e8be2a47f682d87c5f78602758,Aeldari Shadowhunter,Data_HullSlots_Weapons_0_Type,Prow
2,ddf557a81bca41f79c04b848ac4406bb,Murder-Class Cruiser,Data_HullSlots_Weapons_0_Type,Prow
4,2074b0cda9de41f2a446483f98605215,Hellbringer-Class Light Cruiser,Data_HullSlots_Weapons_0_Type,Dorsal
5,c8537d688ad243b7a1c7d9f5c4c637c9,Styx-Class Heavy Cruiser,Data_HullSlots_Weapons_0_Type,Dorsal


,AssetId,Ship_Name,Ship_Weapon,Weapon_Slot
5,c8537d688ad243b7a1c7d9f5c4c637c9,Styx-Class Heavy Cruiser,Data_HullSlots_Weapons_0_Type,Dorsal
38,c8537d688ad243b7a1c7d9f5c4c637c9,Styx-Class Heavy Cruiser,Data_HullSlots_Weapons_1_Type,Dorsal
71,c8537d688ad243b7a1c7d9f5c4c637c9,Styx-Class Heavy Cruiser,Data_HullSlots_Weapons_2_Type,Port
104,c8537d688ad243b7a1c7d9f5c4c637c9,Styx-Class Heavy Cruiser,Data_HullSlots_Weapons_3_Type,Port
137,c8537d688ad243b7a1c7d9f5c4c637c9,Styx-Class Heavy Cruiser,Data_HullSlots_Weapons_4_Type,Starboard
170,c8537d688ad243b7a1c7d9f5c4c637c9,Styx-Class Heavy Cruiser,Data_HullSlots_Weapons_5_Type,Starboard


Merge ```enemies_equipment``` and ```weapon_slots``` to add slot column for each ship weapon:

In [2072]:
# Use join() to merge by indexes
# english version
enemies_equipment = enemies_equipment.join(weapon_slots['Weapon_Slot'], how = 'left',
                                           lsuffix='_x', rsuffix='y')
#russian version
enemies_equipment_ru = enemies_equipment_ru.join(weapon_slots_ru['Weapon_Slot'], how = 'left',
                                           lsuffix='_x', rsuffix='y')

In [2073]:
enemies_equipment.head(10)

,AssetId,Ship_Name,Ship_Weapon,Ship_WeaponId,Weapon_Slot
0,817b866ad5504ecbbc38d4c61dcc415c,Aeldari Aconite,Data_HullSlots_Weapons_0_Weapon,211a20ba46b94eecb5bc0543cf912b13,Prow
1,148730e8be2a47f682d87c5f78602758,Aeldari Shadowhunter,Data_HullSlots_Weapons_0_Weapon,4e6b51451b2948ad9ecc97f14053e0c2,Prow
2,ddf557a81bca41f79c04b848ac4406bb,Murder-Class Cruiser,Data_HullSlots_Weapons_0_Weapon,8a0d0c07c7ea42eea126dbefe9dc7cfc,Prow
4,2074b0cda9de41f2a446483f98605215,Hellbringer-Class Light Cruiser,Data_HullSlots_Weapons_0_Weapon,657b911e90f943b582983866f43c2082,Dorsal
5,c8537d688ad243b7a1c7d9f5c4c637c9,Styx-Class Heavy Cruiser,Data_HullSlots_Weapons_0_Weapon,8a0d0c07c7ea42eea126dbefe9dc7cfc,Dorsal
6,a3d47366902443afb53a116ee40113cb,Blasphemous Styx-Class Heavy Cruiser,Data_HullSlots_Weapons_0_Weapon,8a0d0c07c7ea42eea126dbefe9dc7cfc,Dorsal
7,aa18013006b04f66aaaed4399e0d5e3b,Iconoclast-Class Destroyer,Data_HullSlots_Weapons_0_Weapon,657b911e90f943b582983866f43c2082,Dorsal
8,dbdc9cbf08e14a99be635907731ce5d2,Infidel-Class Raider,Data_HullSlots_Weapons_0_Weapon,f86db7c04c5946c18f0f3ce3d1042a0f,Prow
9,4f9fe1dac5614d98b7e2b38d2c684a7a,Swiftdeath Fighter,Data_HullSlots_Weapons_0_Weapon,d53283d69cb14ac9b0acadea1283aca5,Prow
10,1a213cfabe2b4a809e5462282f98b2ab,Bloodied Claw Cruiser,Data_HullSlots_Weapons_0_Weapon,5a561ea3c0284dc2aa42300d48289747,Dorsal


Merge ```ship_weapons_df``` with ```enemies_equipment```

In [2074]:
enemies_equipment = enemies_equipment.merge(ship_weapons_df[shipweapons_merge_columns], left_on=['Ship_WeaponId'],
                                            right_on=['AssetId'], how = 'left',
                                            suffixes=(None,'_y')).drop(columns = ['AssetId_y','Ship_Weapon', 'Ship_WeaponId'])

#russian version
enemies_equipment_ru = enemies_equipment_ru.merge(ship_weapons_df_ru[shipweapons_merge_columns], left_on=['Ship_WeaponId'],
                                            right_on=['AssetId'], how = 'left',
                                            suffixes=(None,'_y')).drop(columns = ['AssetId_y','Ship_Weapon', 'Ship_WeaponId'])

In [2075]:
enemies_equipment.query('Ship_Name=="Styx-Class Heavy Cruiser"').head()

,AssetId,Ship_Name,Weapon_Slot,Weapon_Name,Weapon_Descr,Data_WeaponType,Data_WeaponAbilities_Ability1_m_Ability_range,Data_m_DefaultAmmo_damage,Data_m_DefaultAmmo_max_damage,Data_DamageInstances,Data_m_DefaultAmmo_damage_type
4,c8537d688ad243b7a1c7d9f5c4c637c9,Styx-Class Heavy Cruiser,Dorsal,Starflare Lance,A long-ranged lance weapon dealing (from 55 to 65) damage to a chosen enemy within a narrow firing arc.,Lances,14,11,13,5,Energy
34,c8537d688ad243b7a1c7d9f5c4c637c9,Styx-Class Heavy Cruiser,Dorsal,Starflare Lance,A long-ranged lance weapon dealing (from 55 to 65) damage to a chosen enemy within a narrow firing arc.,Lances,14,11,13,5,Energy
57,c8537d688ad243b7a1c7d9f5c4c637c9,Styx-Class Heavy Cruiser,Port,Hecutor-Pattern Plasma Battery,"Long-range broadside plasma macro-cannons, which fire 3 shots, dealing 40 damage per shot.\n.",Macrobatteries,9,38,0,3,Energy
72,c8537d688ad243b7a1c7d9f5c4c637c9,Styx-Class Heavy Cruiser,Port,Hecutor-Pattern Plasma Battery,"Long-range broadside plasma macro-cannons, which fire 3 shots, dealing 40 damage per shot.\n.",Macrobatteries,9,38,0,3,Energy
79,c8537d688ad243b7a1c7d9f5c4c637c9,Styx-Class Heavy Cruiser,Starboard,Hecutor-Pattern Plasma Battery,"Long-range broadside plasma macro-cannons, which fire 3 shots, dealing 40 damage per shot.\n.",Macrobatteries,9,38,0,3,Energy


Merge ```enemy_stats``` with ```enemies_equipment``` to create comlete table:

In [2076]:
#english version
enemies_stats = enemies_stats.merge(
                                    enemies_equipment, on = 'AssetId', suffixes=(None,'_y')
                                    ).drop(columns = ['Ship_Name_y'], axis = 1)
#russian version
enemies_stats_ru = enemies_stats_ru.merge(
                                    enemies_equipment_ru, on = 'AssetId', suffixes=(None,'_y')
                                    ).drop(columns = ['Ship_Name_y'], axis = 1)

In [2077]:
enemies_equipment.query('Ship_Name=="Cobra-Class Destroyer"').head()

,AssetId,Ship_Name,Weapon_Slot,Weapon_Name,Weapon_Descr,Data_WeaponType,Data_WeaponAbilities_Ability1_m_Ability_range,Data_m_DefaultAmmo_damage,Data_m_DefaultAmmo_max_damage,Data_DamageInstances,Data_m_DefaultAmmo_damage_type
27,f9e619867a7c4a67b574d346d10cffa1,Cobra-Class Destroyer,Dorsal,Thunderstrike Macro-Cannons,"Short-range dorsal macro-cannons, which fire 2 shots, dealing 21 damage per shot.",Macrobatteries,6,21,0,2,Impact
52,f9e619867a7c4a67b574d346d10cffa1,Cobra-Class Destroyer,Dorsal,Thunderstrike Macro-Cannons,"Short-range dorsal macro-cannons, which fire 2 shots, dealing 21 damage per shot.",Macrobatteries,6,21,0,2,Impact


In [2078]:
enemies_stats_ru[enemies_stats_ru['Ship_Name']=='Тяжелый крейсер класса "Стикс"']

,AssetId,Ship_Name,Data_DifficultyType,Data_MilitaryRating,Data_StarshipSpeed,Data_Initiative,Data_Evasion,Data_HullIntegrity,Data_ArmourAft,Data_ArmourFore,Data_ArmourPort,Data_ArmourStarboard,Data_HullSlots_m_AugerArray_hitChance,Data_HullSlots_m_AugerArray_critChance,Shields_Stats,Data_AiDesiredDistanceToPlayer,Data_Components_1_Rewards_0_Scrap,Data_Size,Weapon_Slot,Weapon_Name,Weapon_Descr,Data_WeaponType,Data_WeaponAbilities_Ability1_m_Ability_range,Data_m_DefaultAmmo_damage,Data_m_DefaultAmmo_max_damage,Data_DamageInstances,Data_m_DefaultAmmo_damage_type
13,c8537d688ad243b7a1c7d9f5c4c637c9,"Тяжелый крейсер класса ""Стикс""",MiniBoss,3,4,20,0,380,12,12,12,12,70,15,"Нос: 225, Левый борт: 225, Правый борт: 225, Корма: 225,",4.0,122,GrandCruiser_3x6,Dorsal,"Лэнс-излучатель ""Звездная вспышка""","Дальнобойный лэнс-излучатель, который наносит (от 55 до 65) урона выбранному врагу в узком секторе стрельбы.",Lances,14,11,13,5,Energy
14,c8537d688ad243b7a1c7d9f5c4c637c9,"Тяжелый крейсер класса ""Стикс""",MiniBoss,3,4,20,0,380,12,12,12,12,70,15,"Нос: 225, Левый борт: 225, Правый борт: 225, Корма: 225,",4.0,122,GrandCruiser_3x6,Dorsal,"Лэнс-излучатель ""Звездная вспышка""","Дальнобойный лэнс-излучатель, который наносит (от 55 до 65) урона выбранному врагу в узком секторе стрельбы.",Lances,14,11,13,5,Energy
15,c8537d688ad243b7a1c7d9f5c4c637c9,"Тяжелый крейсер класса ""Стикс""",MiniBoss,3,4,20,0,380,12,12,12,12,70,15,"Нос: 225, Левый борт: 225, Правый борт: 225, Корма: 225,",4.0,122,GrandCruiser_3x6,Port,"Плазменная батарея ""Гекутор""","Дальнобойные бортовые макропушки с 3 плазменными зарядами, которые наносят по 40 урона.",Macrobatteries,9,38,0,3,Energy
16,c8537d688ad243b7a1c7d9f5c4c637c9,"Тяжелый крейсер класса ""Стикс""",MiniBoss,3,4,20,0,380,12,12,12,12,70,15,"Нос: 225, Левый борт: 225, Правый борт: 225, Корма: 225,",4.0,122,GrandCruiser_3x6,Port,"Плазменная батарея ""Гекутор""","Дальнобойные бортовые макропушки с 3 плазменными зарядами, которые наносят по 40 урона.",Macrobatteries,9,38,0,3,Energy
17,c8537d688ad243b7a1c7d9f5c4c637c9,"Тяжелый крейсер класса ""Стикс""",MiniBoss,3,4,20,0,380,12,12,12,12,70,15,"Нос: 225, Левый борт: 225, Правый борт: 225, Корма: 225,",4.0,122,GrandCruiser_3x6,Starboard,"Плазменная батарея ""Гекутор""","Дальнобойные бортовые макропушки с 3 плазменными зарядами, которые наносят по 40 урона.",Macrobatteries,9,38,0,3,Energy
18,c8537d688ad243b7a1c7d9f5c4c637c9,"Тяжелый крейсер класса ""Стикс""",MiniBoss,3,4,20,0,380,12,12,12,12,70,15,"Нос: 225, Левый борт: 225, Правый борт: 225, Корма: 225,",4.0,122,GrandCruiser_3x6,Starboard,"Плазменная батарея ""Гекутор""","Дальнобойные бортовые макропушки с 3 плазменными зарядами, которые наносят по 40 урона.",Macrobatteries,9,38,0,3,Energy


Cerate column with good shields against type of damage:

map_dict = {'Energy': ','.join(list_shields_vs_energy),
            'Impact': ','.join(list_shields_vs_kinetic)}
map_dict_ru = {'Energy': ','.join(list_shields_vs_energy_ru),
            'Impact': ','.join(list_shields_vs_kinetic_ru)}
enemies_stats['Shields_Resist_Damage'] = enemies_stats['Data_m_DefaultAmmo_damage_type'].map(map_dict)
enemies_stats_ru['Shields_Resist_Damage'] = enemies_stats_ru['Data_m_DefaultAmmo_damage_type'].map(map_dict_ru)
enemies_stats_ru.head()

## Format output table

In [2079]:
#drop AssetId column
enemies_stats = enemies_stats.drop(columns = ['AssetId'])
enemies_stats_ru = enemies_stats_ru.drop(columns = ['AssetId'])

Create dictionary with column names as keys and output column names as values:

In [2080]:
# create dictionary with column names for renaming
#column_names_dict = dict(zip(enemies_stats.columns.tolist(), enemies_stats.columns.tolist()))

In [2081]:
#dict used for mapping new names to old names
column_name_transcription = {'Ship_Name': 'Ship_Name',
 'Data_DifficultyType': 'Difficulty_Type',
 'Data_MilitaryRating': 'Military_Rating', 
 'Data_StarshipSpeed': 'Starship_Speed',
 'Data_Initiative': 'Initiative',
 'Data_Evasion': 'Evasion',
 'Data_HullIntegrity': 'Hull_Integrity',
 'Data_ArmourAft': 'Armour_Aft',
 'Data_ArmourFore': 'Armour_Fore',
 'Data_ArmourPort': 'Armour_Port',
 'Data_ArmourStarboard': 'Armour_Starboard',
 'Data_HullSlots_m_AugerArray_hitChance': 'Hit_Chance',
 'Data_HullSlots_m_AugerArray_critChance': 'Crit_Chance',
 'Shields_Stats': 'Shields_Stats',
 'Data_AiDesiredDistanceToPlayer': 'Enemy_Desired_DistanceTo_Player',
 'Data_Components_1_Rewards_0_Scrap': 'Scrap_Reward',
 'Data_Size': 'Size',
 'Weapon_Slot': 'Weapon_Slot',
 'Weapon_Name': 'Weapon_Name',
 'Weapon_Descr': 'Weapon_Descr',
 'Data_WeaponType': 'Weapon_Type',
 'Data_WeaponAbilities_Ability1_m_Ability_range': 'Ability_Range', 
 'Data_m_DefaultAmmo_damage': 'Ammo_Damage',
 'Data_m_DefaultAmmo_max_damage': 'Ammo_Max_Damage',
 'Data_DamageInstances': 'Damage_Instances',
 'Data_m_DefaultAmmo_damage_type': 'Ammo_Damage_Type',
 #'Shields_Resist_Damage': 'Shields_Resist_Damage'
 }
enemies_stats = enemies_stats.rename(columns = column_name_transcription)
enemies_stats_ru = enemies_stats_ru.rename(columns = column_name_transcription)
display(enemies_stats.head(1), enemies_stats_ru.head(1))

,Ship_Name,Difficulty_Type,Military_Rating,Starship_Speed,Initiative,Evasion,Hull_Integrity,Armour_Aft,Armour_Fore,Armour_Port,Armour_Starboard,Hit_Chance,Crit_Chance,Shields_Stats,Enemy_Desired_DistanceTo_Player,Scrap_Reward,Size,Weapon_Slot,Weapon_Name,Weapon_Descr,Weapon_Type,Ability_Range,Ammo_Damage,Ammo_Max_Damage,Damage_Instances,Ammo_Damage_Type
0,Aeldari Aconite,Hard,3,16,40,25,146,4,4,4,4,95,5,"Fore: 0, Port: 0, Starboard: 0, Aft: 0,",6.0,33,Frigate_1x2,Prow,Pulsar Lance,A long-ranged lance weapon dealing 84 damage to a chosen enemy within a narrow firing arc.,Lances,15,28,0,3,Energy


,Ship_Name,Difficulty_Type,Military_Rating,Starship_Speed,Initiative,Evasion,Hull_Integrity,Armour_Aft,Armour_Fore,Armour_Port,Armour_Starboard,Hit_Chance,Crit_Chance,Shields_Stats,Enemy_Desired_DistanceTo_Player,Scrap_Reward,Size,Weapon_Slot,Weapon_Name,Weapon_Descr,Weapon_Type,Ability_Range,Ammo_Damage,Ammo_Max_Damage,Damage_Instances,Ammo_Damage_Type
0,"Эльдарский фрегат класса ""Аконит""",Hard,3,16,40,25,146,4,4,4,4,95,5,"Нос: 0, Левый борт: 0, Правый борт: 0, Корма: 0,",6.0,33,Frigate_1x2,Prow,Пульсарное копье,"Дальнобойный лэнс-излучатель, который наносит 64 урона выбранному врагу в узком секторе стрельбы.",Lances,15,28,0,3,Energy


Replace duplicate values for empty rows in the subset of columns for a better look:  
Variant 1:

In [2082]:
# create columns-separators
enemies_stats['Ship_Stats'] = 'Other stats: '
enemies_stats['Weapons'] = 'Weapons:'

column_order = [ #intro columns
                'Ship_Name','Difficulty_Type',
                #column-sparator
                'Weapons',      
                # weapon columns
                'Weapon_Slot', 'Weapon_Name', 'Weapon_Descr',	'Weapon_Type',	'Ability_Range',
                'Ammo_Damage',	'Ammo_Max_Damage',	'Damage_Instances',	'Ammo_Damage_Type',
                # column-separator
                'Ship_Stats', 
                # ship stats columns                 
                'Starship_Speed',	'Initiative',	'Evasion',	'Hull_Integrity',
                'Armour_Aft',	'Armour_Fore',	'Armour_Port',	'Armour_Starboard',
        	'Hit_Chance',	'Crit_Chance',	'Shields_Stats',	'Enemy_Desired_DistanceTo_Player',
            	'Scrap_Reward',	'Size'	]
enemies_stats = enemies_stats[column_order]
#group of columns with duplicate values
columns = ['Ship_Name',	'Difficulty_Type', 'Weapons', 'Starship_Speed',
'Initiative',	'Evasion',	'Hull_Integrity',	'Armour_Aft',	'Armour_Fore',
'Armour_Port',	'Armour_Starboard',	'Hit_Chance',	'Crit_Chance',
'Shields_Stats','Enemy_Desired_DistanceTo_Player',	'Scrap_Reward','Size','Ship_Stats']
enemies_stats.loc[enemies_stats.duplicated(subset = columns),columns] = ""
display(enemies_stats.head())

,Ship_Name,Difficulty_Type,Weapons,Weapon_Slot,Weapon_Name,Weapon_Descr,Weapon_Type,Ability_Range,Ammo_Damage,Ammo_Max_Damage,Damage_Instances,Ammo_Damage_Type,Ship_Stats,Starship_Speed,Initiative,Evasion,Hull_Integrity,Armour_Aft,Armour_Fore,Armour_Port,Armour_Starboard,Hit_Chance,Crit_Chance,Shields_Stats,Enemy_Desired_DistanceTo_Player,Scrap_Reward,Size
0,Aeldari Aconite,Hard,Weapons:,Prow,Pulsar Lance,A long-ranged lance weapon dealing 84 damage to a chosen enemy within a narrow firing arc.,Lances,15,28,0,3,Energy,Other stats:,16,40,25,146,4,4,4,4,95,5,"Fore: 0, Port: 0, Starboard: 0, Aft: 0,",6.0,33,Frigate_1x2
1,,,,Prow,Aspect Torpedo Tubes,Torpedo tubes capable of firing 5 neutron torpedoes per salvo.\nReload: 1 round,TorpedoTubes,15,24,0,1,Energy,,,,,,,,,,,,,,,
2,Aeldari Shadowhunter,Common,Weapons:,Prow,Shadow Lance,A long-ranged lance weapon dealing 63 damage to a chosen enemy within a narrow firing arc.,Lances,10,21,0,3,Energy,Other stats:,12,90,35,55,0,0,0,0,90,5,"Fore: 0, Port: 0, Starboard: 0, Aft: 0,",6.0,15,Raider_1x1
3,,,,Prow,Aeldari Torpedo Tubes,Torpedo tubes capable of firing 5 neutron torpedoes per salvo.\nReload: 2 rounds,TorpedoTubes,12,24,0,1,Energy,,,,,,,,,,,,,,,
4,Murder-Class Cruiser,Elite,Weapons:,Prow,Starflare Lance,A long-ranged lance weapon dealing (from 55 to 65) damage to a chosen enemy within a narrow firing arc.,Lances,14,11,13,5,Energy,Other stats:,7,30,0,180,8,8,8,8,70,15,"Fore: 105, Port: 105, Starboard: 105, Aft: 105,",4.0,57,Cruiser_2x4


Variant 2:

#enemies_stats['Weapons'] = 'Weapons:'
#insert new column at position 17, that divide row in ship stats part and ship weapons part:
enemies_stats.insert(loc = 17,column= 'Weapons', value = 'Weapons:')
#column group wih duplicate values
columns = ['Ship_Name',	'Difficulty_Type',	'Military_Rating',	'Starship_Speed',
'Initiative',	'Evasion',	'Hull_Integrity',	'Armour_Aft',	'Armour_Fore',
'Armour_Port',	'Armour_Starboard',	'Hit_Chance',	'Crit_Chance',
'Shields_Stats','Enemy_Desired_DistanceTo_Player',	'Scrap_Reward','Size','Weapons']
#replace duplicated values in the subset of columns of "enemy_stats" for ""
enemies_stats.loc[enemies_stats.duplicated(subset = columns),columns] = ""
display(enemies_stats.head())
## an obsolete variant
#enemies_stats = enemies_stats.set_index(columns)

Final looks:

In [2083]:
enemies_stats_table.head()

,Ship_Name,Difficulty_Type,Weapon_Slot,Weapon_Name,Weapon_Descr,Weapon_Type,Ability_Range,Ammo_Damage,Ammo_Max_Damage,Damage_Instances,Ammo_Damage_Type,Starship_Speed,Initiative,Evasion,Hull_Integrity,Armour_Aft,Armour_Fore,Armour_Port,Armour_Starboard,Hit_Chance,Crit_Chance,Shields_Stats,Enemy_Desired_DistanceTo_Player,Scrap_Reward,Size
0,,,,,,,,,,,,,,,,,,,,,,,,,
1,Aeldari Aconite,Hard,Prow,,,,,,,,,16,40,25,146,4,4,4,4,95,5,"Fore: 0, Port: 0, Starboard: 0, Aft: 0,",6.0,33,Frigate_1x2
2,,,Prow,,,,,,,,,,,,,,,,,,5,"Fore: 0, Port: 0, Starboard: 0, Aft: 0,",6.0,,
3,Aeldari Shadowhunter,Common,Prow,,,,,,,,,12,90,35,55,0,0,0,0,90,5,"Fore: 0, Port: 0, Starboard: 0, Aft: 0,",6.0,15,Raider_1x1
4,,,Prow,,,,,,,,,,,,,,,,,,,,,,


### Russian version

In [2084]:
enemies_stats_ru.head(2)

,Ship_Name,Difficulty_Type,Military_Rating,Starship_Speed,Initiative,Evasion,Hull_Integrity,Armour_Aft,Armour_Fore,Armour_Port,Armour_Starboard,Hit_Chance,Crit_Chance,Shields_Stats,Enemy_Desired_DistanceTo_Player,Scrap_Reward,Size,Weapon_Slot,Weapon_Name,Weapon_Descr,Weapon_Type,Ability_Range,Ammo_Damage,Ammo_Max_Damage,Damage_Instances,Ammo_Damage_Type
0,"Эльдарский фрегат класса ""Аконит""",Hard,3,16,40,25,146,4,4,4,4,95,5,"Нос: 0, Левый борт: 0, Правый борт: 0, Корма: 0,",6.0,33,Frigate_1x2,Prow,Пульсарное копье,"Дальнобойный лэнс-излучатель, который наносит 64 урона выбранному врагу в узком секторе стрельбы.",Lances,15,28,0,3,Energy
1,"Эльдарский фрегат класса ""Аконит""",Hard,3,16,40,25,146,4,4,4,4,95,5,"Нос: 0, Левый борт: 0, Правый борт: 0, Корма: 0,",6.0,33,Frigate_1x2,Prow,Аспектный торпедный аппарат,"Аппарат, выпускающий 5 нейтронных торпед за залп.\nПерезарядка: 1 раунд",TorpedoTubes,15,24,0,1,Energy


Renaming columns:

In [2085]:
#d = dict(zip(enemies_stats_ru.columns.tolist(), enemies_stats_ru.columns.tolist()))

In [2086]:
# create columns-separators
enemies_stats_ru['Ship_Stats'] = 'Характеристики: '
enemies_stats_ru['Weapons'] = 'Вооружение:'

column_order = [ #intro columns
                'Ship_Name','Difficulty_Type',
                #column-sparator
                'Weapons',      
                # weapon columns
                'Weapon_Slot', 'Weapon_Name', 'Weapon_Descr',	'Weapon_Type',	'Ability_Range',
                'Ammo_Damage',	'Ammo_Max_Damage',	'Damage_Instances',	'Ammo_Damage_Type',
                # column-separator
                'Ship_Stats', 
                # ship stats columns                 
                'Starship_Speed',	'Initiative',	'Evasion',	'Hull_Integrity',
                'Armour_Aft',	'Armour_Fore',	'Armour_Port',	'Armour_Starboard',
        	'Hit_Chance',	'Crit_Chance',	'Shields_Stats',	'Enemy_Desired_DistanceTo_Player',
            	'Scrap_Reward',	'Size'	]
enemies_stats_ru = enemies_stats_ru[column_order]
#group of columns with duplicate values
columns = ['Ship_Name',	'Difficulty_Type', 'Weapons', 'Starship_Speed',
'Initiative',	'Evasion',	'Hull_Integrity',	'Armour_Aft',	'Armour_Fore',
'Armour_Port',	'Armour_Starboard',	'Hit_Chance',	'Crit_Chance',
'Shields_Stats','Enemy_Desired_DistanceTo_Player',	'Scrap_Reward','Size','Ship_Stats']
enemies_stats_ru.loc[enemies_stats_ru.duplicated(subset = columns),columns] = ""
display(enemies_stats_ru.head())

,Ship_Name,Difficulty_Type,Weapons,Weapon_Slot,Weapon_Name,Weapon_Descr,Weapon_Type,Ability_Range,Ammo_Damage,Ammo_Max_Damage,Damage_Instances,Ammo_Damage_Type,Ship_Stats,Starship_Speed,Initiative,Evasion,Hull_Integrity,Armour_Aft,Armour_Fore,Armour_Port,Armour_Starboard,Hit_Chance,Crit_Chance,Shields_Stats,Enemy_Desired_DistanceTo_Player,Scrap_Reward,Size
0,"Эльдарский фрегат класса ""Аконит""",Hard,Вооружение:,Prow,Пульсарное копье,"Дальнобойный лэнс-излучатель, который наносит 64 урона выбранному врагу в узком секторе стрельбы.",Lances,15,28,0,3,Energy,Характеристики:,16,40,25,146,4,4,4,4,95,5,"Нос: 0, Левый борт: 0, Правый борт: 0, Корма: 0,",6.0,33,Frigate_1x2
1,,,,Prow,Аспектный торпедный аппарат,"Аппарат, выпускающий 5 нейтронных торпед за залп.\nПерезарядка: 1 раунд",TorpedoTubes,15,24,0,1,Energy,,,,,,,,,,,,,,,
2,"Эльдарский корабль класса ""Теневой охотник""",Common,Вооружение:,Prow,Теневое копье,"Дальнобойный лэнс-излучатель, который наносит 25 урона выбранному врагу в узком секторе стрельбы.",Lances,10,21,0,3,Energy,Характеристики:,12,90,35,55,0,0,0,0,90,5,"Нос: 0, Левый борт: 0, Правый борт: 0, Корма: 0,",6.0,15,Raider_1x1
3,,,,Prow,Эльдарский торпедный аппарат,"Аппарат, выпускающий 5 нейтронных торпед за залп.\nПерезарядка: 2 раунда",TorpedoTubes,12,24,0,1,Energy,,,,,,,,,,,,,,,
4,"Крейсер класса ""Резня""",Elite,Вооружение:,Prow,"Лэнс-излучатель ""Звездная вспышка""","Дальнобойный лэнс-излучатель, который наносит (от 55 до 65) урона выбранному врагу в узком секторе стрельбы.",Lances,14,11,13,5,Energy,Характеристики:,7,30,0,180,8,8,8,8,70,15,"Нос: 105, Левый борт: 105, Правый борт: 105, Корма: 105,",4.0,57,Cruiser_2x4


In [2087]:
column_name_transcription_ru = {'Ship_Name': 'Название',
 'Difficulty_Type': 'Сложность',
 #'Military_Rating': 'Военный_Рейтинг',
 'Ship_Stats':'Характеристики',
 'Starship_Speed': 'Скорость',
 'Initiative': 'Инициатива',
 'Evasion': 'Уклонение',
 'Hull_Integrity': 'Целостность_Корпуса',
 'Armour_Aft': 'Броня_Корма',
 'Armour_Fore': 'Броня_Нос',
 'Armour_Port': 'Броня_ЛевыйБорт',
 'Armour_Starboard': 'Броня_ПравыйБорт',
 'Hit_Chance': 'Шанс_Попадания',
 'Crit_Chance': 'Шанс_КритПопадания',
 'Shields_Stats': 'Щиты',
 'Enemy_Desired_DistanceTo_Player': 'Желаемая_ДистанцияДо_Игрока',
 'Scrap_Reward': 'Награда_Металлолом',
 'Size': 'Размер',
 'Weapons':'Вооружение',
 'Weapon_Slot': 'Позиция_Оружия',
 'Weapon_Name': 'Название_Оружия',
 'Weapon_Descr': 'Описание_Оружия',
 'Weapon_Type': 'Тип_Оружия',
 'Ability_Range': 'Дальность_Оружия',
 'Ammo_Damage': 'Урон_Снаряда',
 'Ammo_Max_Damage': 'Макс_Урон_Снаряда',
 'Damage_Instances': 'Количество_Залпов',
 'Ammo_Damage_Type': 'Снаряд_Тип_Урона',
 #'Shields_Resist_Damage': 'Щиты_С_Бонусом_К_ТипУрона'
 }
enemies_stats_ru = enemies_stats_ru.rename(columns = column_name_transcription_ru)


Replacing values:

In [2088]:
enemies_stats_ru['Позиция_Оружия'].unique()
enemies_stats_ru['Позиция_Оружия'] = enemies_stats_ru['Позиция_Оружия'].replace({'Prow':'Нос',
                                                                                 'Port':'Левый борт',
                                                                                 'Starboard':'Правый борт',
                                                                                 'Dorsal':'Башня'})
enemies_stats_ru['Снаряд_Тип_Урона'] = enemies_stats_ru['Снаряд_Тип_Урона'].replace({'Energy':'Энергетический',                                                                                 
                                                                                 'Impact':'Кинетический'})


In [2089]:
enemies_stats_ru.head()

,Название,Сложность,Вооружение,Позиция_Оружия,Название_Оружия,Описание_Оружия,Тип_Оружия,Дальность_Оружия,Урон_Снаряда,Макс_Урон_Снаряда,Количество_Залпов,Снаряд_Тип_Урона,Характеристики,Скорость,Инициатива,Уклонение,Целостность_Корпуса,Броня_Корма,Броня_Нос,Броня_ЛевыйБорт,Броня_ПравыйБорт,Шанс_Попадания,Шанс_КритПопадания,Щиты,Желаемая_ДистанцияДо_Игрока,Награда_Металлолом,Размер
0,"Эльдарский фрегат класса ""Аконит""",Hard,Вооружение:,Нос,Пульсарное копье,"Дальнобойный лэнс-излучатель, который наносит 64 урона выбранному врагу в узком секторе стрельбы.",Lances,15,28,0,3,Энергетический,Характеристики:,16,40,25,146,4,4,4,4,95,5,"Нос: 0, Левый борт: 0, Правый борт: 0, Корма: 0,",6.0,33,Frigate_1x2
1,,,,Нос,Аспектный торпедный аппарат,"Аппарат, выпускающий 5 нейтронных торпед за залп.\nПерезарядка: 1 раунд",TorpedoTubes,15,24,0,1,Энергетический,,,,,,,,,,,,,,,
2,"Эльдарский корабль класса ""Теневой охотник""",Common,Вооружение:,Нос,Теневое копье,"Дальнобойный лэнс-излучатель, который наносит 25 урона выбранному врагу в узком секторе стрельбы.",Lances,10,21,0,3,Энергетический,Характеристики:,12,90,35,55,0,0,0,0,90,5,"Нос: 0, Левый борт: 0, Правый борт: 0, Корма: 0,",6.0,15,Raider_1x1
3,,,,Нос,Эльдарский торпедный аппарат,"Аппарат, выпускающий 5 нейтронных торпед за залп.\nПерезарядка: 2 раунда",TorpedoTubes,12,24,0,1,Энергетический,,,,,,,,,,,,,,,
4,"Крейсер класса ""Резня""",Elite,Вооружение:,Нос,"Лэнс-излучатель ""Звездная вспышка""","Дальнобойный лэнс-излучатель, который наносит (от 55 до 65) урона выбранному врагу в узком секторе стрельбы.",Lances,14,11,13,5,Энергетический,Характеристики:,7,30,0,180,8,8,8,8,70,15,"Нос: 105, Левый борт: 105, Правый борт: 105, Корма: 105,",4.0,57,Cruiser_2x4


## Variant 2 style
# insert column at position 17
enemies_stats_ru.insert(loc = 17,column= 'Вооружение', value = 'Вооружение:')
# list of columns with duplicate values
columns = ['Название',	'Сложность',	'Военный_Рейтинг',	'Скорость',
'Инициатива',	'Уклонение',	'Целостность_Корпуса',	'Броня_Корма',	'Броня_Нос',
'Броня_ЛевыйБорт',	'Броня_ПравыйБорт',	'Шанс_Попадания',	'Шанс_КритПопадания',
'Щиты','Желаемая_ДистанцияДо_Игрока',	'Награда_Металлолом','Размер','Вооружение']
#replace duplicated values in the subset of columns of "enemy_stats" for ""
enemies_stats_ru.loc[enemies_stats_ru.duplicated(subset = columns),columns] = ""
display(enemies_stats_ru.head())

### Final outputs

In [2090]:
display(enemies_stats.head())

,Ship_Name,Difficulty_Type,Weapons,Weapon_Slot,Weapon_Name,Weapon_Descr,Weapon_Type,Ability_Range,Ammo_Damage,Ammo_Max_Damage,Damage_Instances,Ammo_Damage_Type,Ship_Stats,Starship_Speed,Initiative,Evasion,Hull_Integrity,Armour_Aft,Armour_Fore,Armour_Port,Armour_Starboard,Hit_Chance,Crit_Chance,Shields_Stats,Enemy_Desired_DistanceTo_Player,Scrap_Reward,Size
0,Aeldari Aconite,Hard,Weapons:,Prow,Pulsar Lance,A long-ranged lance weapon dealing 84 damage to a chosen enemy within a narrow firing arc.,Lances,15,28,0,3,Energy,Other stats:,16,40,25,146,4,4,4,4,95,5,"Fore: 0, Port: 0, Starboard: 0, Aft: 0,",6.0,33,Frigate_1x2
1,,,,Prow,Aspect Torpedo Tubes,Torpedo tubes capable of firing 5 neutron torpedoes per salvo.\nReload: 1 round,TorpedoTubes,15,24,0,1,Energy,,,,,,,,,,,,,,,
2,Aeldari Shadowhunter,Common,Weapons:,Prow,Shadow Lance,A long-ranged lance weapon dealing 63 damage to a chosen enemy within a narrow firing arc.,Lances,10,21,0,3,Energy,Other stats:,12,90,35,55,0,0,0,0,90,5,"Fore: 0, Port: 0, Starboard: 0, Aft: 0,",6.0,15,Raider_1x1
3,,,,Prow,Aeldari Torpedo Tubes,Torpedo tubes capable of firing 5 neutron torpedoes per salvo.\nReload: 2 rounds,TorpedoTubes,12,24,0,1,Energy,,,,,,,,,,,,,,,
4,Murder-Class Cruiser,Elite,Weapons:,Prow,Starflare Lance,A long-ranged lance weapon dealing (from 55 to 65) damage to a chosen enemy within a narrow firing arc.,Lances,14,11,13,5,Energy,Other stats:,7,30,0,180,8,8,8,8,70,15,"Fore: 105, Port: 105, Starboard: 105, Aft: 105,",4.0,57,Cruiser_2x4


In [2091]:
display(enemies_stats_ru.head())

,Название,Сложность,Вооружение,Позиция_Оружия,Название_Оружия,Описание_Оружия,Тип_Оружия,Дальность_Оружия,Урон_Снаряда,Макс_Урон_Снаряда,Количество_Залпов,Снаряд_Тип_Урона,Характеристики,Скорость,Инициатива,Уклонение,Целостность_Корпуса,Броня_Корма,Броня_Нос,Броня_ЛевыйБорт,Броня_ПравыйБорт,Шанс_Попадания,Шанс_КритПопадания,Щиты,Желаемая_ДистанцияДо_Игрока,Награда_Металлолом,Размер
0,"Эльдарский фрегат класса ""Аконит""",Hard,Вооружение:,Нос,Пульсарное копье,"Дальнобойный лэнс-излучатель, который наносит 64 урона выбранному врагу в узком секторе стрельбы.",Lances,15,28,0,3,Энергетический,Характеристики:,16,40,25,146,4,4,4,4,95,5,"Нос: 0, Левый борт: 0, Правый борт: 0, Корма: 0,",6.0,33,Frigate_1x2
1,,,,Нос,Аспектный торпедный аппарат,"Аппарат, выпускающий 5 нейтронных торпед за залп.\nПерезарядка: 1 раунд",TorpedoTubes,15,24,0,1,Энергетический,,,,,,,,,,,,,,,
2,"Эльдарский корабль класса ""Теневой охотник""",Common,Вооружение:,Нос,Теневое копье,"Дальнобойный лэнс-излучатель, который наносит 25 урона выбранному врагу в узком секторе стрельбы.",Lances,10,21,0,3,Энергетический,Характеристики:,12,90,35,55,0,0,0,0,90,5,"Нос: 0, Левый борт: 0, Правый борт: 0, Корма: 0,",6.0,15,Raider_1x1
3,,,,Нос,Эльдарский торпедный аппарат,"Аппарат, выпускающий 5 нейтронных торпед за залп.\nПерезарядка: 2 раунда",TorpedoTubes,12,24,0,1,Энергетический,,,,,,,,,,,,,,,
4,"Крейсер класса ""Резня""",Elite,Вооружение:,Нос,"Лэнс-излучатель ""Звездная вспышка""","Дальнобойный лэнс-излучатель, который наносит (от 55 до 65) урона выбранному врагу в узком секторе стрельбы.",Lances,14,11,13,5,Энергетический,Характеристики:,7,30,0,180,8,8,8,8,70,15,"Нос: 105, Левый борт: 105, Правый борт: 105, Корма: 105,",4.0,57,Cruiser_2x4


## Appendix

### Service cells

English version:

In [2092]:
# search text by strinkey 7dd797bddb414f7684c7608bd7f1943f
index_df.query('index == "54dcfdea-73ef-4bb8-9f94-f75ffcfb403d"')['text']

16875    Compound Void Shield Array
Name: text, dtype: object

In [2093]:
#search filepath by file's asset id 
assets_df.query('AssetId=="1f4ab5b04c584ca89e6d15840486a0a6"')['File']

15845    e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Units\Enemies\Chaos\Chaos_Cruiser5_Shield.jbp
Name: File, dtype: object

Finding a particular value in dataframe:

In [2094]:
ship_weapons_df[ship_weapons_df.eq("Stare into the Abyss").any(axis = 1)]

,AssetId,Data_AllowedSlots_0,Data_AllowedSlots_1,Data_AllowedSlots_2,Data_Components_0_$type,Data_Components_0_m_Fact,Data_Components_0_name,Data_DamageInstances,Data_DelayBetweenProjectiles,Data_DelayInSeries,Data_RateOfFire,Data_RemoveFromSlotWhenNoCharges,Data_ShotsInSeries,Data_WeaponAbilities_Ability1_Type,Data_WeaponAbilities_Ability1_m_Ability,Data_WeaponAbilities_Ability2_Type,Data_WeaponAbilities_Ability2_m_Ability,Data_WeaponType,Data_m_AlternateAmmo,Data_m_CargoVolumePercent,Data_m_DefaultAmmo,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey,Data_m_Origin,Data_m_Overrides_0,Data_m_Overrides_1,Data_m_Overrides_10,Data_m_Overrides_2,Data_m_Overrides_3,Data_m_Overrides_4,Data_m_Overrides_5,Data_m_Overrides_6,Data_m_Overrides_7,Data_m_Overrides_8,Data_m_Overrides_9,Data_m_ProfitFactorCost,Filepath,Weapon_Name,Weapon_Descr,Data_WeaponAbilities_Ability1_m_Ability_text,Data_WeaponAbilities_Ability2_m_Ability_text,Data_m_DefaultAmmo_text,Data_m_DefaultAmmo_damage,Data_m_AlternateAmmo_damage,Data_m_DefaultAmmo_max_damage,Data_m_AlternateAmmo_max_damage,Data_m_DefaultAmmo_damage_type,Data_m_AlternateAmmo_damage_type,Data_m_DefaultAmmo_proj_id,Data_m_AlternateAmmo_proj_id,Data_WeaponAbilities_Ability1_m_Ability_range,Data_WeaponAbilities_Ability2_m_Ability_range
425,ccdfe40b2aa04247ae9a102aa9078706,Prow,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN,0.0,False,NaN,Custom,40b0baf127574b21a3c8b6a6d67ead74,None,40b0baf127574b21a3c8b6a6d67ead74,Other,NaN,10.0,NaN,,,af4f82c1-84fd-4133-a741-4a41d0f006f4,,,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Spacecombat\Units\Enemies\Chaos\ChaosStarship_StareToAbyss_Weapon.jbp,Stare into the Abyss,,Stare into the Abyss,Stare into the Abyss,,,,,,,,,,14,14


Russian version:

In [2095]:
index_df_ru.query('index == "b3ed026f-d2d0-40cb-a54d-85d1c7f63507"')['text']

36273    В Просторе Коронус нет более достойно{mf|го|й} опекун{mf|а|ши} для обители веры, чем глава династии фон Валанциус. В столице мира в е{mf|го|е} честь будет возведена великолепная базилика, украшенная грандиозной парной скульптурой Императора и Вольного Торговца. Тысячи прихожан и жрецов будут пом...
Name: text, dtype: object

### Saving

In [2096]:
enemies_stats.to_excel(save_folder + r'\enemy_stats_eng_test.xlsx', index = False)
enemies_stats_ru.to_excel(save_folder + r'\enemy_stats_ru_test.xlsx', index = False)
enemies_stats.to_csv(save_folder + r'\enemyships_stats_eng.csv', index = False)
enemies_stats_ru.to_csv(save_folder + r'\enemyships_stats_ru.csv', index = False)

